<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/EXP_Gold__1_2%2C_3_%26_4_20251027.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install pandas
!pip install python-gedcom
!pip install openpyxl
!pip install xlsxwriter
!pip install mlxtend
!pip caas_jupyter_tools

ERROR: unknown command "caas_jupyter_tools"


In [21]:
#credentials

import os

# Gmail SMTP creds
os.environ['GMAIL_USER']         = 'yatesvilleron@gmail.com'
os.environ['GMAIL_APP_PASSWORD'] = 'qtziwiblytgrlzvx'

# FTPS upload creds — make sure FTP_PASS is exactly your password, no < or >
os.environ['FTP_HOST']       = 'ftp.one-name.net'
os.environ['FTP_PORT']       = '21'
os.environ['FTP_USER']       = 'admin@yates.one-name.net'
os.environ['FTP_PASS']       = 'v(i83lfQB@dB'


In [22]:
# Cell 1 20250513-cell2 is good to use; adding more lineage functionality next
#!/usr/bin/env python
"""
GEDCOM Composite Score Script using:
 - Chunk-based Parallel Processing for Speed (Stage 1: genealogical line creation)
 - A Trie-based approach, then final "Value" = 5 * (number of couples with node.count >=2) + (total couples)

For ancestral lines where none of the couples are repeated (a one-off line), the Value is still computed.
Now, instead of composite scoring, two new columns are added:
  - Value Range (the numeric bracket)
  - Value Label (a descriptive label)

Exports final CSV/HTML sorted by "Yates DNA Ancestral Line", including a 'haplogroup' column.
"""
import csv
import glob
import logging
import functools
import os
from datetime import datetime
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from IPython.display import display, Javascript

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

###############################################################################
# Global Variables
###############################################################################
anchor_gen1 = None
visited_pairs = set()
generation_table = []

###############################################################################
# Trie Data Structure
###############################################################################
class TrieNode:
    """A simple Trie node for storing a couple and counting how many lines pass here."""
    def __init__(self):
        self.count = 0
        self.children = {}

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert_line(self, couples_list):
        current = self.root
        for couple in couples_list:
            if couple not in current.children:
                current.children[couple] = TrieNode()
            current = current.children[couple]
            current.count += 1

    def get_couple_count(self, couples_list):
        counts = []
        current = self.root
        for couple in couples_list:
            if couple in current.children:
                current = current.children[couple]
                counts.append(current.count)
            else:
                counts.append(0)
                break
        return counts

###############################################################################
# Utility: chunk generator
###############################################################################
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

###############################################################################
# GedcomDataset
###############################################################################
class GedcomDataset:
    def __init__(self, gen_person):
        self.gen_person = gen_person
        self.extractable_detail = {}
        self.anchor_gen1 = None

    def add_extractable_detail(self, key, value):
        self.extractable_detail[key] = value

    def get_gen_person(self):
        name = self.extractable_detail.get('NAME', '')
        parts = name.split('/', 1)
        first_name = parts[0].split(' ')[0]
        last_name = parts[1].rstrip('/') if len(parts) > 1 else ""
        self.anchor_gen1 = last_name.replace(" ", "") + first_name.replace(" ", "")
        global anchor_gen1
        anchor_gen1 = self.anchor_gen1
        return self.gen_person.strip('@')

    def get_extractable_NPFX(self):
        return self.extractable_detail.get('NPFX', '')

    def get_extractable_cm(self):
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '&' in npfx_value:
            cm_value = npfx_value.split('&')[0].strip()
        elif '**' in npfx_value:
            cm_value = npfx_value.split('**')[0].strip()
        else:
            cm_value = npfx_value.strip()
        try:
            int(cm_value)
            return cm_value
        except ValueError:
            return ''

    def get_extractable_sort(self):
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '&' in npfx_value:
            sort_part = npfx_value.split('&')[1]
            if '**' in sort_part:
                sort_value = sort_part.split('**')[0].strip()
            else:
                sort_value = sort_part.strip()
            return sort_value
        return ''

    def get_extractable_YDNA(self):
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '**' in npfx_value:
            ydna_value = npfx_value.split('**')[1].strip()
            return ydna_value
        return ''

    def get_extractable_FAMC(self):
        return self.extractable_detail.get('FAMC', '').strip('@')

###############################################################################
# Gedcom Class
###############################################################################
class Gedcom:
    def __init__(self, file_name):
        self.file_name = file_name
        self.gedcom_datasets = []
        self.filter_pool = []

    def parse_gedcom(self):
        with open(self.file_name, 'r', encoding='utf-8-sig') as f:
            lines = f.readlines()

        current_dataset = None
        npfx_count = 0
        ydna_count = 0
        total_count = 0

        for line in lines:
            parts = line.strip().split(' ', 2)
            level = int(parts[0])
            tag = parts[1]
            value = parts[2] if len(parts) > 2 else None

            if level == 0 and tag.startswith('@') and tag.endswith('@') and value == 'INDI':
                total_count += 1
                current_dataset = GedcomDataset(tag)
                self.gedcom_datasets.append(current_dataset)
            elif current_dataset is not None:
                if level == 1 and tag in ['NAME', 'FAMC']:
                    current_dataset.add_extractable_detail(tag, value)
                elif level == 2 and tag == 'NPFX':
                    npfx_count += 1
                    current_dataset.add_extractable_detail(tag, value)
                    if '**' in value:
                        ydna_count += 1

        autosomal_count = npfx_count - ydna_count
        print(f"GEDCOM contained {total_count} total records")
        print(f"Records tagged and filtered by NPFX: {npfx_count}")
        print(f"Records with YDNA information: {ydna_count}")
        print(f"Autosomal matches: {autosomal_count}")

        for ds in self.gedcom_datasets:
            if ds.get_extractable_NPFX():
                self.filter_pool.append(ds)

        manual_filter_activated = True
        if manual_filter_activated:
            try:
                df = pd.read_excel('filtered_ids.xlsx')
            except FileNotFoundError:
                logger.warning("filtered_ids.xlsx not found. Skipping second-level manual filter.")
            else:
                manual_filtered_ids = set(df['ID'])
                self.filter_pool = [d for d in self.filter_pool if d.get_gen_person() in manual_filtered_ids]
                print(f"After manual filter, total records: {len(self.filter_pool)}")
                logger.info(f"After manual filter, total records: {len(self.filter_pool)}")

        return autosomal_count

###############################################################################
# quick_extract_name
###############################################################################
def quick_extract_name(full_text):
    name_marker = "\n1 NAME "
    idx = full_text.find(name_marker)
    if idx == -1:
        if full_text.startswith("1 NAME "):
            idx = 0
        else:
            return "UnknownName"
    start = idx + len(name_marker)
    end = full_text.find('\n', start)
    if end == -1:
        end = len(full_text)
    name_line = full_text[start:end].strip()
    if '/' not in name_line:
        return name_line[:10].replace(" ", "")
    first_name, last_name = name_line.split('/', 1)
    last_name = last_name.replace("/", "").strip()
    return last_name[:10].replace(" ", "") + first_name[:10].replace(" ", "")

###############################################################################
# Parents & Ancestors
###############################################################################
def find_parents(individual_id, generation, parents_map):
    global visited_pairs, generation_table
    if individual_id not in parents_map:
        return
    father_id, mother_id = parents_map[individual_id]
    if not father_id and not mother_id:
        return
    pair = (father_id, mother_id)
    if pair not in visited_pairs:
        visited_pairs.add(pair)
        generation_table.append((generation, pair))
    if father_id:
        find_parents(father_id, generation+1, parents_map)
    if mother_id:
        find_parents(mother_id, generation+1, parents_map)

def find_distant_ancestors(individual_id, parents_map, path=None):
    if path is None:
        path = []
    path.append(individual_id)
    if individual_id not in parents_map:
        return [path]
    father_id, mother_id = parents_map[individual_id]
    if not father_id and not mother_id:
        return [path]
    paths = []
    if father_id:
        paths.extend(find_distant_ancestors(father_id, parents_map, path[:]))
    if mother_id:
        paths.extend(find_distant_ancestors(mother_id, parents_map, path[:]))
    return paths if paths else [path]

###############################################################################
# filter_ancestral_line
###############################################################################
def filter_ancestral_line(winning_path_ids, generation_table_local, names_map):
    matching_table = []
    for generation, pair in generation_table_local:
        id1, id2 = pair
        if id1 in winning_path_ids or id2 in winning_path_ids:
            matching_table.append((generation, pair))
    matching_table.sort(key=lambda x: x[0])
    lines = []
    for gen, pair in matching_table:
        name_pair = [names_map.get(pid, "UnknownName") for pid in pair]
        lines.append(f"{name_pair[0]}&{name_pair[1]}")
    lines.reverse()
    return "~~~".join(lines)

###############################################################################
# process_record_wrapper (parallel) - STAGE 1
###############################################################################
def process_record_wrapper(individual_id, gedcom_instance, parents_map, names_map):
    global generation_table, visited_pairs, anchor_gen1
    generation_table = []
    visited_pairs = set()

    find_parents(individual_id, 1, parents_map)
    distant_anc_paths = find_distant_ancestors(individual_id, parents_map)

    best_score = None
    best_path = None
    for path in distant_anc_paths:
        name_path = [names_map.get(pid, "UnknownName") for pid in path]
        score = sum((idx+1) for idx, nm in enumerate(name_path) if 'Yates' in nm)
        if best_score is None or score > best_score:
            best_score = score
            best_path = path

    if not best_path:
        best_path = []

    best_path_cleaned = [pid for pid in best_path if pid != individual_id]
    line_str = filter_ancestral_line(set(best_path_cleaned), generation_table, names_map)

    cm_value = ''
    sort_value = ''
    ydna_value = ''
    for ds in gedcom_instance.filter_pool:
        if ds.get_gen_person() == individual_id:
            cm_value = ds.get_extractable_cm()
            sort_value = ds.get_extractable_sort()
            ydna_value = ds.get_extractable_YDNA()
            break

    short_name = names_map.get(individual_id, "UnknownName")
    # Return columns: ID#, Match to, Name, cM, Yates DNA Ancestral Line, haplogroup
    return [individual_id, sort_value, short_name, cm_value, line_str, ydna_value]

###############################################################################
# main()
###############################################################################
def main():
    def select_gedcom():
        files = glob.glob("*.ged")
        if not files:
            print("No GEDCOM files found.")
            return None
        print("Automatically selecting the first GEDCOM file.")
        return files[0]

    gedcom_file_path = select_gedcom()
    if not gedcom_file_path:
        print("No GEDCOM file selected; exiting.")
        return

    ged = Gedcom(gedcom_file_path)
    autosomal_count = ged.parse_gedcom()
    filter_count = len(ged.filter_pool)

    with open("autosomal_count.txt", "w") as f:
        f.write(str(autosomal_count))

    print("Records tagged and filtered by NPFX:", filter_count)

    with open(gedcom_file_path, 'r', encoding='utf-8') as f:
        raw_data = f.read()

    blocks = raw_data.split('\n0 ')
    all_records = {}
    for blk in blocks:
        blk = blk.strip()
        if not blk:
            continue
        flend = blk.find('\n')
        if flend == -1:
            flend = len(blk)
        first_line = blk[:flend]
        if '@' in first_line:
            start = first_line.find('@') + 1
            end = first_line.find('@', start)
            rec_id = first_line[start:end].strip()
            all_records[rec_id] = blk

    parents_map = {}
    names_map = {}
    for rec_id, txt in all_records.items():
        nm = quick_extract_name("\n" + txt)
        names_map[rec_id] = nm

    families = {}
    for rec_id, txt in all_records.items():
        if 'FAM' in txt[:50]:
            father_idx = txt.find('1 HUSB @')
            husb_id = txt[father_idx+len('1 HUSB @'):txt.find('@', father_idx+len('1 HUSB @'))] if father_idx != -1 else None
            wife_idx = txt.find('1 WIFE @')
            wife_id = txt[wife_idx+len('1 WIFE @'):txt.find('@', wife_idx+len('1 WIFE @'))] if wife_idx != -1 else None
            kids = [ln.split('@')[1] for ln in txt.split('\n') if ln.strip().startswith('1 CHIL @')]
            families[rec_id] = (husb_id, wife_id, kids)

    for fam_id, (f_id, m_id, k_list) in families.items():
        for kid in k_list:
            parents_map[kid] = (f_id, m_id)

    individual_ids = [d.get_gen_person() for d in ged.filter_pool]
    print(f"Processing {len(individual_ids)} individuals with chunk-based parallel...")

    combined_rows = []
    chunk_size = 50
    max_workers = os.cpu_count() or 4
    logger.info("Starting chunk-based parallel processing with %d workers.", max_workers)

    with ProcessPoolExecutor(max_workers=max_workers) as executor, tqdm(total=len(individual_ids), desc="Building Yates Lines (Stage 1)") as pbar:
        for chunk in chunks(individual_ids, chunk_size):
            func = functools.partial(process_record_wrapper, gedcom_instance=ged, parents_map=parents_map, names_map=names_map)
            results = list(executor.map(func, chunk))
            combined_rows.extend(results)
            pbar.update(len(chunk))

    columns = ["ID#", "Match to", "Name", "cM", "Yates DNA Ancestral Line", "haplogroup"]
    df = pd.DataFrame(combined_rows, columns=columns)
    df.index += 1

    def remove_specific_prefix(row):
        prefix = "YatesJohn&SearchingStill~~~YatesWilliam&SearchingStill~~~YatesWilliam&SearchingStill~~~YatesEdmund&CornellMargaret~~~YatesRichard&AshendonJoan~~~YatesJohn&HydeAlice~~~YatesThomas&FauconerElizabeth~~~"
        if row["Yates DNA Ancestral Line"].startswith(prefix):
            row["Yates DNA Ancestral Line"] = row["Yates DNA Ancestral Line"][len(prefix):]
        return row

    df = df.apply(remove_specific_prefix, axis=1)

    logger.info("Building Trie from reversed lines...")
    trie = Trie()
    for _, row in df.iterrows():
        line_str = row["Yates DNA Ancestral Line"]
        if pd.notna(line_str) and line_str.strip():
            trie.insert_line([x.strip() for x in line_str.split("~~~") if x.strip()])

    values, prefix_counts = [], []
    logger.info("Computing 'Value' = 5*(#couples with node.count >=2) + (total couples) ...")
    for _, row in df.iterrows():
        line_str = row["Yates DNA Ancestral Line"]
        if pd.isna(line_str) or not line_str.strip():
            values.append(0)
            prefix_counts.append(0)
        else:
            couples_list = [x.strip() for x in line_str.split("~~~") if x.strip()]
            node_counts = trie.get_couple_count(couples_list)
            prefix_count = sum(1 for c in node_counts if c >= 2)
            values.append(5 * prefix_count + len(couples_list))
            prefix_counts.append(prefix_count)

    df["Value"], df["PrefixCount"] = values, prefix_counts

    def assign_value_range_label(val):
        try:
            v = float(val)
        except:
            return "", ""
        if v >= 60: return ">=60", "1-likely correct"
        if 47 <= v <= 59: return "59~47", "2-lines forming"
        if 34 <= v <= 46: return "46~34", "3-patterns emerging"
        if 21 <= v <= 33: return "33~21", "4-notable patterns"
        if 8 <= v <= 20: return "20~8", "5-patterns stable"
        if 1 <= v <= 7:  return f"{v:.0f}", "6-need research"
        return f"{v:.0f}", "0-uncategorized"

    ranges, labels = zip(*(assign_value_range_label(v) for v in df["Value"]))
    df["Value Range"], df["Value Label"] = ranges, labels

    df.sort_values(by=["Yates DNA Ancestral Line"], inplace=True)
    df.drop("PrefixCount", axis=1, inplace=True)

    csv_name = "final_combined_df_with_value_labels.csv"
    df.to_csv(csv_name, index=False)
    logger.info("Exported final DataFrame to '%s'.", csv_name)

    html_name = "HTML_combined_df_with_value_labels.html"
    css_style = """
    <style>
    table { width: 100%; border-collapse: collapse; margin: 20px 0; }
    table, th, td { border: 1px solid #333; }
    th, td { padding: 8px 12px; text-align: center; }
    th { background-color: #f2f2f2; }
    /* Left-align the last column */
    td:nth-child(7) { text-align: left; }
    </style>
    """
    final_cols = ["ID#", "cM", "haplogroup", "Match to", "Value Range", "Value Label", "Yates DNA Ancestral Line"]
    html_content = css_style + df.to_html(index=False, columns=final_cols, escape=False)
    with open(html_name, "w", encoding="utf-8") as f:
        f.write(html_content)
    logger.info("Exported HTML to '%s'.", html_name)

if __name__ == '__main__':
    main()
    try:
        display(Javascript('alert("✅ GEDCOM processing (and HTML export) is complete!");'))
    except:
        pass

import os
import pandas as pd
import smtplib, ssl
from email.mime.text import MIMEText

def send_email(subject, body, to_addr):
    smtp_server = 'smtp.gmail.com'
    port = 465
    sender = os.environ['GMAIL_USER']
    password = os.environ['GMAIL_APP_PASSWORD']
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = to_addr
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender, password)
        server.send_message(msg)

# Email summary (only total lines)
df_summary = pd.read_csv("final_combined_df_with_value_labels.csv")
total = len(df_summary)
summary = f"GEDCOM processing complete!\n\nTotal lines: {total}"

send_email(
    subject="✅ Cell #1 Report Ready",
    body=summary,
    to_addr=os.environ['GMAIL_USER']
)





Automatically selecting the first GEDCOM file.


GEDCOM contained 62184 total records
Records tagged and filtered by NPFX: 1556
Records with YDNA information: 1
Autosomal matches: 1555
Records tagged and filtered by NPFX: 1556
Processing 1556 individuals with chunk-based parallel...


Building Yates Lines (Stage 1): 100%|██████████| 1556/1556 [11:50<00:00,  2.19it/s]


<IPython.core.display.Javascript object>

In [27]:
# Gold 2 — create the Report Card (mobile-friendly, sortable)

# ====== CUT START [A] REFACTOR-Gold 2 — Config + FTP + Constants + Rules =========================
import os
import posixpath
import socket
from ftplib import FTP_TLS

# ---------- Script identity ----------
SCRIPT_NAME = "REFACTOR-Gold 2"
SECTION_NAME = "[A] Config + FTP + Constants + Rules"
DOM_ID_PREFIX = "g2-"  # all HTML ids/classes must begin with "g2-"
NS_PREFIX = "G2_"      # Python-visible constant/function prefix

# ---------- Encoding ----------
ENC = "iso-8859-15"

# ---------- Environment / FTP ----------
FTP_HOST = os.getenv("FTP_HOST", "")
FTP_USER = os.getenv("FTP_USER", "")
FTP_PASS = os.getenv("FTP_PASS", "")
FTP_DIR  = os.getenv("FTP_DIR", "/gengen/")
FTP_TIMEOUT = int(os.getenv("FTP_TIMEOUT", "45"))
FTP_TLS_REQUIRED = True  # enforce FTPS

# ---------- File naming (Gold 2: Register) ----------
LOCAL_NAME  = os.getenv("G2_LOCAL_NAME",  "ons_yates_dna_register.htm")
REMOTE_NAME = os.getenv("G2_REMOTE_NAME", "ons_yates_dna_register.htm")

# Partials unique to Gold 2
PARTIAL_MATCH_COUNT   = os.getenv("G2_PARTIAL_MATCH_COUNT",   "partials/match_count.htm")
PARTIAL_LINEAGE_COUNT = os.getenv("G2_PARTIAL_LINEAGE_COUNT", "partials/lineage_count.htm")
PARTIAL_PRINTABLE     = os.getenv("G2_PARTIAL_PRINTABLE",     "partials/cousin_list_print.htm")

# Optional counters/diagnostics (distinct from Gold 3)
COUNT_PUBLIC_URL = os.getenv("G2_COUNT_PUBLIC_URL", "")
LOCAL_COUNT_FILE = os.getenv("G2_LOCAL_COUNT_FILE", "autosomal_count.txt")
REMOTE_COUNT_NAME = os.getenv("G2_REMOTE_COUNT_NAME", "autosomal_count.txt")

# ---------- Paths ----------
def _remote_path(name: str) -> str:
    """Join FTP_DIR and name with posix semantics."""
    base = FTP_DIR.rstrip("/")
    return f"{base}/{name.lstrip('/')}"

# ---------- FTP helpers (namespaced logic, no side effects on import) ----------
def ftp_connect() -> FTP_TLS:
    """Return a logged-in FTPS handle with passive mode and timeout."""
    if not FTP_HOST or not FTP_USER or not FTP_PASS:
        raise RuntimeError("FTP credentials are not set in environment variables.")
    ftps = FTP_TLS(timeout=FTP_TIMEOUT)
    ftps.connect(host=FTP_HOST, port=21)
    ftps.auth()  # upgrade to TLS
    ftps.prot_p()  # secure data connection
    ftps.login(user=FTP_USER, passwd=FTP_PASS)
    ftps.set_pasv(True)
    # Best-effort hostname note (for diagnostics)
    try:
        ftps.hostname = socket.gethostbyname(FTP_HOST)  # type: ignore[attr-defined]
    except Exception:
        pass
    return ftps

def ftp_upload_overwrite(ftps: FTP_TLS, local_path: str, remote_name: str) -> None:
    """Upload file to FTP_DIR/remote_name, overwriting if exists."""
    remote = _remote_path(remote_name)
    # Ensure remote directory exists (best-effort, ignore failures on nested)
    parts = remote.split("/")
    dir_parts, filename = parts[:-1], parts[-1]
    cwd = "/"
    try:
        ftps.cwd("/")
    except Exception:
        pass
    for d in dir_parts:
        if not d:
            continue
        cwd = posixpath.join(cwd, d)
        try:
            ftps.mkd(cwd)
        except Exception:
            pass
    with open(local_path, "rb") as fh:
        ftps.storbinary(f"STOR {remote}", fh)

# ---------- Guardrails ----------
# This script must not import or reference any G3_* symbols.
# All DOM ids, classes, and JS functions created by this script must use the "g2-" prefix.
# All files written by this script must use the Gold 2 filenames defined above.
# ====== CUT STOP  [A] REFACTOR-Gold 2 — Add 5 returns for spacing ================
#
#
#
#
#
# ====== CUT START [B] REFACTOR-Gold 2 — Helpers / Resolver (root-level /partials + utilities) ====






import os, io, re, csv, posixpath
from typing import Dict, Tuple, List, Optional

try:
    import pandas as pd
except Exception:
    pd = None

# ---------- Display / DOM / TNG defaults (Gold 2 only) ----------
DOM_ID_PREFIX = globals().get("DOM_ID_PREFIX", "g2-")
NS_PREFIX     = globals().get("NS_PREFIX", "G2_")

# Column A width (Match Summary) and total table width (pixels)
COL_A_PX       = int(os.getenv("G2_COL_A_PX", "520"))
TABLE_WIDTH_PX = int(os.getenv("G2_TABLE_WIDTH_PX", "1200"))

# Text headers/entities
LINEAGE_HEADER  = "Lineage (Starting with oldest ancestor, the line is:)"
ARROW_ENTITY    = "&rarr;"

# TNG linking (used in [4/6])
TNG_BASE = os.getenv("TNG_BASE", "https://yates.one-name.net/tng")
TNG_TREE = os.getenv("TNG_TREE", "yates")

# Side artifact CSV (Column A export) used by [4/6] and [6/6]
MATCH_COUSINS_CSV = os.getenv("G2_MATCH_COUSINS_CSV", "the_match_cousins.csv")

# ---------- Absolute root-level /partials support (Resolver) ----------
# Primary location: /partials/match_to_unmasked.csv (absolute from server root)
# Fallback location: /gengen/partials/match_to_unmasked.csv (under FTP_DIR)
SERVER_PARTIALS_DIR     = os.getenv("G2_SERVER_PARTIALS_DIR", "/partials")
SERVER_MAPPING_BASENAME = os.getenv("G2_SERVER_MAPPING_BASENAME", "match_to_unmasked.csv")
LOCAL_MAPPING_CACHE     = os.getenv("G2_LOCAL_MAPPING_CACHE", "match_to_unmasked.csv")

def _remote_abs(*parts: str) -> str:
    """Join into an absolute POSIX path that DOES NOT prepend FTP_DIR."""
    cleaned = [str(p).strip("/") for p in parts if p is not None]
    return "/" + "/".join([p for p in cleaned if p])

def _remote_under_ftpdir(name: str) -> str:
    """Fallback to FTP_DIR-prefixed path (e.g., /gengen/partials/...)."""
    base = str(globals().get("FTP_DIR", "/")).rstrip("/")
    return f"{base}/{name.lstrip('/')}"

def _ftps_fetch_to_local(ftps, remote_path: str, local_path: str) -> Tuple[bool, str]:
    """Try to RETR a remote file to local_path. Return (ok, error_text)."""
    try:
        parent = os.path.dirname(local_path)
        if parent and not os.path.isdir(parent):
            os.makedirs(parent, exist_ok=True)
        with open(local_path, "wb") as fh:
            ftps.retrbinary(f"RETR {remote_path}", fh.write)
        return True, ""
    except Exception as e:
        return False, f"{type(e).__name__}: {e}"

def _detect_cols(df) -> Tuple[str, str]:
    """Pick code and unmasked columns from a DataFrame by heuristics."""
    cols = [str(c) for c in df.columns]
    code_candidates = ["Match to", "match_to", "match to", "code", "Code"]
    name_candidates = ["Unmasked", "unmasked", "name", "Name", "Unmask", "unmask"]
    def pick(cands):
        for c in cols:
            cl = c.lower().replace(" ", "")
            for k in cands:
                if cl == k.replace(" ", "").lower():
                    return c
        return None
    code_col = pick(code_candidates) or (cols[0] if cols else "")
    name_col = pick(name_candidates) or (cols[1] if len(cols) > 1 else cols[0] if cols else "")
    return code_col, name_col

def _read_resolver_csv(local_path: str) -> Dict[str, str]:
    """Load resolver csv to {code: unmasked} with minimal normalization."""
    if pd is not None:
        df = pd.read_csv(local_path, dtype=str, encoding="iso-8859-15", keep_default_na=False)
        code_col, name_col = _detect_cols(df)
        mapping = {}
        for _, row in df.iterrows():
            k = str(row.get(code_col, "")).strip()
            v = str(row.get(name_col, "")).strip()
            if k:
                mapping[k] = v
        return mapping
    # Fallback without pandas
    mapping = {}
    with open(local_path, "r", encoding="iso-8859-15", errors="replace", newline="") as fh:
        reader = csv.reader(fh)
        rows = list(reader)
    if not rows:
        return mapping
    header = rows[0]
    if len(header) >= 2 and not header[0].strip().isdigit():
        data_rows = rows[1:]
        code_idx = 0
        name_idx = 1
    else:
        data_rows = rows
        code_idx = 0
        name_idx = 1 if len(rows[0]) > 1 else 0
    for r in data_rows:
        if not r:
            continue
        k = (r[code_idx] if len(r) > code_idx else "").strip()
        v = (r[name_idx] if len(r) > name_idx else "").strip()
        if k:
            mapping[k] = v
    return mapping

def load_resolver_from_server() -> Dict[str, str]:
    """Load resolver CSV from root-level /partials first, then fallback to /gengen/partials."""
    if "ftp_connect" not in globals():
        raise RuntimeError("ftp_connect() is not available; ensure Section [A] is loaded first.")
    ftps = ftp_connect()
    try:
        primary  = _remote_abs(SERVER_PARTIALS_DIR, SERVER_MAPPING_BASENAME)  # /partials/match_to_unmasked.csv
        ok, err  = _ftps_fetch_to_local(ftps, primary, LOCAL_MAPPING_CACHE)
        if not ok:
            fallback = _remote_under_ftpdir(posixpath.join("partials", SERVER_MAPPING_BASENAME))
            ok2, err2 = _ftps_fetch_to_local(ftps, fallback, LOCAL_MAPPING_CACHE)
            if not ok2:
                raise RuntimeError(
                    "Resolver not found on server at either location:\n"
                    f"  1) {primary}\n"
                    f"  2) {fallback}\n"
                    "Action: Upload match_to_unmasked.csv to /partials/ (root) or to /gengen/partials/ and re-run.\n"
                    f"Details: primary error: {err}; fallback error: {err2}"
                )
        mapping = _read_resolver_csv(LOCAL_MAPPING_CACHE)
        return mapping
    finally:
        try:
            ftps.quit()
        except Exception:
            pass

# Global resolver dict (available to other sections)
MATCH_TO_UNMASKED = load_resolver_from_server()

# ---------- Published helpers to other sections ----------

def resolve_match_to(code_value: str) -> str:
    """Return unmasked name for a resolver 'Match to' code, or empty string if unknown."""
    if not code_value and code_value != 0:
        return ""
    code = str(code_value).strip()
    if not code:
        return ""
    return MATCH_TO_UNMASKED.get(code, "")

def html_escape(s: str) -> str:
    import html as _h
    return _h.escape("" if s is None else str(s), quote=True)

def smart_titlecase(s: str) -> str:
    """Title-case with a few surname niceties; ASCII/iso-8859-15 safe."""
    if not isinstance(s, str):
        return ""
    s = re.sub(r"\s+", " ", s.strip())
    if not s:
        return ""
    parts = s.split(" ")
    out = []
    for p in parts:
        if not p:
            out.append("")
            continue
        # O'Neill, O'CONNOR
        m = re.match(r"^([Oo]')([A-Za-z]+)$", p)
        if m:
            out.append(m.group(1).capitalize() + m.group(2).capitalize())
            continue
        # Mc/Mac
        m = re.match(r"^(Mc|MC)([A-Za-z]+)$", p)
        if m:
            out.append("Mc" + m.group(2).capitalize())
            continue
        if "-" in p:
            out.append("-".join([x.capitalize() for x in p.split("-")]))
        else:
            out.append(p.capitalize())
    return " ".join(out)

def truncate_first(name: str, n: int = 7) -> str:
    """Truncate the first token to n letters, keep the rest as-is."""
    if not isinstance(name, str):
        return ""
    name = smart_titlecase(name)
    toks = name.split()
    if not toks:
        return ""
    first = re.sub(r"[^A-Za-z]", "", toks[0])[:max(1, n)]
    return " ".join([first] + toks[1:]) if len(toks) > 1 else first

_CAMEL_WORDS = re.compile(r"[A-Z][a-z]*|[A-Z]+(?![a-z])|[a-z]+")

def surname_given_from_token(tok: str) -> Tuple[str, str, str]:
    """
    Parse glued or CamelCase tokens like 'RosenbalmJessica' or 'ReedLindaGai'.
    Returns (full_name, surname, given). Full name is 'Given Surname'.
    Falls back to simple titlecase of tok if nothing reliable found.
    """
    raw = str(tok or "").strip()
    if not raw:
        return ("", "", "")
    # Already spaced or comma
    if " " in raw or "," in raw:
        nm = smart_titlecase(raw)
        parts = nm.split()
        if len(parts) == 1:
            return (nm, parts[0], "")
        given = parts[0]
        surname = parts[-1]
        return (f"{given} {surname}", surname, given)
    # Camel extraction
    words = _CAMEL_WORDS.findall(raw)
    while words and len(words[0]) == 1:
        words.pop(0)
    if not words:
        nm = smart_titlecase(raw)
        ps = nm.split()
        if len(ps) == 1:
            return (nm, ps[0], "")
        return (f"{ps[0]} {ps[-1]}", ps[-1], ps[0])
    surname = smart_titlecase(words[0])
    given_candidates = [w for w in words[1:] if w.lower() != surname.lower()]
    if not given_candidates:
        return (surname, surname, "")
    given = smart_titlecase(given_candidates[0])
    return (f"{given} {surname}", surname, given)

def split_tokens(path_str: str) -> List[str]:
    """
    Split lineage path into tokens (oldest → youngest).
    Supports '→', '->', '&rarr;', ' > ' and single '>' as separators.
    """
    s = str(path_str or "")
    if not s.strip():
        return []
    s = s.replace("&rarr;", "→")
    s = re.sub(r"\s*->\s*", " → ", s)
    s = re.sub(r"\s*>\s*", " → ", s)
    parts = [p.strip() for p in s.split("→")]
    return [p for p in parts if p]

def derive_common_from_first_token(tokens: List[str]) -> Tuple[str, str]:
    """
    Given lineage tokens, return (husband, wife) from the first token.
    Supports 'A & B' as well as glued 'YatesStephen&ParsonsLydia' variants.
    """
    if not tokens:
        return ("", "")
    t0 = tokens[0].strip()
    if "&" in t0:
        a, b = [smart_titlecase(p.strip()) for p in t0.split("&", 1)]
        return (a, b)
    # Try to split glued halves around last uppercase run
    m = re.match(r"^(.*?)([A-Z][a-zA-Z]+)&([A-Z][a-zA-Z].*)$", t0)
    if m:
        return (smart_titlecase(m.group(2)), smart_titlecase(m.group(3)))
    # Fallback: best-effort two-name recovery using commas or spaces
    bits = re.split(r"\s*,\s*|\s+and\s+|\s*\+\s*|\s*/\s*", t0, maxsplit=1)
    if len(bits) == 2:
        return (smart_titlecase(bits[0]), smart_titlecase(bits[1]))
    return (smart_titlecase(t0), "")

def build_header(
    subject_name_b_html: str,
    cm_val: str,
    matchee_html: str,
    gens_total: int,
    common_husband_short: str,
    common_wife_short: str
) -> str:
    """
    Compose the Match Summary sentence (HTML). Inputs should already be escaped where needed.
    """
    cm_txt   = html_escape(cm_val)
    gens_txt = f"{int(gens_total)}" if str(gens_total).isdigit() else html_escape(gens_total)
    pair_txt = ""
    if common_husband_short or common_wife_short:
        a = html_escape(common_husband_short or "")
        b = html_escape(common_wife_short or "")
        if a and b:
            pair_txt = f" &mdash; oldest DISTANT common ancestors: {a} &amp; {b}"
        else:
            pair_txt = f" &mdash; oldest DISTANT common ancestor: {a or b}"
    return (
        f"{subject_name_b_html} shares <strong>{cm_txt} cM</strong> with {matchee_html} "
        f"across a documented path of <strong>{gens_txt}</strong> steps.{pair_txt}"
    )

def normalize_person_name(s: str) -> str:
    """Return a title-cased person name with safe punctuation for XHTML/iso-8859-15."""
    if not isinstance(s, str):
        return ""
    s = s.strip()
    if not s:
        return ""
    s = re.sub(r"\s+", " ", s)
    return html_escape(smart_titlecase(s))

# ---------- General helpers used by [3]/[4]/[5]/[6] ----------

def find_col(df, regex_list: Optional[List[str]] = None, fallback_list: Optional[List[str]] = None) -> Optional[str]:
    """Find a column by regex OR fallback names; returns the first match or None."""
    if df is None or getattr(df, "columns", None) is None:
        return None
    regex_list = regex_list or []
    fallback_list = fallback_list or []
    cols = list(df.columns)

    for pattern in regex_list:
        try:
            rx = re.compile(pattern, flags=re.IGNORECASE)
        except Exception:
            continue
        for c in cols:
            if rx.search(str(c)):
                return c

    for name in fallback_list:
        target = str(name).strip().lower()
        for c in cols:
            if str(c).strip().lower() == target:
                return c

    return cols[0] if cols else None

def find_cols(df, patterns_or_names: List[str]) -> List[str]:
    """Return all columns that match any regex or exact name (case-insensitive)."""
    hits = []
    if df is None or getattr(df, "columns", None) is None:
        return hits
    cols = list(df.columns)
    for token in patterns_or_names:
        is_regex = bool(re.search(r"[\\^$.*+?()\\[\\]|]", token))
        if is_regex:
            try:
                rx = re.compile(token, flags=re.IGNORECASE)
            except Exception:
                continue
            for c in cols:
                if rx.search(str(c)):
                    hits.append(c)
        else:
            low = token.strip().lower()
            for c in cols:
                if str(c).strip().lower() == low:
                    hits.append(c)
    seen, uniq = set(), []
    for c in hits:
        if c not in seen:
            uniq.append(c); seen.add(c)
    return uniq

def normalize_colnames(df):
    """Return a copy with normalized lowercase, single-space column names (no mutation)."""
    if df is None or getattr(df, "columns", None) is None:
        return df
    new = df.copy()
    new.columns = [re.sub(r"\s+", " ", str(c)).strip().lower() for c in df.columns]
    return new

def coalesce_col(df, candidates: List[str]) -> Optional[str]:
    """Return the first existing column from candidates (exact match)."""
    if df is None or getattr(df, "columns", None) is None:
        return None
    cols = set([str(c) for c in df.columns])
    for c in candidates:
        if c in cols:
            return c
    return None

def comma_int(n) -> str:
    try:
        return f"{int(float(str(n).replace(',', ''))):,}"
    except Exception:
        return str(n)

def is_htmlish(s: str) -> bool:
    if not isinstance(s, str):
        return False
    return ("<a " in s) or ("<span" in s) or ("<div" in s) or ("<br" in s) or ("</" in s)

def safe_get(row, col, default=""):
    try:
        return row.get(col, default)
    except Exception:
        try:
            return row[col]
        except Exception:
            return default

def slugify(s: str) -> str:
    s = re.sub(r"[^A-Za-z0-9]+", "-", str(s)).strip("-")
    s = re.sub(r"-{2,}", "-", s)
    return s.lower()

# ====== CUT STOP  [B] REFACTOR-Gold 2 — Add 5 returns for spacing ===============================
#
#
#
#
#
# ====== CUT START [3/6] REFACTOR-Gold 2 — CSV Load + Column Detection + Row Helpers ================






import os, re, pandas as pd, urllib.parse as _u

# Use the path defined earlier; default to expected file
CSV_IN = os.environ.get("CSV_IN", CSV_PATH if 'CSV_PATH' in globals() else "/content/final_combined_df_with_value_labels.csv")

# ---- Load CSV (robust encodings) ----
_encs = ("utf-8-sig", "utf-8", "cp1252", "iso-8859-15", "latin1")
_last_err = None
df = None
for _e in _encs:
    try:
        df = pd.read_csv(CSV_IN, encoding=_e, dtype=str, keep_default_na=False)
        break
    except Exception as _ex:
        _last_err = _ex
        df = None
if df is None:
    raise RuntimeError(f"Unable to read CSV: {CSV_IN} ({_last_err})")
print(f"✅ Loaded CSV — {len(df)} rows, {len(df.columns)} columns from {os.path.abspath(CSV_IN)}")

# ---- Locate columns (names vary slightly across exports) ----
# relies on find_col() from [2/6]
id_col = find_col(
    df,
    [r'^(id#|personid)$', r'^\s*id\s*#?\s*$', r'^person\s*id$'],
    ["ID#", "ID", "PersonID", "personID", "person id", "id #"]
)
match_to_col  = find_col(df, [r'^match\s*to$'], ["Match to","Match"])
name_col      = find_col(df, [r'^name$'], ["Name"])
cm_col        = find_col(df, [r'^(c\s*:?m|cm)$', r'centi.?morgan'], ["cM","cm"])
path_col      = find_col(df, [r'(yates\s*dna\s*ancestral\s*line|ancestral\s*line|lineage)'],
                         ["Yates DNA Ancestral Line","Ancestral Line","Lineage"])

if not match_to_col: raise ValueError("CSV missing 'Match to' column.")
if not name_col:     raise ValueError("CSV missing 'Name' column.")
if not cm_col:       raise ValueError("CSV missing 'cM' column.")
if not path_col:     raise ValueError("CSV missing lineage/path column.")

# --- Back-compat aliases: ensure literal "ID#" and "id#" both exist to stop KeyErrors in legacy paths ---
if id_col and id_col in df.columns:
    if "ID#" not in df.columns:
        df["ID#"] = df[id_col]
    if "id#" not in df.columns:
        df["id#"] = df[id_col]
else:
    # No detectable id column: create empty aliases so any legacy access won't crash
    if "ID#" not in df.columns:
        df["ID#"] = ""
    if "id#" not in df.columns:
        df["id#"] = ""

# ---- Row helpers (match ID, camel-case names, etc.) ----
ID_PAT = re.compile(r"\bI\d+\b", re.I)
_CAMEL_WORDS = re.compile(r"[A-Z][a-z]*|[A-Z]+(?![a-z])|[a-z]+")

def extract_person_id(s: str) -> str:
    """Return TNG-style personID like 'I1234' if present, else ''."""
    m = ID_PAT.search(str(s or ""))
    return m.group(0).upper() if m else ""

def _truncate_alpha(s: str, n: int) -> str:
    """Keep only letters, truncate to n."""
    return re.sub(r"[^A-Za-z]", "", s)[:n]

def norm_matchee_name(raw: str) -> str:
    """
    Normalize single-token CSV names like 'RosenbalmJessica' or 'ReedLindaGai'
    into 'Jessica Rosenbalm' / 'Linda Reed', with given-name truncated to 7.
    Uses smart_titlecase()/surname_given_from_token() from [2/6].
    """
    raw = str(raw or "").strip()
    if not raw:
        return ""
    # If already spaced/comma, just titlecase and use first token as given, last as surname
    if " " in raw or "," in raw:
        nm = smart_titlecase(raw)
        parts = nm.split()
        if len(parts) == 1:
            return nm
        given = parts[0]
        surname = parts[-1]
        return f"{_truncate_alpha(given, 7)} {surname}".strip()
    # CamelCase (or glued lowercase tail) token
    words = _CAMEL_WORDS.findall(raw)
    while words and len(words[0]) == 1:
        words.pop(0)  # drop leading initials
    if not words:
        nm = smart_titlecase(surname_given_from_token(raw)[0])
        ps = nm.split()
        if len(ps) == 1:
            return nm
        return f"{_truncate_alpha(ps[0], 7)} {ps[-1]}".strip()
    surname = smart_titlecase(words[0])
    given_candidates = [w for w in words[1:] if w.lower() != surname.lower()]
    if not given_candidates:
        return surname
    given = smart_titlecase(given_candidates[0])
    return f"{_truncate_alpha(given, 7)} {surname}".strip()

print("✅ Columns:", {
    "ID (detected)": id_col or "(none; using 'ID#' alias)",
    "Match to": match_to_col,
    "Name": name_col,
    "cM": cm_col,
    "Lineage": path_col
})
# ====== CUT STOP [3/6] REFACTOR-Gold 2 — CSV Load + Column Detection + Row Helpers =================





# ====== CUT START [4/6] REFACTOR-Gold 2 — Transform & Column A (build display_df) ==================
import urllib.parse as _u

# ---------- Build the three display columns ----------
headers  = []
lineages = []
findcol  = []

# Allow absolute link to this page: /{REMOTE_NAME}?q=...
REMOTE_NAME_ABS = "/" + (REMOTE_NAME if 'REMOTE_NAME' in globals() else "ons_yates_dna_register.htm")

for _, row in df.iterrows():
    # Subject: the resolver code in "Match to" → unmasked name (bolded in sentence)
    subject_raw  = row.get(match_to_col, "")
    subject_name = normalize_person_name(resolve_match_to(subject_raw))
    subject_name_b = f"<strong>{subject_name}</strong>" if subject_name else subject_name

    # Matchee: name from CSV (normalized) + TNG hotlink if personID present
    pid = extract_person_id(row.get(id_col, ""))
    matchee_name = norm_matchee_name(row.get(name_col, "")) or subject_name
    if pid:
        matchee_html = (
            f'<a href="{TNG_BASE}/verticalchart.php?personID={pid}&tree={TNG_TREE}&parentset=0&display=vertical&generations=15" '
            f'target="_blank">{matchee_name}</a>'
        )
    else:
        matchee_html = matchee_name

    # cM value
    cm_val = row.get(cm_col, "0")

    # Lineage path (tokens oldest→youngest); first token is common couple
    tokens     = split_tokens(row.get(path_col, ""))
    gens_total = len(tokens)
    tokens_disp = tokens[:7]  # show first 7 pairs

    # Husband & Wife from first token unless explicit columns exist
    if "common_husband" in df.columns and "common_wife" in df.columns:
        husband_raw = str(row.get("common_husband", "")).strip()
        wife_raw    = str(row.get("common_wife", "")).strip()
        if not husband_raw and not wife_raw:
            husband_raw, wife_raw = derive_common_from_first_token(tokens)
    else:
        husband_raw, wife_raw = derive_common_from_first_token(tokens)

    # Build the sentence header (truncate first names to 7 chars for readability)
    header_html = build_header(
        subject_name_b,
        cm_val,
        matchee_html,
        gens_total,
        truncate_first(husband_raw, 7) if husband_raw else "",
        truncate_first(wife_raw, 7) if wife_raw else ""
    )

    # Bold the first pair in lineage view
    if tokens_disp:
        tokens_disp[0] = f"<strong>{tokens_disp[0]}</strong>"
    sep = f" {ARROW_ENTITY} "
    lineage_text = sep.join(tokens_disp) if tokens_disp else ""

    # Per-row quick-open search button (?q=Subject Name)
    q = _u.quote(subject_name)
    quick = (f'<a class="find-btn" href="{REMOTE_NAME_ABS}?q={q}" target="_blank" rel="noopener" '
             f'title="Open a filtered view for {subject_name}">Find</a>')

    headers.append(header_html)
    lineages.append(lineage_text)
    findcol.append(quick)

# Ensure constant used for the lineage column heading exists
LINEAGE_HEADER_SAFE = LINEAGE_HEADER if 'LINEAGE_HEADER' in globals() else "Lineage (Starting with oldest ancestor, the line is:)"

# Assemble the display dataframe (GUARDED so “Find” always exists)
df["Match Summary"]             = headers
df[LINEAGE_HEADER_SAFE]         = lineages
df["Find"]                      = findcol  # guarantees presence for [5/6]
display_df = df[["Find", "Match Summary", LINEAGE_HEADER_SAFE]]

# Column A CSV for side-uses
display_df[["Match Summary"]].to_csv(MATCH_COUSINS_CSV, index=False, encoding="iso-8859-15")
print("✅ Wrote local CSV (Column A):", os.path.abspath(MATCH_COUSINS_CSV))
# ====== CUT STOP [4/6] REFACTOR-Gold 2 — Transform & Column A (build display_df) ===================




# ====== CUT START [5/6] REFACTOR-Gold 2 — HTML (table + CSS + JS, no uploads) ======================
import html as _html

# Use this as the Home link everywhere
HOME_URL = "https://yates.one-name.net/ons_yates_dna_register.htm"

# ---------- Build HTML table ----------
# Assumes display_df + LINEAGE_HEADER_SAFE were prepared in earlier sections
display_for_html = display_df

html_table = display_for_html[["Find", "Match Summary", LINEAGE_HEADER_SAFE]].to_html(
    index=False, escape=False, classes="sortable"
)

# tag table + first row
html_table = html_table.replace(
    '<table border="1" class="dataframe sortable">',
    '<table border="1" class="dataframe sortable" id="refactor-table">', 1
)
html_table = html_table.replace('<tbody>\n<tr>', '<tbody>\n<tr id="first-row">', 1)

# widths via <colgroup>
FIND_PX = 110  # width of Find column (checkbox + Email + Find). Adjust here.
colgroup_html = (
    "<colgroup>\n"
    f"  <col style=\"width:{FIND_PX}px;\" />\n"
    f"  <col style=\"width:{COL_A_PX}px;\" />\n"
    "  <col />\n"
    "</colgroup>\n"
)
html_table = html_table.replace(
    '<table border="1" class="dataframe sortable" id="refactor-table">',
    '<table border="1" class="dataframe sortable" id="refactor-table">\n' + colgroup_html, 1
)

# add Select-all checkbox to header cell “Find”
html_table = html_table.replace(
    "<th>Find</th>",
    "<th>Find&nbsp;<input type=\"checkbox\" id=\"sel-all\" title=\"Select all visible\" /></th>",
    1
)

html_table_scrolling = '<div class="table-scroll">\n' + html_table + '\n</div>'

# ---------- CSS ----------
TABLE_CSS = (
    "<style type=\"text/css\">\n"
    "  html { scroll-behavior: smooth; }\n"
    "  body { font-family: Georgia, 'Times New Roman', serif; background:#ffffff; color:#222; margin:0; padding:0; line-height:1.5; }\n"
    f"  .wrap {{ max-width:{TABLE_WIDTH_PX}px; margin:0 auto; background:#ffffff; padding:20px; padding-bottom:48px; }}\n"
    "  a { color:#154b8b; text-decoration:none; } a:hover { text-decoration:underline; }\n"
    "  h1 { margin:0 0 6px 0; font-size:26px; line-height:1.2; text-align:center; }\n"
    "  .updated { font-size:12px; color:#555; text-align:center; margin:2px 0 10px 0; }\n"
    "  .sortbar { margin:6px 0 10px 0; font-size:13px; background:#ffffff; padding:6px 8px; border-radius:6px;\n"
    "             display:flex; flex-wrap:wrap; gap:5px; align-items:center; border:1px solid #ddd; }\n"
    "  .btn { display:inline-block; border:1px solid #5b79b8; background:#5b79b8; color:#fff; padding:4px 9px;\n"
    "         text-decoration:none; cursor:pointer; border-radius:5px; line-height:1.2; transition:background 0.2s, transform 0.1s; user-select:none; }\n"
    "  .btn:hover { background:#4668aa; transform:translateY(-1px); }\n"
    "  input.btn.search { background:#fff; color:#111; border-color:#bbb; }\n"
    "  .btn-mini { font-size:12px; padding:2px 6px; line-height:1.1; margin-left:6px; }\n"
    "  .find-cell { white-space:nowrap; }\n"
    "  .selbox { margin-right:6px; vertical-align:middle; }\n"
    "  .table-scroll { max-height:70vh; overflow-y:auto; overflow-x:auto; border:1px solid #ddd; }\n"
    f"  table.sortable {{ border-collapse:collapse; width:{TABLE_WIDTH_PX}px; table-layout:fixed; }}\n"
    "  table.sortable th, table.sortable td { border:1px solid #ddd; padding:6px 8px; vertical-align:top; }\n"
    "  table.sortable th { background:#e3eaf8; text-align:left; position:sticky; top:0; z-index:2; box-shadow:0 1px 0 #ccc; }\n"
    "  table.sortable td { word-wrap:break-word; overflow-wrap:break-word; }\n"
    "  #first-row td { border-top:2px solid #999; }\n"
    "  .back-to-top { position:fixed; right:16px; bottom:16px; padding:6px 10px; border:1px solid #3e5a97; background:#5b79b8; color:#fff;\n"
    "                 cursor:pointer; border-radius:6px; font-size:12px; display:none; z-index:9999; }\n"
    "  .back-to-top:hover { background:#4668aa; }\n"
    "  #dynamicContent { margin:10px 0 14px 0; }\n"
    "  @media screen and (max-width: 820px) { .wrap { padding:12px; } h1 { font-size:22px; } }\n"
    "</style>\n"
)

# ---------- Toolbar ----------
DYNAMIC_BLOCK = (
    "<div class=\"sortbar\">\n"
    "  <a class=\"btn\" href=\"https://yates.one-name.net/gengen/dna_cousin_surname_study.htm\" target=\"_blank\">Study Details</a>\n"
    "  <a class=\"btn\" href=\"https://yates.one-name.net/gengen/dna_theory_of_the_case.htm\" target=\"_blank\">Theory in Action</a>\n"
    "  <span class=\"btn\" data-sort-col=\"1\" data-sort-dir=\"asc\">Sort Match &uarr;</span>\n"
    "  <span class=\"btn\" data-sort-col=\"1\" data-sort-dir=\"desc\">Sort Match &darr;</span>\n"
    "  <span class=\"btn\" data-sort-col=\"2\" data-sort-dir=\"asc\">Sort Lineage &uarr;</span>\n"
    "  <span class=\"btn\" data-sort-col=\"2\" data-sort-dir=\"desc\">Sort Lineage &darr;</span>\n"
    "  <a class=\"btn\" href=\"gengen/images/cousin-calculator.jpg\" target=\"_blank\">Cousin Connection</a>\n"
    "  <a class=\"btn\" href=\"gengen/images/Shared_cM_Project_v4.jpg\" target=\"_blank\">Cousin by DNA</a>\n"
    "  <a class=\"btn\" href=\"partials/match_count.htm\" target=\"_blank\" rel=\"noopener\">Match Count</a>\n"
    "  <a class=\"btn\" href=\"partials/lineage_count.htm\" target=\"_blank\" rel=\"noopener\">Lineage Count</a>\n"
    "  <a class=\"btn\" href=\"/partials/cousin_list_print.htm\" target=\"_blank\">Cousin List</a>\n"
    "  <span class=\"btn\" id=\"email-selected\">Email Selected</span>\n"
    "  <span class=\"btn\" id=\"clear-selected\">Clear</span>\n"
    "  <input type=\"text\" id=\"search-box\" class=\"btn search\" size=\"24\" value=\"\" placeholder=\"Search&hellip;\" "
    "         autocomplete=\"off\" autocapitalize=\"off\" spellcheck=\"false\" inputmode=\"search\" enterkeyhint=\"search\" />\n"
    "</div>\n"
    "<div id=\"dynamicContent\"></div>\n"
)

# ---------- Full HTML (string only) ----------
JS_COUNT_URL = COUNT_PUBLIC_URL.replace("'", "%27")

# Updated status line (top, dynamic) with Home link
UPDATED_BLOCK = (
    "<div class=\"updated\">"
    f"<a href=\"{HOME_URL}\" target=\"_blank\" rel=\"noopener\">Home</a>"
    " &nbsp;|&nbsp; Last updated: <span id=\"last-updated\"></span>"
    " &nbsp;|&nbsp; Autosomal matches: <span id=\"auto-count\" class=\"js-count\"></span>"
    "</div>"
)

TEMPLATE_HTML = """<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>ONS Yates Study Autosomal DNA Register</title>
{TABLE_CSS}
</head>
<body id="top">
<div class="wrap">
  <h1>ONS Yates Study Autosomal DNA Register</h1>
  {UPDATED_BLOCK}
  {DYNAMIC_BLOCK}
{HTML_TABLE_SCROLLING}
</div>
<button id="back-to-top" class="back-to-top">&#9650; Top</button>
<script type="text/javascript">
//<![CDATA[
(function(){
  function textOf(cell){ return (cell && (cell.textContent || cell.innerText) || '').replace(/\\s+/g,' ').trim().toLowerCase(); }
  function sortTable(tbl, colIndex, dir){
    var tb=tbl && tbl.tBodies ? tbl.tBodies[0] : null; if(!tb) return;
    var rows=[].slice.call(tb.rows||[]);
    var asc=(dir==='asc');
    rows.sort(function(a,b){
      var A=textOf(a.cells[colIndex]), B=textOf(b.cells[colIndex]);
      if(A<B) return asc?-1:1; if(A>B) return asc?1:-1; return 0;
    });
    var frag=document.createDocumentFragment();
    for(var i=0;i<rows.length;i++) frag.appendChild(rows[i]);
    tb.appendChild(frag);
    updateSelAll(); // keep master checkbox in sync after sort
  }
  function bindSortButtons(){
    var tbl=document.getElementById('refactor-table'); var bar=document.querySelector('.sortbar'); if(!(tbl && bar)) return;
    bar.addEventListener('click',function(e){
      var btn=e.target && e.target.closest ? e.target.closest('.btn') : null; if(!btn) return;
      var colAttr=btn.getAttribute('data-sort-col'); if(colAttr==null) return; var col=parseInt(colAttr,10); if(isNaN(col)) return;
      var dir=btn.getAttribute('data-sort-dir')||'asc'; sortTable(tbl,col,dir); e.preventDefault(); return false;
    },false);
  }
  function bindHeaderSort(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tHead && tbl.tHead.rows.length)) return; var ths=tbl.tHead.rows[0].cells;
    for(var i=0;i<ths.length;i++)(function(idx){
      var dir='asc'; ths[idx].addEventListener('click',function(){ dir=(dir==='asc')?'desc':'asc'; sortTable(tbl,idx,dir); },false);
    })(i);
  }

  // partials
  var PARTIAL_BASES=['/partials/','partials/','gengen/partials/','/gengen/partials/'];
  function tryFetchSequential(urls,onOK,onFail){
    if(!urls.length) return onFail('No valid locations'); var url=urls.shift();
    fetch(url,{cache:'no-store'}).then(function(r){ if(!r.ok) throw new Error('HTTP '+r.status); return r.text();})
      .then(onOK).catch(function(){ tryFetchSequential(urls,onOK,onFail);});
  }
  function bindPartials(){
    var bar=document.querySelector('.sortbar'); if(!bar) return;
    bar.addEventListener('click',function(e){
      var btn=e.target && e.target.closest ? e.target.closest('.btn') : null; if(!btn) return;
      var rel=btn.getAttribute('data-load-partial'); if(!rel) return;
      var c=document.getElementById('dynamicContent'); if(!c) return; c.innerHTML='<p><em>Loading latest data&hellip;</em></p>';
      var bust=encodeURIComponent(document.lastModified||(new Date()).toUTCString());
      var bases=PARTIAL_BASES.slice(); var candidates=bases.map(function(b){return b+rel+'?v='+bust;});
      tryFetchSequential(candidates.slice(), function(html){ c.innerHTML=html; }, function(){ c.innerHTML='<p style=\\\"color:#a00;\\\">Could not load content.</p>'; });
    });
  }

  // updated stamp + count
  function stampLastUpdated(){
    var el=document.getElementById('last-updated'); if(!el) return;
    var d=new Date(document.lastModified||new Date());
    function z(n){return(n<10?'0':'')+n;}
    el.innerHTML=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());
  }
  function formatWithCommas(n){
    try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US'); }catch(e){ return String(n||''); }
  }
  function loadAutoCount(){
    var el=document.getElementById('auto-count'); if(!el) return;
    var url='{COUNT_URL}';
    try{
      var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);
      xhr.onreadystatechange=function(){ if(xhr.readyState===4){ if(xhr.status>=200&&xhr.status<300){
        var m=(xhr.responseText||'').match(/(\\d+)/); var num=m?m[1]:'';
        el.textContent = formatWithCommas(num) || '(unavailable)';
      } else { el.textContent='(unavailable)'; } } };
      xhr.send(null);
    }catch(e){ el.textContent='(unavailable)'; }
  }

  // search (filters rows; select-all respects visibility)
  function getParam(name){ var m=location.search.match(new RegExp('[?&]'+name+'=([^&]+)')); return m?decodeURIComponent(m[1].replace(/\\+/g,' ')):''; }
  function bindSearch(){
    var box=document.getElementById('search-box'); var tbl=document.getElementById('refactor-table'); if(!(box && tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb=tbl.tBodies[0];
    function norm(s){ return String(s||'').replace(/\\s+/g,' ').toLowerCase(); }
    function rowText(tr){ var t=''; for(var i=0;i<tr.cells.length;i++){ t+= ' ' + (tr.cells[i].textContent||tr.cells[i].innerText||''); } return norm(t); }
    var cached=[]; (function seed(){ var rows=tb.rows; cached=[]; for(var i=0;i<rows.length;i++){ cached.push({el:rows[i], txt:rowText(rows[i])}); } })();
    function apply(q){
      q=norm(q);
      for(var i=0;i<cached.length;i++){
        var hit = !q || cached[i].txt.indexOf(q)>-1;
        cached[i].el.style.display = hit? '' : 'none';
      }
      updateSelAll();
    }
    var to=null; function onInput(){ if(to) clearTimeout(to); to=setTimeout(function(){ apply(box.value); }, 60); }
    box.addEventListener('input', onInput, false);
    var q0=getParam('q');
    if(q0){ box.value=q0; apply(q0); try{ history.replaceState(null, '', location.pathname); }catch(e){} }
    else { box.value=''; apply(''); setTimeout(function(){ if(!getParam('q')){ box.value=''; apply(''); } }, 0); }
  }

  // --- selection controls ---
  function visibleRowCheckboxes(){
    var tbl=document.getElementById('refactor-table');
    if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return [];
    var tb=tbl.tBodies[0], out=[];
    for(var i=0;i<tb.rows.length;i++){
      var tr=tb.rows[i];
      if(tr.style.display==='none') continue;
      var cb = tr.cells[0] && tr.cells[0].querySelector('.selbox');
      if(cb) out.push(cb);
    }
    return out;
  }
  function updateSelAll(){
    var master=document.getElementById('sel-all'); if(!master) return;
    var cbs=visibleRowCheckboxes();
    if(!cbs.length){ master.indeterminate=false; master.checked=false; return; }
    var checked=cbs.filter(function(cb){ return cb.checked; }).length;
    master.checked = (checked===cbs.length);
    master.indeterminate = (checked>0 && checked<cbs.length);
  }
  function bindSelectAll(){
    var master=document.getElementById('sel-all'); if(!master) return;
    master.addEventListener('change', function(){
      var want=master.checked;
      var cbs=visibleRowCheckboxes();
      for(var i=0;i<cbs.length;i++) cbs[i].checked = want;
      updateSelAll();
    });
  }
  function pageBase(){ try{ return location.origin + location.pathname; }catch(e){ return location.pathname; } }
  function encodeQS(s){ return encodeURIComponent(String(s||'')); }
  function buildRowEmail(tr){
    var cells = tr.cells; if(!cells || cells.length < 3) return null;
    var summary = (cells[1].textContent||'').trim();
    var lineage = (cells[2].textContent||'').trim();
    var subj = 'ONS Yates Register: ' + summary.substring(0, 90);
    var qMatch = (function(){ try{ var em = cells[1].querySelector('strong'); return (em?em.textContent:'').trim(); }catch(e){ return ''; } })();
    var link = pageBase() + (qMatch? ('?q='+encodeQS(qMatch)) : '');
    var body = summary + '\\n\\nLineage:\\n' + lineage + '\\n\\nLink: ' + link;
    return {subject: subj, body: body};
  }
  function openMailto(subject, body){
    var href = 'mailto:?subject=' + encodeQS(subject) + '&body=' + encodeQS(body);
    window.location.href = href;
  }
  function injectRowControls(){
    var tbl = document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb = tbl.tBodies[0];
    for(var i=0;i<tb.rows.length;i++){
      (function(tr){
        var cell = tr.cells[0]; if(!cell) return; if(cell.className.indexOf('find-cell')===-1) cell.className += ' find-cell';
        var existingFind = cell.querySelector('a.find-btn'); // preserve your “Find” link
        var cb = document.createElement('input'); cb.type='checkbox'; cb.className='selbox'; cb.title='Select row';
        cb.addEventListener('change', updateSelAll);
        cell.insertBefore(cb, existingFind || cell.firstChild);
        // optional per-row Email button
        var emailBtn = document.createElement('a'); emailBtn.href='#'; emailBtn.className='btn btn-mini'; emailBtn.textContent='Email'; emailBtn.title='Email this row'; emailBtn.style.marginLeft='6px';
        emailBtn.addEventListener('click', function(ev){ ev.preventDefault(); var payload = buildRowEmail(tr); if(payload){ openMailto(payload.subject, payload.body); } });
        if(existingFind){ existingFind.insertAdjacentElement('afterend', emailBtn); } else { cell.appendChild(emailBtn); }
      })(tb.rows[i]);
    }
  }
  function bindBulkEmail(){
    var btn = document.getElementById('email-selected'); if(!btn) return;
    btn.addEventListener('click', function(){
      var tbl = document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb = tbl.tBodies[0];
      var parts = []; var count = 0;
      for(var i=0;i<tb.rows.length;i++){
        var tr = tb.rows[i]; if(tr.style.display==='none') continue;
        var cb = tr.cells[0] && tr.cells[0].querySelector('.selbox');
        if(cb && cb.checked){ var p = buildRowEmail(tr); if(p){ parts.push(p.body); count++; } }
      }
      if(!count){ alert('No rows selected. Tick the checkboxes in the Find column.'); return; }
      var subject = 'ONS Yates Register: ' + count + ' selection' + (count>1?'s':'');
      var body = parts.join('\\n\\n---\\n\\n');
      openMailto(subject, body);
    });
    var clr = document.getElementById('clear-selected'); if(clr){ clr.addEventListener('click', function(){
      var tbl = document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb = tbl.tBodies[0];
      for(var i=0;i<tb.rows.length;i++){ var cb = tb.rows[i].cells[0] && tb.rows[i].cells[0].querySelector('.selbox'); if(cb) cb.checked=false; }
      updateSelAll();
    }); }
  }

  // misc UI
  function setupBackToTop(){
    var btt=document.getElementById('back-to-top'); var cont=document.querySelector('.table-scroll');
    function onAny(){ var y=(window.scrollY||window.pageYOffset||0); var cy=cont?cont.scrollTop:0; btt.style.display=(y>200||cy>200)?'block':'none'; }
    window.addEventListener('scroll',onAny,{passive:true}); if(cont) cont.addEventListener('scroll',onAny,{passive:true});
    onAny(); if(btt){ btt.addEventListener('click',function(){ if(cont) cont.scrollTo({top:0,behavior:'smooth'}); window.scrollTo({top:0,behavior:'smooth'}); }); }
  }

  // init
  function init(){
    bindSortButtons(); bindHeaderSort(); bindPartials(); stampLastUpdated(); loadAutoCount();
    setupBackToTop(); bindSearch();
    injectRowControls(); bindSelectAll(); bindBulkEmail(); updateSelAll();
  }
  init();
})();
//]]>
</script>
</body></html>
"""

FULL_HTML = (TEMPLATE_HTML
    .replace("{TABLE_CSS}", TABLE_CSS)
    .replace("{UPDATED_BLOCK}", UPDATED_BLOCK)
    .replace("{DYNAMIC_BLOCK}", DYNAMIC_BLOCK)
    .replace("{HTML_TABLE_SCROLLING}", html_table_scrolling)
    .replace("{COUNT_URL}", JS_COUNT_URL)
)
# ====== CUT STOP [5/6] REFACTOR-Gold 2 — HTML (table + CSS + JS, no uploads) =======================




# ====== CUT START [6/6] REFACTOR-Gold 2 — Save + Upload + Reports + Partials + Printable ===========
import os, re, posixpath
from collections import Counter
import pandas as pd

ENC          = "iso-8859-15"
LOCAL_NAME   = globals().get("LOCAL_NAME",  "ons_yates_dna_register.htm")
REMOTE_NAME  = globals().get("REMOTE_NAME", "ons_yates_dna_register.htm")
HOME_URL     = "https://yates.one-name.net/ons_yates_dna_register.htm"

# Safely embed the JS count URL (escape single quotes)
JS_COUNT_URL = COUNT_PUBLIC_URL.replace("'", "%27")

# 1) Write main page HTML exactly as built upstream
with open(LOCAL_NAME, "w", encoding=ENC, errors="xmlcharrefreplace") as f:
    f.write(FULL_HTML)
print(f"✅ Wrote HTML: {os.path.abspath(LOCAL_NAME)}")

# 2) Upload main page + optional assets
with ftp_connect() as ftps:
    try:
        ftps.mkd("partials")
    except Exception:
        pass
    try:
        ftps.delete(_remote_path(REMOTE_NAME))
    except Exception:
        pass

    ftp_upload_overwrite(ftps, LOCAL_NAME, _remote_path(REMOTE_NAME))

    if os.path.exists(MATCH_COUSINS_CSV):
        ftp_upload_overwrite(ftps, MATCH_COUSINS_CSV, _remote_path(MATCH_COUSINS_CSV))

    if os.path.exists(LOCAL_COUNT_FILE):
        ftp_upload_overwrite(ftps, LOCAL_COUNT_FILE, _remote_path(REMOTE_COUNT_NAME))
        print(f"✅ Published autosomal count: {LOCAL_COUNT_FILE} → {COUNT_PUBLIC_URL}")
    else:
        print(f"⚠️ Count file not found locally: {LOCAL_COUNT_FILE}")

    try:
        ftps.quit()
    except Exception:
        pass

# 3) Build filtered data for reports
SUPPRESS_PREFIXES = ["y-dna", "dar", "sar"]
def _suppress_by_match_summary(ms: str) -> bool:
    s = (str(ms) or "").strip().lower()
    return bool(s) and any(s.startswith(prefix + " is") for prefix in SUPPRESS_PREFIXES)

display_df_all = display_df.copy()
_suppress_mask = display_df_all["Match Summary"].map(_suppress_by_match_summary)
display_df_filtered = display_df_all[~_suppress_mask].reset_index(drop=True)
filtered_df_for_reports = df.loc[display_df_all.index[~_suppress_mask]].copy()

# Bind resolver-code column name (i.e., "Match to") for safety
subject_code_col = (
    globals().get("subject_code_col")
    or globals().get("match_to_col")
    or ("Match to" if 'df' in globals() and "Match to" in df.columns else list(df.columns)[0])
)


# Resolver usage / counts
used_series = filtered_df_for_reports[subject_code_col].astype(str).map(lambda x: str(x).strip().lower())
counts = Counter([c for c in used_series if c and c != "nan"])

rows = []
all_keys = set(globals().get("MATCH_TO_UNMASKED", {}).keys())
for code in sorted(all_keys):
    rows.append((code, globals()["MATCH_TO_UNMASKED"].get(code, ""), counts.get(code, 0)))
for code in sorted(set(counts.keys()) - all_keys):
    rows.append((code, "(unmapped)", counts.get(code, 0)))

usage_df = pd.DataFrame(rows, columns=["Match to (code)", "Unmasked", "Count"]).sort_values(
    ["Match to (code)"], ascending=[True]
)
usage_df_alpha = usage_df.reset_index(drop=True)

RESOLVER_USAGE_CSV = "resolver_usage_report.csv"
usage_df_alpha.to_csv(RESOLVER_USAGE_CSV, index=False, encoding=ENC)
print("✅ Wrote resolver usage CSV:", os.path.abspath(RESOLVER_USAGE_CSV))

# Lineage Count
def _escape_html(s: str) -> str:
    return (str(s).replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;"))

def _norm_key_component(fullname: str) -> str:
    nm = smart_titlecase(str(fullname)); parts = nm.split()
    if not parts: return ""
    last   = re.sub(r"[^A-Za-z]", "", parts[-1])
    firsts = re.sub(r"[^A-Za-z]", "", "".join(parts[:-1]))[:7]
    return f"{last}{firsts}"

def oldest_pair_key(token_list):
    if not token_list: return ""
    first = token_list[0]
    a, b = derive_common_from_first_token([first])
    if not a and not b:
        parts = re.split(r"\s*&\s*", first)
        a = parts[0] if parts else ""
        b = parts[1] if len(parts) > 1 else ""
    ka = _norm_key_component(a) if a else ""
    kb = _norm_key_component(b) if b else ""
    return f"{ka}&{kb}" if ka or kb else ""

lc_counter = Counter()
for _, row in filtered_df_for_reports.iterrows():
    toks = split_tokens(row[path_col])
    key = oldest_pair_key(toks)
    if key: lc_counter[key] += 1

lc_rows = sorted(lc_counter.items(), key=lambda x: (-x[1], x[0]))
lineage_count_df = pd.DataFrame(lc_rows, columns=["Oldest Distant Ancestor (normalized Last+First7) pair", "Count"])

LINEAGE_COUNT_CSV = "lineage_count_report.csv"
lineage_count_df.to_csv(LINEAGE_COUNT_CSV, index=False, encoding=ENC)
print("✅ Wrote lineage count CSV:", os.path.abspath(LINEAGE_COUNT_CSV))

# Upload report CSVs
try:
    with ftp_connect() as ftps:
        ftp_upload_overwrite(ftps, RESOLVER_USAGE_CSV, _remote_path(RESOLVER_USAGE_CSV))
        ftp_upload_overwrite(ftps, LINEAGE_COUNT_CSV, _remote_path(LINEAGE_COUNT_CSV))
        try: ftps.quit()
        except Exception: pass
    print("✅ Published report CSVs.")
except Exception as e:
    print(f"⚠️ Upload of report CSVs failed: {e}")

# ---------- Helpers for Match Count partial ----------
def _build_person_details_html(code_lc: str) -> str:
    mask = filtered_df_for_reports[subject_code_col].astype(str).str.strip().str.lower() == code_lc
    if not mask.any(): return "<em>No rows for this person.</em>"
    rows_idx = filtered_df_for_reports[mask].index
    mini = display_df.loc[rows_idx, ["Find", "Match Summary", LINEAGE_HEADER_SAFE]].copy()
    return mini.to_html(index=False, escape=False, classes="mini-table", border=0)

# ---------- Match Count partial (tight wrapper + working toolbar) ----------
def _render_match_count_full_html(usage_alpha: pd.DataFrame) -> str:
    try:
        total_persons = int(usage_alpha.shape[0])
        total_matches = int(pd.to_numeric(usage_alpha["Count"], errors="coerce").fillna(0).sum())
    except Exception:
        total_persons, total_matches = usage_alpha.shape[0], 0

    # Build rows
    rows_html = []
    for _, r in usage_alpha.iterrows():
        code    = str(r["Match to (code)"])
        code_lc = code.strip().lower()
        unm     = _escape_html(r["Unmasked"])
        cnt     = int(pd.to_numeric(r["Count"], errors="coerce")) if str(r["Count"]).strip() else 0
        rows_html.append(
            "<tr class='top-row' data-code='{code_lc}'>"
            "<td class='sel'><input type='checkbox' class='selbox' data-code='{code_lc}' title='Select' /></td>"
            "<td class='code'>{code}</td>"
            "<td class='unmasked'>{unm}</td>"
            "<td class='count'><a href='#' class='count-toggle' data-code='{code_lc}' aria-expanded='false'>{cnt}</a></td>"
            "</tr>".format(code_lc=_escape_html(code_lc), code=_escape_html(code), unm=unm, cnt=f"{cnt:,}")
        )
        rows_html.append(
            "<tr class='details-row' id='details-{code_lc}' style='display:none;'>"
            "<td colspan='4'><div class='details-outer'>{detail}</div></td>"
            "</tr>".format(code_lc=_escape_html(code_lc), detail=_build_person_details_html(code_lc))
        )

    # CSS (avoid %-formatting; use concatenation for injected values)
    style = (
        "<style>"
        " body{font-family:Georgia,'Times New Roman',serif;background:#fff;color:#111;margin:0;padding:18px;line-height:1.6;font-size:15px}"
        " .wrap{max-width:900px;margin:0 auto}"
        " h2{font-size:20px;margin:0 0 10px 0;text-align:center;border-bottom:2px solid #5b79b8;padding-bottom:6px}"
        " .updated{font-size:12px;color:#555;text-align:center;margin:2px 0 10px 0}"
        " .stats{text-align:center;color:#444;font-size:13px;margin:6px 0 10px 0}"
        " .toolbar{display:flex;flex-wrap:wrap;gap:8px;align-items:center;justify-content:flex-start;margin:8px 0 10px 0}"
        " .btn{display:inline-block;border:1px solid #5b79b8;background:#5b79b8;color:#fff;padding:6px 12px;border-radius:6px;text-decoration:none;cursor:pointer}"
        " .btn:hover{background:#4668aa}"
        " .master{margin-left:auto;font-size:13px;color:#333}"
        " table#mc-table{border-collapse:collapse;width:100%;table-layout:fixed}"
        " #mc-table col.sel{width:72px} #mc-table col.code{width:26%} #mc-table col.unmasked{width:48%} #mc-table col.count{width:16%}"
        " #mc-table th,#mc-table td{border:1px solid #ccc;padding:8px 10px;vertical-align:top;word-wrap:break-word;overflow-wrap:break-word}"
        " #mc-table th{background:#e9eef9;position:sticky;top:0;z-index:1}"
        " #mc-table td.count{text-align:right}"
        " #mc-table a.count-toggle{display:inline-block;padding:2px 6px;background:#5b79b8;color:#fff;border-radius:4px;border:1px solid #3e5a97;text-decoration:none}"
        " #mc-table a.count-toggle:hover{background:#4668aa}"
        " .details-outer{background:#f9fbff;border:1px solid #d7e1fb;margin:8px 2px;padding:8px;border-radius:6px}"
        " .mini-table{border-collapse:collapse;width:100%;table-layout:fixed}"
        " .mini-table th,.mini-table td{border:1px solid #ddd;padding:6px 8px;vertical-align:top}"
        " .mini-table th{background:#f2f6ff}"
        " .mini-table col:nth-child(1){width:140px} "
        " .mini-table col:nth-child(2){width:" + str(COL_A_PX) + "px} "
        " .mini-table col:nth-child(3){width:auto}"
        " @media screen and (max-width:900px){ body{padding:12px;font-size:14px} .toolbar{gap:6px} }"
        "</style>"
    )

    updated = (
        "<div class='updated'>"
        "<a href='" + HOME_URL + "' target='_blank' rel='noopener'>Home</a>"
        " &nbsp;|&nbsp; Last updated: <span id='last-updated'></span>"
        " &nbsp;|&nbsp; Autosomal matches: <span id='auto-count' class='js-count'></span>"
        "</div>"
    )

    toolbar = (
        "<div class='toolbar'>"
        " <button type='button' class='btn' id='btn-display'>Display Checked</button>"
        " <button type='button' class='btn' id='btn-email'>Email Checked</button>"
        " <button type='button' class='btn' id='btn-print'>Print Checked</button>"
        " <label class='master'><input type='checkbox' id='sel-all' /> Select all</label>"
        "</div>"
    )

    html = (
        "<!DOCTYPE html><html lang='en'><head>"
        "<meta charset='iso-8859-15' />"
        "<meta name='viewport' content='width=device-width, initial-scale=1.0' />"
        "<title>Match Count (by resolver code)</title>"
        + style +
        "</head><body>"
        "<div class='wrap'>"
        "<h2>Match Count (by resolver code, A&rightarrow;Z)</h2>"
        + updated +
        "<div class='stats'>Total distinct &ldquo;Match to&rdquo; persons: " + f"{total_persons:,}" +
        " &nbsp;|&nbsp; Total matches counted: " + f"{total_matches:,}" + "</div>"
        + toolbar +
        "<table id='mc-table'>"
        "<colgroup><col class='sel'><col class='code'><col class='unmasked'><col class='count'></colgroup>"
        "<thead><tr><th>Select</th><th>Match to (code)</th><th>Unmasked</th><th>Count</th></tr></thead>"
        "<tbody>" + "".join(rows_html) + "</tbody></table>"
        "</div>"
        "<script>(function(){"
        " document.addEventListener('DOMContentLoaded', function(){"
        "  function z(n){return(n<10?'0':'')+n;}"
        "  function stamp(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date());"
        "   el.textContent=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());}"
        "  function fmt(n){try{var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US');}catch(e){return String(n||'');}}"
        "  function loadCnt(){var el=document.getElementById('auto-count'); if(!el) return; var url='" + JS_COUNT_URL + "';"
        "   try{var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);"
        "    xhr.onreadystatechange=function(){ if(xhr.readyState===4){ if(xhr.status>=200&&xhr.status<300){ var m=(xhr.responseText||'').match(/(\\d+)/); el.textContent = fmt(m?m[1]:'') || '(unavailable)'; } else { el.textContent='(unavailable)'; } } };"
        "    xhr.send(null);}catch(e){ el.textContent='(unavailable)'; }}"
        "  stamp(); loadCnt();"

        "  function toggleDetails(code, wantOpen){"
        "    var row=document.getElementById('details-'+code); if(!row) return;"
        "    row.style.display = wantOpen ? '' : 'none';"
        "    var a=document.querySelector(\"a.count-toggle[data-code='\"+code+\"']\"); if(a){ a.setAttribute('aria-expanded', wantOpen?'true':'false'); }"
        "  }"
        "  function selectedCodes(){"
        "    var out=[]; var cbs=document.querySelectorAll('#mc-table .selbox');"
        "    for(var i=0;i<cbs.length;i++){ if(cbs[i].checked){ var c=cbs[i].getAttribute('data-code'); if(c) out.push(c); }}"
        "    return out;"
        "  }"

        "  var master=document.getElementById('sel-all');"
        "  if(master){ master.addEventListener('change', function(){"
        "    var want=master.checked; var cbs=document.querySelectorAll('#mc-table .selbox');"
        "    for(var i=0;i<cbs.length;i++){ cbs[i].checked = want; }"
        "  }); }"

        "  var btnDisp=document.getElementById('btn-display');"
        "  if(btnDisp){ btnDisp.addEventListener('click', function(){"
        "    var sel=selectedCodes();"
        "    var all=document.querySelectorAll('tr.details-row'); for(var i=0;i<all.length;i++){ all[i].style.display='none'; }"
        "    for(var j=0;j<sel.length;j++){ toggleDetails(sel[j], true); }"
        "    if(sel.length){ try{ document.getElementById('details-'+sel[0]).scrollIntoView({behavior:'smooth',block:'start'});}catch(e){} }"
        "  }); }"

        "  function enc(s){ return encodeURIComponent(String(s||'')); }"
        "  function textOf(el){ return (el && (el.textContent||el.innerText)||'').trim(); }"

        "  var btnEmail=document.getElementById('btn-email');"
        "  if(btnEmail){ btnEmail.addEventListener('click', function(){"
        "    var sel=selectedCodes(); if(!sel.length){ alert('No persons selected.'); return; }"
        "    function collectMatches(code){"
        "      var out=[]; var host=document.getElementById('details-'+code); if(!host) return out;"
        "      var rows=host.querySelectorAll('table.mini-table tbody tr');"
        "      for(var i=0;i<rows.length;i++){"
        "        var tds=rows[i].cells; if(!tds || tds.length<3) continue;"
        "        var a=tds[0].querySelector('a'); var href=a ? a.href : '';"
        "        var summary=textOf(tds[1]);"
        "        out.push({summary:summary, href:href});"
        "      }"
        "      return out;"
        "    }"
        "    var lines=[], total=0;"
        "    for(var k=0;k<sel.length;k++){"
        "      var code=sel[k];"
        "      var tr=document.querySelector(\"tr.top-row[data-code='\"+code+\"']\"); if(!tr) continue;"
        "      var codeTxt=textOf(tr.querySelector('.code'));"
        "      var unmTxt =textOf(tr.querySelector('.unmasked'));"
        "      var cntTxt =textOf(tr.querySelector('.count a'));"
        "      var header = codeTxt + (unmTxt? ' — '+unmTxt : '') + (cntTxt? ' ('+cntTxt+')' : '');"
        "      lines.push('== ' + header + ' ==');"
        "      var matches = collectMatches(code);"
        "      if(!matches.length){"
        "        lines.push('- (no inline rows available)');"
        "      } else {"
        "        total += matches.length;"
        "        for(var m=0;m<matches.length;m++){"
        "          var row = matches[m];"
        "          lines.push('- ' + row.summary);"
        "          if(row.href){ lines.push(row.href); }"
        "        }"
        "      }"
        "      if(k < sel.length-1) lines.push('');"
        "    }"
        "    var subj='ONS Yates: ' + sel.length + ' person(s), ' + total + ' match row(s)';"
        "    var body=lines.join('\\n');"
        "    var href='mailto:?subject='+enc(subj)+'&body='+enc(body);"
        "    window.location.href=href;"
        "  }); }"

        "  var btnPrint=document.getElementById('btn-print');"
        "  if(btnPrint){ btnPrint.addEventListener('click', function(){"
        "    var sel=selectedCodes(); if(!sel.length){ alert('No persons selected.'); return; }"
        "    for(var i=0;i<sel.length;i++){ toggleDetails(sel[i], true); }"
        "    setTimeout(function(){ window.print(); }, 100);"
        "  }); }"

        "  var tbl=document.getElementById('mc-table');"
        "  if(tbl){ tbl.addEventListener('click',function(ev){"
        "    var a=ev.target && ev.target.closest ? ev.target.closest('a.count-toggle') : null;"
        "    if(!a) return; ev.preventDefault();"
        "    var code=a.getAttribute('data-code'); if(!code) return;"
        "    var row=document.getElementById('details-'+code); if(!row) return;"
        "    var open=row.style.display!== 'none';"
        "    row.style.display = open ? 'none' : '';"
        "    a.setAttribute('aria-expanded', open?'false':'true');"
        "    if(!open){ try{ row.scrollIntoView({behavior:'smooth',block:'start'});}catch(e){} }"
        "  },false); }"
        " });"
        "})();</script>"
        "</body></html>"
    )
    return html

match_partial_html = _render_match_count_full_html(usage_df_alpha)

# ---------- Lineage Count partial (keeps dynamic header) ----------
lineage_style = (
    "<style>"
    " body{font-family:Georgia,'Times New Roman',serif;background:#fff;color:#111;margin:0;padding:18px;line-height:1.6;font-size:15px}"
    " .wrap{max-width:1100px;margin:0 auto}"
    " h2{font-size:20px;margin:0 0 10px 0;text-align:center;border-bottom:2px solid #5b79b8;padding-bottom:6px}"
    " .updated{font-size:12px;color:#555;text-align:center;margin:2px 0 10px 0}"
    " table.partial-table{border-collapse:collapse;width:100%;table-layout:auto}"
    " table.partial-table th,table.partial-table td{border:1px solid #ccc;padding:8px 10px;vertical-align:top}"
    " table.partial-table th{background:#e9eef9;position:sticky;top:0}"
    " @media screen and (max-width:900px){ body{padding:12px;font-size:14px} }"
    "</style>"
)
lineage_updated = (
    "<div class='updated'>"
    "<a href='" + HOME_URL + "' target='_blank' rel='noopener'>Home</a>"
    " &nbsp;|&nbsp; Last updated: <span id='last-updated'></span>"
    " &nbsp;|&nbsp; Autosomal matches: <span id='auto-count' class='js-count'></span>"
    "</div>"
)
lineage_js = (
    "<script>(function(){"
    " document.addEventListener('DOMContentLoaded', function(){"
    "  function z(n){return(n<10?'0':'')+n;}"
    "  function stamp(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date());"
    "   el.textContent=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());}"
    "  function fmt(n){try{var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US');}catch(e){return String(n||'');}}"
    "  function loadCnt(){var el=document.getElementById('auto-count'); if(!el) return; var url='" + JS_COUNT_URL + "';"
    "   try{var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);"
    "    xhr.onreadystatechange=function(){ if(xhr.readyState===4){ if(xhr.status>=200&&xhr.status<300){ var m=(xhr.responseText||'').match(/(\\d+)/); el.textContent = fmt(m?m[1]:'') || '(unavailable)'; } else { el.textContent='(unavailable)'; } } };"
    "    xhr.send(null);}catch(e){ el.textContent='(unavailable)'; }}"
    "  stamp(); loadCnt();"
    " });"
    "})();</script>"
)

lineage_partial_html = (
    "<!DOCTYPE html><html lang='en'><head>"
    "<meta charset='iso-8859-15' />"
    "<meta name='viewport' content='width=device-width, initial-scale=1.0' />"
    "<title>Lineage Count (by oldest distant ancestor pair)</title>"
    + lineage_style +
    "</head><body>"
    "<div class='wrap'>"
    "<h2>Lineage Count (by oldest distant ancestor pair)</h2>"
    + lineage_updated +
    lineage_count_df.to_html(index=False, escape=False, classes='partial-table', border=0) +
    "</div>"
    + lineage_js +
    "</body></html>"
)

# 4) Write & upload partials
with open("match_count.htm", "w", encoding=ENC, errors="xmlcharrefreplace") as f:
    f.write(match_partial_html)
with open("lineage_count.htm", "w", encoding=ENC, errors="xmlcharrefreplace") as f:
    f.write(lineage_partial_html)
print("✅ Wrote partials locally: match_count.htm, lineage_count.htm")

try:
    with ftp_connect() as ftps:
        try: ftps.mkd("partials")
        except Exception: pass
        ftp_upload_overwrite(ftps, "match_count.htm",   "partials/match_count.htm")
        ftp_upload_overwrite(ftps, "lineage_count.htm", "partials/lineage_count.htm")
        try: ftps.quit()
        except Exception: pass
    print("✅ Published count partials to /partials/")
except Exception as e:
    print(f"⚠️ Upload of count partials failed: {e}")

# 5) Printable page (adds dynamic header)
PRINT_PARTIAL_NAME  = "cousin_list_print.htm"
PRINT_PARTIAL_LOCAL = PRINT_PARTIAL_NAME
_print_table_html = (display_df_filtered if not display_df_filtered.empty else display_df_all).to_html(
    index=False, escape=False, classes="print-table", border=1
)
PRINT_PARTIAL_HTML = (
    "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\"\n"
    "  \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">\n"
    "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\">\n"
    "<head>\n"
    "<meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />\n"
    "<meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\" />\n"
    "<title>ONS Yates Study &ndash; Cousin List (Printable)</title>\n"
    "<style type=\"text/css\">\n"
    "  body { font-family: Georgia, 'Times New Roman', serif; color:#000; background:#fff; margin:0; padding:16px; line-height:1.45; }\n"
    "  .wrap { max-width: 1100px; margin: 0 auto; }\n"
    "  h1 { font-size: 22px; margin: 0 0 8px 0; text-align:center; }\n"
    "  .updated { font-size:12px; color:#555; text-align:center; margin:2px 0 12px 0; }\n"
    "  .actions { text-align:center; margin: 10px 0 16px 0; }\n"
    "  .btn { display:inline-block; border:1px solid #444; background:#f4f4f4; color:#000; padding:6px 12px; text-decoration:none; }\n"
    "  table.print-table { border-collapse: collapse; width: 100%; table-layout: fixed; }\n"
    "  table.print-table th, table.print-table td { border:  1px solid #222; padding: 6px 8px; vertical-align: top; }\n"
    "  table.print-table th { background: #eee; }\n"
    "  @media print { .actions { display: none; } @page { margin: 0.6in; } }\n"
    "</style>\n"
    "</head>\n"
    "<body>\n"
    "  <div class=\"wrap\">\n"
    "    <h1>ONS Yates Study &ndash; Cousin List (Printable)</h1>\n"
    "    <div class=\"updated\"><a href=\"" + HOME_URL + "\" target=\"_blank\" rel=\"noopener\">Home</a>"
    " &nbsp;|&nbsp; Last updated: <span id=\"last-updated\"></span>"
    " &nbsp;|&nbsp; Autosomal matches: <span id=\"auto-count\" class=\"js-count\"></span></div>\n"
    "    <div class=\"actions\"><a class=\"btn\" href=\"#\" onclick=\"window.print();return false;\">Print this page</a></div>\n"
    "   " + _print_table_html + "\n"
    "    <div class=\"actions\" style=\"margin-top:16px;\"><a class=\"btn\" href=\"#\" onclick=\"window.print();return false;\">Print this page</a></div>\n"
    "  </div>\n"
    "<script>(function(){"
    " document.addEventListener('DOMContentLoaded', function(){"
    "  function z(n){return(n<10?'0':'')+n;}"
    "  function stamp(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date());"
    "   el.textContent=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());}"
    "  function fmt(n){try{var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US');}catch(e){return String(n||'');}}"
    "  function loadCnt(){var el=document.getElementById('auto-count'); if(!el) return; var url='" + JS_COUNT_URL + "';"
    "   try{var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);"
    "    xhr.onreadystatechange=function(){ if(xhr.readyState===4){ if(xhr.status>=200&&xhr.status<300){ var m=(xhr.responseText||'').match(/(\\d+)/); el.textContent = fmt(m?m[1]:'') || '(unavailable)'; } else { el.textContent='(unavailable)'; } } };"
    "    xhr.send(null);}catch(e){ el.textContent='(unavailable)'; }}"
    "  stamp(); loadCnt();"
    " });"
    "})();</script>"
    "</body>\n"
    "</html>\n"
)
with open(PRINT_PARTIAL_LOCAL, "w", encoding=ENC, errors="xmlcharrefreplace") as f:
    f.write(PRINT_PARTIAL_HTML)

try:
    with ftp_connect() as ftps:
        try: ftps.mkd("partials")
        except Exception: pass
        ftp_upload_overwrite(ftps, PRINT_PARTIAL_LOCAL, "partials/" + PRINT_PARTIAL_NAME)
        try: ftps.quit()
        except Exception: pass
    print(f"✅ Uploaded printable page: /partials/{PRINT_PARTIAL_NAME}")
except Exception as e:
    print(f"⚠️ Printable page upload failed: {e}")

print(f"✅ HTML published at https://yates.one-name.net/{REMOTE_NAME}")
# ====== CUT STOP [6/6] REFACTOR-Gold 2 — Save + Upload + Reports + Partials + Printable ===========




# ====== CUT START [B] REFACTOR-Gold 2 — Helpers / Resolver (root-level /partials + utilities) ====
import os
import re
import csv
import posixpath
from typing import Dict, Tuple, List, Optional

try:
    import pandas as pd
except Exception:
    pd = None

# ---------- Absolute root-level /partials support (Resolver) ----------
# Primary:   /partials/match_to_unmasked.csv   (absolute from server root)
# Fallback:  /gengen/partials/match_to_unmasked.csv  (under FTP_DIR)
SERVER_PARTIALS_DIR = os.getenv("G2_SERVER_PARTIALS_DIR", "/partials")
SERVER_MAPPING_BASENAME = os.getenv("G2_SERVER_MAPPING_BASENAME", "match_to_unmasked.csv")
LOCAL_MAPPING_CACHE = os.getenv("G2_LOCAL_MAPPING_CACHE", "match_to_unmasked.csv")

def _remote_abs(*parts: str) -> str:
    """Absolute POSIX path; DOES NOT prepend FTP_DIR."""
    cleaned = [str(p).strip("/") for p in parts if p is not None]
    return "/" + "/".join([p for p in cleaned if p])

def _remote_under_ftpdir(name: str) -> str:
    """Fallback to FTP_DIR-prefixed path (e.g., /gengen/partials/...)."""
    base = str(globals().get("FTP_DIR", "/")).rstrip("/")
    return f"{base}/{name.lstrip('/')}"

def _ftps_fetch_to_local(ftps, remote_path: str, local_path: str) -> Tuple[bool, str]:
    """Try to RETR a remote file to local_path. Return (ok, error_text)."""
    try:
        parent = os.path.dirname(local_path)
        if parent and not os.path.isdir(parent):
            os.makedirs(parent, exist_ok=True)
        with open(local_path, "wb") as fh:
            ftps.retrbinary(f"RETR {remote_path}", fh.write)
        return True, ""
    except Exception as e:
        return False, f"{type(e).__name__}: {e}"

def _detect_cols(df) -> Tuple[str, str]:
    """Pick code and unmasked columns from a DataFrame by heuristics."""
    cols = [str(c) for c in df.columns]
    code_candidates = ["Match to", "match_to", "match to", "code", "Code"]
    name_candidates = ["Unmasked", "unmasked", "name", "Name", "Unmask", "unmask"]
    code_col = None
    name_col = None
    for c in cols:
        if c.lower().replace(" ", "") in [x.replace(" ", "").lower() for x in code_candidates]:
            code_col = c; break
    for c in cols:
        if c.lower().replace(" ", "") in [x.replace(" ", "").lower() for x in name_candidates]:
            name_col = c; break
    if code_col is None: code_col = cols[0]
    if name_col is None: name_col = cols[1] if len(cols) > 1 else cols[0]
    return code_col, name_col

def _read_resolver_csv(local_path: str) -> Dict[str, str]:
    """Load resolver csv to {code: unmasked} with minimal normalization."""
    if pd is not None:
        df = pd.read_csv(local_path, dtype=str, encoding=ENC, keep_default_na=False)
        code_col, name_col = _detect_cols(df)
        mapping = {}
        for _, row in df.iterrows():
            k = str(row.get(code_col, "")).strip()
            v = str(row.get(name_col, "")).strip()
            if k:
                mapping[k] = v
        return mapping
    # Fallback without pandas
    mapping = {}
    with open(local_path, "r", encoding=ENC, errors="replace", newline="") as fh:
        reader = csv.reader(fh)
        rows = list(reader)
    if not rows:
        return mapping
    header = rows[0]
    if len(header) >= 2 and not header[0].strip().isdigit():
        # Assume first row is header
        data_rows = rows[1:]
        code_idx = 0
        name_idx = 1
    else:
        # No header
        data_rows = rows
        code_idx = 0
        name_idx = 1 if len(rows[0]) > 1 else 0
    for r in data_rows:
        if not r:
            continue
        k = (r[code_idx] if len(r) > code_idx else "").strip()
        v = (r[name_idx] if len(r) > name_idx else "").strip()
        if k:
            mapping[k] = v
    return mapping

def load_resolver_from_server() -> Dict[str, str]:
    """Load resolver from /partials first, then /gengen/partials. Cache locally."""
    if "ftp_connect" not in globals():
        raise RuntimeError("ftp_connect() is not available; ensure Section [A] is loaded first.")
    ftps = ftp_connect()
    try:
        # Primary: absolute root /partials
        primary = _remote_abs(SERVER_PARTIALS_DIR, SERVER_MAPPING_BASENAME)  # e.g., /partials/match_to_unmasked.csv
        ok, err = _ftps_fetch_to_local(ftps, primary, LOCAL_MAPPING_CACHE)
        if not ok:
            # Fallback: FTP_DIR/partials (e.g., /gengen/partials/...)
            fallback = _remote_under_ftpdir(posixpath.join("partials", SERVER_MAPPING_BASENAME))
            ok2, err2 = _ftps_fetch_to_local(ftps, fallback, LOCAL_MAPPING_CACHE)
            if not ok2:
                raise RuntimeError(
                    "Resolver not found on server at either location:\n"
                    f"  1) {primary}\n"
                    f"  2) {fallback}\n"
                    "Action: Upload match_to_unmasked.csv to /partials/ (root) or to /gengen/partials/ and re-run.\n"
                    f"Details: primary error: {err}; fallback error: {err2}"
                )
        mapping = _read_resolver_csv(LOCAL_MAPPING_CACHE)
        return mapping
    finally:
        try:
            ftps.quit()
        except Exception:
            pass

# Global resolver dict (available to other sections)
MATCH_TO_UNMASKED = load_resolver_from_server()

# ---------- Resolver helpers exposed to other sections ----------
def resolve_match_to(code_value: str) -> str:
    """Return unmapped name for a resolver 'Match to' code, or empty string if unknown."""
    if not code_value and code_value != 0:
        return ""
    code = str(code_value).strip()
    if not code:
        return ""
    mapping = globals().get("MATCH_TO_UNMASKED", {})
    return mapping.get(code, "")

def normalize_person_name(s: str) -> str:
    """Return a title-cased person name with safe punctuation for XHTML/iso-8859-15."""
    if not isinstance(s, str):
        return ""
    s = s.strip()
    if not s:
        return ""
    s = re.sub(r"\s+", " ", s)
    def _title_token(tok: str) -> str:
        if not tok:
            return tok
        if len(tok) == 1 and tok.isalpha():
            return tok.upper() + "."
        if "-" in tok:
            return "-".join([p.capitalize() if p else "" for p in tok.split("-")])
        return tok.capitalize()
    parts = s.split(" ")
    fixed = [_title_token(p) for p in parts]
    name = " ".join(fixed)
    import html as _h
    return _h.escape(name, quote=True)

# ---------- General Helpers (used by later sections) ----------
def find_col(df, regex_list: Optional[List[str]] = None, fallback_list: Optional[List[str]] = None) -> Optional[str]:
    """Find a column by regex OR fallback names; returns the first match or None."""
    if df is None or getattr(df, "columns", None) is None:
        return None
    regex_list = regex_list or []
    fallback_list = fallback_list or []
    cols = list(df.columns)

    # Try regex matches
    for pattern in regex_list:
        try:
            rx = re.compile(pattern, flags=re.IGNORECASE)
        except Exception:
            continue
        for c in cols:
            if rx.search(str(c)):
                return c

    # Try fallback exact, case-insensitive
    for name in fallback_list:
        target = str(name).strip().lower()
        for c in cols:
            if str(c).strip().lower() == target:
                return c

    # Last resort: first column if present
    return cols[0] if cols else None

def find_cols(df, patterns_or_names: List[str]) -> List[str]:
    """Return all columns that match any regex or exact name (case-insensitive)."""
    hits = []
    if df is None or getattr(df, "columns", None) is None:
        return hits
    cols = list(df.columns)
    for token in patterns_or_names:
        is_regex = bool(re.search(r"[\\^$.*+?()\\[\\]|]", token))
        if is_regex:
            try:
                rx = re.compile(token, flags=re.IGNORECASE)
            except Exception:
                continue
            for c in cols:
                if rx.search(str(c)):
                    hits.append(c)
        else:
            low = token.strip().lower()
            for c in cols:
                if str(c).strip().lower() == low:
                    hits.append(c)
    # de-dupe preserving order
    seen = set()
    uniq = []
    for c in hits:
        if c not in seen:
            uniq.append(c)
            seen.add(c)
    return uniq

def normalize_colnames(df):
    """Return a copy with normalized column names (no mutation)."""
    if df is None or getattr(df, "columns", None) is None:
        return df
    new = df.copy()
    new.columns = [re.sub(r"\s+", " ", str(c)).strip().lower() for c in df.columns]
    return new

def html_escape(s: str) -> str:
    import html as _h
    return _h.escape("" if s is None else str(s), quote=True)

def is_htmlish(s: str) -> bool:
    if not isinstance(s, str):
        return False
    return ("<a " in s) or ("<span" in s) or ("<div" in s) or ("<br" in s) or ("</" in s)

def comma_int(n) -> str:
    try:
        return f"{int(n):,}"
    except Exception:
        return str(n)

def safe_get(row, col, default=""):
    try:
        return row.get(col, default)
    except Exception:
        try:
            return row[col]
        except Exception:
            return default

def slugify(s: str) -> str:
    s = re.sub(r"[^A-Za-z0-9]+", "-", str(s)).strip("-")
    s = re.sub(r"-{2,}", "-", s)
    return s.lower()

# ---- Gold 2 specific helpers needed by transform ----
ID_PAT = re.compile(r"\bI\d+\b", re.I)
def extract_person_id(s: str) -> str:
    m = ID_PAT.search(str(s or ""))
    return m.group(0).upper() if m else ""

_CAMEL_WORDS = re.compile(r"[A-Z][a-z]*|[A-Z]+(?![a-z])|[a-z]+")
def smart_titlecase(s: str) -> str:
    s = str(s or "").strip()
    if not s: return ""
    s = re.sub(r"\s+", " ", s)
    parts = [p.capitalize() for p in s.split(" ") if p]
    return " ".join(parts)

def surname_given_from_token(token: str) -> tuple:
    token = str(token or "").strip()
    if not token: return "", ""
    words = _CAMEL_WORDS.findall(token)
    if not words: return token, ""
    return words[0].capitalize(), (" ".join(words[1:])).capitalize()

def _truncate_alpha(s: str, n: int) -> str:
    return re.sub(r"[^A-Za-z]", "", str(s))[:n]

def truncate_first(name: str, n: int = 7) -> str:
    name = str(name or "").strip()
    if not name: return ""
    parts = name.split()
    if not parts: return ""
    parts[0] = _truncate_alpha(parts[0], n).capitalize()
    return " ".join(parts)

def norm_matchee_name(raw: str) -> str:
    raw = str(raw or "").strip()
    if not raw:
        return ""
    if " " in raw or "," in raw:
        nm = smart_titlecase(raw)
        parts = nm.split()
        if len(parts) == 1:
            return nm
        given = parts[0]
        surname = parts[-1]
        return f"{_truncate_alpha(given, 7)} {surname}".strip()
    words = _CAMEL_WORDS.findall(raw)
    while words and len(words[0]) == 1:
        words.pop(0)
    if not words:
        nm = smart_titlecase(surname_given_from_token(raw)[0])
        ps = nm.split()
        if len(ps) == 1:
            return nm
        return f"{_truncate_alpha(ps[0], 7)} {ps[-1]}".strip()
    surname = smart_titlecase(words[0])
    given_candidates = [w for w in words[1:] if w.lower() != surname.lower()]
    if not given_candidates:
        return surname
    given = smart_titlecase(given_candidates[0])
    return f"{_truncate_alpha(given, 7)} {surname}".strip()

def split_tokens(lineage: str) -> List[str]:
    s = str(lineage or "").strip()
    if not s: return []
    # tokens separated by ' -> ' (plus a couple of safe fallbacks)
    parts = [p.strip() for p in re.split(r"\s*->\s*|\s*\u2192\s*|\s*&gt;\s*", s) if p.strip()]
    return parts

def derive_common_from_first_token(tokens: List[str]) -> tuple:
    if not tokens: return "", ""
    first = tokens[0]
    parts = re.split(r"\s*&\s*|\s+and\s+", first)
    a = parts[0].strip() if parts else ""
    b = parts[1].strip() if len(parts) > 1 else ""
    return a, b

def build_header(subject_bold_html: str, cm_val: str, matchee_html: str, gens_total: int, husband: str, wife: str) -> str:
    cm_txt = html_escape(cm_val) if cm_val else "0"
    gens_txt = comma_int(gens_total)
    pair = f"{html_escape(husband)} &amp; {html_escape(wife)}" if (husband or wife) else "&hellip;"
    return (
        f"{subject_bold_html} is a DNA match to {matchee_html}. "
        f"Shared DNA: <em>{cm_txt} cM</em>. "
        f"Oldest distant ancestor couple: <em>{pair}</em>. "
        f"Total generations in path: <em>{gens_txt}</em>."
    )
# ====== CUT STOP  [B] REFACTOR-Gold 2 — Add 5 returns for spacing ===============================
#
#
#
#
#
# ====== CUT START [3/6] REFACTOR-Gold 2 — CSV Load + Column Detection + Row Helpers ================
import os
import pandas as pd
import urllib.parse as _u

# Use the path defined in [A]; allow override via env
CSV_IN = os.environ.get("CSV_IN", CSV_PATH if 'CSV_PATH' in globals() else "final_combined_df_with_value_labels.csv")

# ---- Load CSV (robust encodings) ----
_encs = ("utf-8-sig", "utf-8", "cp1252", "iso-8859-15", "latin1")
_last_err = None
df = None
for _e in _encs:
    try:
        df = pd.read_csv(CSV_IN, encoding=_e, dtype=str, keep_default_na=False)
        break
    except Exception as _ex:
        _last_err = _ex
        df = None
if df is None:
    raise RuntimeError(f"Unable to read CSV: {CSV_IN} ({_last_err})")
print(f"✅ Loaded CSV — {len(df)} rows, {len(df.columns)} columns from {os.path.abspath(CSV_IN)}")

# Normalize header spacing/case (but keep original values intact for content)
df = normalize_colnames(df)

# ---- Locate columns (names vary slightly across exports) ----
id_col        = find_col(df, [r'^(id#|personid|id)$', r'person\s*id'], ["ID#", "ID", "PersonID", "personID", "id"])
match_to_col  = find_col(df, [r'^match\s*to$', r'^matchto$'], ["Match to","Match","match_to"])
name_col      = find_col(df, [r'^(name|full\s*name)$'], ["Name","Full Name"])
cm_col        = find_col(df, [r'^(c\s*:?m|cm)$', r'centi.?morgan'], ["cM","cm","Centimorgans"])
path_col      = find_col(df, [r'(yates\s*dna\s*ancestral\s*line|ancestral\s*line|lineage)'],
                         ["Yates DNA Ancestral Line","Ancestral Line","Lineage","lineage"])
subject_code_col = find_col(df, [r'^(subject\s*code|subjectcode|code)$'], ["Subject Code","subject_code","Code","code"])

if not id_col:       raise ValueError("CSV missing an ID#/PersonID column.")
if not match_to_col: raise ValueError("CSV missing 'Match to' column.")
if not name_col:     raise ValueError("CSV missing 'Name' column.")
if not cm_col:       raise ValueError("CSV missing 'cM' column.")
if not path_col:     raise ValueError("CSV missing lineage/path column.")
# subject_code_col is optional but recommended; continue even if None

print("✅ Columns:", {"ID": id_col, "Match to": match_to_col, "Name": name_col, "cM": cm_col, "Lineage": path_col, "SubjectCode": subject_code_col})
# ====== CUT STOP [3/6] REFACTOR-Gold 2 — CSV Load + Column Detection + Row Helpers =================





# ====== CUT START [4/6] REFACTOR-Gold 2 — Transform & Column A (build display_df) ==================
import urllib.parse as _u
import os

# ---------- Safety checks for upstream dependencies ----------
_required = [
    "df", "id_col", "match_to_col", "name_col", "cm_col", "path_col",
    "normalize_person_name", "resolve_match_to", "extract_person_id",
    "norm_matchee_name", "split_tokens", "derive_common_from_first_token",
    "build_header", "truncate_first",
    "TNG_BASE", "TNG_TREE", "ARROW_ENTITY"
]
_missing = [x for x in _required if x not in globals()]
if _missing:
    raise RuntimeError(f"[4/6] missing prerequisites from earlier sections: {_missing}")

# Defaults if not set in [A]
REMOTE_NAME_ABS = "/" + (REMOTE_NAME if 'REMOTE_NAME' in globals() else "ons_yates_dna_register.htm")
LINEAGE_HEADER_SAFE = globals().get("LINEAGE_HEADER", "Lineage (Starting with oldest ancestor, the line is:)")
MATCH_COUSINS_CSV_SAFE = globals().get("MATCH_COUSINS_CSV", "match_cousins_columnA.csv")
ENC_SAFE = globals().get("ENC", "iso-8859-15")

# ---------- Build the three display columns ----------
headers: list[str]  = []
lineages: list[str] = []
findcol: list[str]  = []

for _, row in df.iterrows():
    # Subject: resolver code in "Match to" → unmasked name (bolded in sentence)
    subject_raw  = row.get(match_to_col, "")
    subject_name = normalize_person_name(resolve_match_to(subject_raw))
    subject_name_b = f"<strong>{subject_name}</strong>" if subject_name else subject_name

    # Matchee: name from CSV (normalized) + TNG hotlink if personID present
    pid = extract_person_id(row.get(id_col, ""))
    matchee_name = norm_matchee_name(row.get(name_col, "")) or subject_name or ""
    if pid:
        matchee_html = (
            f'<a href="{TNG_BASE}/verticalchart.php?personID={pid}&tree={TNG_TREE}&parentset=0&display=vertical&generations=15" '
            f'target="_blank" rel="noopener">{matchee_name}</a>'
        )
    else:
        matchee_html = matchee_name

    # cM value
    cm_val = row.get(cm_col, "0")

    # Lineage path (tokens oldest→youngest)
    tokens      = split_tokens(row.get(path_col, ""))
    gens_total  = len(tokens)
    tokens_disp = tokens[:7]  # show first 7 pairs

    # Husband & Wife from first token unless explicit columns exist
    if "common_husband" in df.columns and "common_wife" in df.columns:
        husband_raw = str(row.get("common_husband", "")).strip()
        wife_raw    = str(row.get("common_wife", "")).strip()
        if not husband_raw and not wife_raw:
            husband_raw, wife_raw = derive_common_from_first_token(tokens)
    else:
        husband_raw, wife_raw = derive_common_from_first_token(tokens)

    # Build the sentence header (truncate first names to 7 chars for readability)
    header_html = build_header(
        subject_name_b,
        cm_val,
        matchee_html,
        gens_total,
        truncate_first(husband_raw, 7) if husband_raw else "",
        truncate_first(wife_raw, 7) if wife_raw else ""
    )

    # Bold the first pair in lineage view
    if tokens_disp:
        tokens_disp[0] = f"<strong>{tokens_disp[0]}</strong>"

    sep = f" {ARROW_ENTITY} "
    lineage_text = sep.join(tokens_disp) if tokens_disp else ""

    # Per-row quick-open search button (?q=Subject Name)
    q = _u.quote(subject_name)
    quick = (f'<a class="find-btn" href="{REMOTE_NAME_ABS}?q={q}" target="_blank" rel="noopener" '
             f'title="Open a filtered view for {subject_name}">Find</a>')

    headers.append(header_html)
    lineages.append(lineage_text)
    findcol.append(quick)

# ---------- Assemble the display dataframe ----------
df["Find"]               = findcol
df["Match Summary"]      = headers
df[LINEAGE_HEADER_SAFE]  = lineages

display_df = df[["Find", "Match Summary", LINEAGE_HEADER_SAFE]].copy()

# ---------- Column A CSV (side artifact used by other tooling) ----------
try:
    display_df[["Match Summary"]].to_csv(MATCH_COUSINS_CSV_SAFE, index=False, encoding=ENC_SAFE)
    print("✅ Wrote local CSV (Column A):", os.path.abspath(MATCH_COUSINS_CSV_SAFE))
except Exception as e:
    print(f"⚠️ Could not write Column A CSV ({MATCH_COUSINS_CSV_SAFE}): {e}")

print("✅ Built display_df with", len(display_df), "rows.")
# ====== CUT STOP [4/6] REFACTOR-Gold 2 — Transform & Column A (build display_df) ==================



# ====== CUT START [5/6] REFACTOR-Gold 2 — HTML (table + CSS + JS, no uploads) ======================
import html as _html

# ---------- Preconditions ----------
_needed = ["display_df", "LINEAGE_HEADER_SAFE", "TABLE_WIDTH_PX", "COL_A_PX",
           "COUNT_PUBLIC_URL", "ARROW_ENTITY"]
_missing = [x for x in _needed if x not in globals()]
if _missing:
    raise RuntimeError(f"[5/6] missing prerequisites from earlier sections: {_missing}")

# ---------- Constants / page prefs ----------
HOME_URL = "https://yates.one-name.net/ons_yates_dna_register.htm"
DOM_ID_PREFIX = "g2-"  # enforce Gold 2 namespace for DOM

# ---------- Build HTML table ----------
_display = display_df[["Find", "Match Summary", LINEAGE_HEADER_SAFE]].copy()
html_table = _display.to_html(index=False, escape=False, classes=f"sortable {DOM_ID_PREFIX}dataframe")

# Tag table and first row with Gold 2 ids
TABLE_ID = f"{DOM_ID_PREFIX}table"
html_table = html_table.replace(
    '<table border="1" class="dataframe sortable',
    f'<table border="1" class="dataframe sortable {DOM_ID_PREFIX}dataframe',
    1
)
html_table = html_table.replace(
    f'<table border="1" class="dataframe sortable {DOM_ID_PREFIX}dataframe">',
    f'<table border="1" class="dataframe sortable {DOM_ID_PREFIX}dataframe" id="{TABLE_ID}">', 1
)
html_table = html_table.replace('<tbody>\n<tr>', f'<tbody>\n<tr id="{DOM_ID_PREFIX}first-row">', 1)

# widths via <colgroup>
FIND_PX = 110
colgroup_html = (
    "<colgroup>\n"
    f"  <col style=\"width:{FIND_PX}px;\" />\n"
    f"  <col style=\"width:{COL_A_PX}px;\" />\n"
    "  <col />\n"
    "</colgroup>\n"
)
html_table = html_table.replace(
    f'<table border="1" class="dataframe sortable {DOM_ID_PREFIX}dataframe" id="{TABLE_ID}">',
    f'<table border="1" class="dataframe sortable {DOM_ID_PREFIX}dataframe" id="{TABLE_ID}">\n{colgroup_html}', 1
)

# add Select-all checkbox to header cell “Find”
MASTER_CB_ID = f"{DOM_ID_PREFIX}sel-all"
html_table = html_table.replace(
    "<th>Find</th>",
    f'<th>Find&nbsp;<input type="checkbox" id="{MASTER_CB_ID}" title="Select all visible" /></th>',
    1
)

html_table_scrolling = f'<div class="{DOM_ID_PREFIX}table-scroll">\n{html_table}\n</div>'

# ---------- CSS (Gold 2 namespace) ----------
TABLE_CSS = (
    "<style type=\"text/css\">\n"
    "  html { scroll-behavior: smooth; }\n"
    "  body { font-family: Georgia, 'Times New Roman', serif; background:#ffffff; color:#222; margin:0; padding:0; line-height:1.5; }\n"
    f"  .{DOM_ID_PREFIX}wrap {{ max-width:{TABLE_WIDTH_PX}px; margin:0 auto; background:#ffffff; padding:20px; padding-bottom:48px; }}\n"
    "  a { color:#154b8b; text-decoration:none; } a:hover { text-decoration:underline; }\n"
    "  h1 { margin:0 0 6px 0; font-size:26px; line-height:1.2; text-align:center; }\n"
    "  .updated { font-size:12px; color:#555; text-align:center; margin:2px 0 10px 0; }\n"
    f"  .{DOM_ID_PREFIX}sortbar {{ margin:6px 0 10px 0; font-size:13px; background:#ffffff; padding:6px 8px; border-radius:6px;\n"
    "             display:flex; flex-wrap:wrap; gap:5px; align-items:center; border:1px solid #ddd; }}\n"
    f"  .{DOM_ID_PREFIX}btn {{ display:inline-block; border:1px solid #5b79b8; background:#5b79b8; color:#fff; padding:4px 9px;\n"
    "         text-decoration:none; cursor:pointer; border-radius:5px; line-height:1.2; transition:background 0.2s, transform 0.1s; user-select:none; }}\n"
    f"  .{DOM_ID_PREFIX}btn:hover {{ background:#4668aa; transform:translateY(-1px); }}\n"
    f"  input.{DOM_ID_PREFIX}search {{ background:#fff; color:#111; border:1px solid #bbb; padding:4px 9px; border-radius:5px; }}\n"
    f"  .{DOM_ID_PREFIX}btn-mini {{ font-size:12px; padding:2px 6px; line-height:1.1; margin-left:6px; }}\n"
    f"  .{DOM_ID_PREFIX}find-cell {{ white-space:nowrap; }}\n"
    f"  .{DOM_ID_PREFIX}selbox {{ margin-right:6px; vertical-align:middle; }}\n"
    f"  .{DOM_ID_PREFIX}table-scroll {{ max-height:70vh; overflow-y:auto; overflow-x:auto; border:1px solid #ddd; }}\n"
    f"  table.sortable {{ border-collapse:collapse; width:{TABLE_WIDTH_PX}px; table-layout:fixed; }}\n"
    "  table.sortable th, table.sortable td { border:1px solid #ddd; padding:6px 8px; vertical-align:top; }\n"
    "  table.sortable th { background:#e3eaf8; text-align:left; position:sticky; top:0; z-index:2; box-shadow:0 1px 0 #ccc; }\n"
    "  table.sortable td { word-wrap:break-word; overflow-wrap:break-word; }\n"
    f"  #{DOM_ID_PREFIX}first-row td {{ border-top:2px solid #999; }}\n"
    f"  #{DOM_ID_PREFIX}back-to-top {{ position:fixed; right:16px; bottom:16px; padding:6px 10px; border:1px solid #3e5a97; background:#5b79b8; color:#fff;\n"
    "                 cursor:pointer; border-radius:6px; font-size:12px; display:none; z-index:9999; }\n"
    f"  #{DOM_ID_PREFIX}back-to-top:hover {{ background:#4668aa; }}\n"
    f"  #{DOM_ID_PREFIX}dynamic {{ margin:10px 0 14px 0; }}\n"
    "  @media screen and (max-width: 820px) { "
    f".{DOM_ID_PREFIX}wrap {{ padding:12px; }} h1 {{ font-size:22px; }} }}\n"
    "</style>\n"
)

# ---------- Toolbar (Gold 2 namespace + root-level partials) ----------
BTN_EMAIL_SEL_ID = f"{DOM_ID_PREFIX}email-selected"
BTN_CLEAR_SEL_ID = f"{DOM_ID_PREFIX}clear-selected"
SEARCH_ID        = f"{DOM_ID_PREFIX}search-box"

DYNAMIC_BLOCK = (
    f"<div class=\"{DOM_ID_PREFIX}sortbar\">\n"
    "  <a class=\"{btn}\" href=\"https://yates.one-name.net/gengen/dna_cousin_surname_study.htm\" target=\"_blank\">Study Details</a>\n"
    "  <a class=\"{btn}\" href=\"https://yates.one-name.net/gengen/dna_theory_of_the_case.htm\" target=\"_blank\">Theory in Action</a>\n"
    "  <span class=\"{btn}\" data-sort-col=\"1\" data-sort-dir=\"asc\">Sort Match &uarr;</span>\n"
    "  <span class=\"{btn}\" data-sort-col=\"1\" data-sort-dir=\"desc\">Sort Match &darr;</span>\n"
    "  <span class=\"{btn}\" data-sort-col=\"2\" data-sort-dir=\"asc\">Sort Lineage &uarr;</span>\n"
    "  <span class=\"{btn}\" data-sort-col=\"2\" data-sort-dir=\"desc\">Sort Lineage &darr;</span>\n"
    "  <a class=\"{btn}\" href=\"gengen/images/cousin-calculator.jpg\" target=\"_blank\">Cousin Connection</a>\n"
    "  <a class=\"{btn}\" href=\"gengen/images/Shared_cM_Project_v4.jpg\" target=\"_blank\">Cousin by DNA</a>\n"
    # load-able partials into dynamic container (root-level preferred)
    f"  <a class=\"{{btn}}\" data-load-partial=\"match_count.htm\"   href=\"#\">Match Count</a>\n"
    f"  <a class=\"{{btn}}\" data-load-partial=\"lineage_count.htm\" href=\"#\">Lineage Count</a>\n"
    f"  <a class=\"{{btn}}\" href=\"/partials/cousin_list_print.htm\" target=\"_blank\">Cousin List</a>\n"
    f"  <span class=\"{{btn}}\" id=\"{BTN_EMAIL_SEL_ID}\">Email Selected</span>\n"
    f"  <span class=\"{{btn}}\" id=\"{BTN_CLEAR_SEL_ID}\">Clear</span>\n"
    f"  <input type=\"text\" id=\"{SEARCH_ID}\" class=\"{{search}}\" size=\"24\" value=\"\" placeholder=\"Search&hellip;\" "
    "         autocomplete=\"off\" autocapitalize=\"off\" spellcheck=\"false\" inputmode=\"search\" enterkeyhint=\"search\" />\n"
    "</div>\n"
    f"<div id=\"{DOM_ID_PREFIX}dynamic\"></div>\n"
).format(btn=f"{DOM_ID_PREFIX}btn", search=f"{DOM_ID_PREFIX}search")

# ---------- Updated status line (top, dynamic) with Home link ----------
JS_COUNT_URL = COUNT_PUBLIC_URL.replace("'", "%27")
UPDATED_BLOCK = (
    "<div class=\"updated\">"
    f"<a href=\"{HOME_URL}\" target=\"_blank\" rel=\"noopener\">Home</a>"
    " &nbsp;|&nbsp; Last updated: <span id=\"last-updated\"></span>"
    " &nbsp;|&nbsp; Autosomal matches: <span id=\"auto-count\" class=\"js-count\"></span>"
    "</div>"
)

# ---------- Full HTML (string only) ----------
BACK_TO_TOP_ID = f"{DOM_ID_PREFIX}back-to-top"

TEMPLATE_HTML = """<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>ONS Yates Study Autosomal DNA Register</title>
{TABLE_CSS}
</head>
<body id="top">
<div class="{DOM_PREFIX}wrap">
  <h1>ONS Yates Study Autosomal DNA Register</h1>
  {UPDATED_BLOCK}
  {DYNAMIC_BLOCK}
{HTML_TABLE_SCROLLING}
</div>
<button id="{BACK_TO_TOP_ID}" class="{DOM_PREFIX}back-to-top">&#9650; Top</button>
<script type="text/javascript">
//<![CDATA[
(function(){
  var DOM_PREFIX = "{DOM_PREFIX}";
  function textOf(cell){ return (cell && (cell.textContent || cell.innerText) || '').replace(/\\s+/g,' ').trim().toLowerCase(); }
  function sortTable(tbl, colIndex, dir){
    var tb=tbl && tbl.tBodies ? tbl.tBodies[0] : null; if(!tb) return;
    var rows=[].slice.call(tb.rows||[]);
    var asc=(dir==='asc');
    rows.sort(function(a,b){
      var A=textOf(a.cells[colIndex]), B=textOf(b.cells[colIndex]);
      if(A<B) return asc?-1:1; if(A>B) return asc?1:-1; return 0;
    });
    var frag=document.createDocumentFragment();
    for(var i=0;i<rows.length;i++) frag.appendChild(rows[i]);
    tb.appendChild(frag);
    updateSelAll(); // keep master checkbox in sync after sort
  }
  function bindSortButtons(){
    var tbl=document.getElementById('{TABLE_ID}'); var bar=document.querySelector('.{DOM_PREFIX}sortbar'); if(!(tbl && bar)) return;
    bar.addEventListener('click',function(e){
      var t=e.target; if(!t) return;
      var btn=t.closest? t.closest('.{DOM_PREFIX}btn') : null; if(!btn) return;
      var colAttr=btn.getAttribute('data-sort-col'); if(colAttr==null) return; var col=parseInt(colAttr,10); if(isNaN(col)) return;
      var dir=btn.getAttribute('data-sort-dir')||'asc'; sortTable(tbl,col,dir); e.preventDefault(); return false;
    },false);
  }
  function bindHeaderSort(){
    var tbl=document.getElementById('{TABLE_ID}'); if(!(tbl && tbl.tHead && tbl.tHead.rows.length)) return; var ths=tbl.tHead.rows[0].cells;
    for(var i=0;i<ths.length;i++)(function(idx){
      var dir='asc'; ths[idx].addEventListener('click',function(){ dir=(dir==='asc')?'desc':'asc'; sortTable(tbl,idx,dir); },false);
    })(i);
  }

  // partials via multiple base locations (root/relative)
  var PARTIAL_BASES=['/partials/','partials/','gengen/partials/','/gengen/partials/'];
  function tryFetchSequential(urls,onOK,onFail){
    if(!urls.length) return onFail('No valid locations'); var url=urls.shift();
    fetch(url,{cache:'no-store'}).then(function(r){ if(!r.ok) throw new Error('HTTP '+r.status); return r.text();})
      .then(onOK).catch(function(){ tryFetchSequential(urls,onOK,onFail);});
  }
  function bindPartials(){
    var bar=document.querySelector('.{DOM_PREFIX}sortbar'); if(!bar) return;
    bar.addEventListener('click',function(e){
      var t=e.target; if(!t) return;
      var btn=t.closest? t.closest('.{DOM_PREFIX}btn') : null; if(!btn) return;
      var rel=btn.getAttribute('data-load-partial'); if(!rel) return;
      var c=document.getElementById('{DOM_PREFIX}dynamic'); if(!c) return; c.innerHTML='<p><em>Loading latest data&hellip;</em></p>';
      var bust=encodeURIComponent(document.lastModified||(new Date()).toUTCString());
      var candidates=PARTIAL_BASES.map(function(b){return b+rel+'?v='+bust;});
      tryFetchSequential(candidates.slice(), function(html){ c.innerHTML=html; }, function(){ c.innerHTML='<p style=\\\"color:#a00;\\\">Could not load content.</p>'; });
    });
  }

  // updated stamp + count
  function stampLastUpdated(){
    var el=document.getElementById('last-updated'); if(!el) return;
    var d=new Date(document.lastModified||new Date());
    function z(n){return(n<10?'0':'')+n;}
    el.innerHTML=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());
  }
  function formatWithCommas(n){
    try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US'); }catch(e){ return String(n||''); }
  }
  function loadAutoCount(){
    var el=document.getElementById('auto-count'); if(!el) return;
    var url='{COUNT_URL}';
    try{
      var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);
      xhr.onreadystatechange=function(){ if(xhr.readyState===4){ if(xhr.status>=200&&xhr.status<300){
        var m=(xhr.responseText||'').match(/(\\d+)/); var num=m?m[1]:'';
        el.textContent = formatWithCommas(num) || '(unavailable)';
      } else { el.textContent='(unavailable)'; } } };
      xhr.send(null);
    }catch(e){ el.textContent='(unavailable)'; }
  }

  // ---- search (filters rows; select-all respects visibility) ----
  function getParam(name){ var m=location.search.match(new RegExp('[?&]'+name+'=([^&]+)')); return m?decodeURIComponent(m[1].replace(/\\+/g,' ')):''; }
  function bindSearch(){
    var box=document.getElementById('{SEARCH_ID}'); var tbl=document.getElementById('{TABLE_ID}'); if(!(box && tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb=tbl.tBodies[0];
    function norm(s){ return String(s||'').replace(/\\s+/g,' ').toLowerCase(); }
    function rowText(tr){ var t=''; for(var i=0;i<tr.cells.length;i++){ t+= ' ' + (tr.cells[i].textContent||tr.cells[i].innerText||''); } return norm(t); }
    var cached=[]; (function seed(){ var rows=tb.rows; cached=[]; for(var i=0;i<rows.length;i++){ cached.push({el:rows[i], txt:rowText(rows[i])}); } })();
    function apply(q){
      q=norm(q);
      for(var i=0;i<cached.length;i++){
        var hit = !q || cached[i].txt.indexOf(q)>-1;
        cached[i].el.style.display = hit? '' : 'none';
      }
      updateSelAll();
    }
    var to=null; function onInput(){ if(to) clearTimeout(to); to=setTimeout(function(){ apply(box.value); }, 60); }
    box.addEventListener('input', onInput, false);
    var q0=getParam('q');
    if(q0){ box.value=q0; apply(q0); try{ history.replaceState(null, '', location.pathname); }catch(e){} }
    else { box.value=''; apply(''); setTimeout(function(){ if(!getParam('q')){ box.value=''; apply(''); } }, 0); }
  }

  // ---- selection controls (Gold 2) ----
  function visibleRowCheckboxes(){
    var tbl=document.getElementById('{TABLE_ID}');
    if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return [];
    var tb=tbl.tBodies[0], out=[];
    for(var i=0;i<tb.rows.length;i++){
      var tr=tb.rows[i];
      if(tr.style.display==='none') continue;
      var cb = tr.cells[0] && tr.cells[0].querySelector('.{DOM_PREFIX}selbox');
      if(cb) out.push(cb);
    }
    return out;
  }
  function updateSelAll(){
    var master=document.getElementById('{MASTER_CB_ID}'); if(!master) return;
    var cbs=visibleRowCheckboxes();
    if(!cbs.length){ master.indeterminate=false; master.checked=false; return; }
    var checked=cbs.filter(function(cb){ return cb.checked; }).length;
    master.checked = (checked===cbs.length);
    master.indeterminate = (checked>0 && checked<cbs.length);
  }
  function bindSelectAll(){
    var master=document.getElementById('{MASTER_CB_ID}'); if(!master) return;
    master.addEventListener('change', function(){
      var want=master.checked;
      var cbs=visibleRowCheckboxes();
      for(var i=0;i<cbs.length;i++) cbs[i].checked = want;
      updateSelAll();
    });
  }
  function pageBase(){ try{ return location.origin + location.pathname; }catch(e){ return location.pathname; } }
  function encodeQS(s){ return encodeURIComponent(String(s||'')); }
  function buildRowEmail(tr){
    var cells = tr.cells; if(!cells || cells.length < 3) return null;
    var summary = (cells[1].textContent||'').trim();
    var lineage = (cells[2].textContent||'').trim();
    var subj = 'ONS Yates Register: ' + summary.substring(0, 90);
    var qMatch = (function(){ try{ var em = cells[1].querySelector('strong'); return (em?em.textContent:'').trim(); }catch(e){ return ''; } })();
    var link = pageBase() + (qMatch? ('?q='+encodeQS(qMatch)) : '');
    var body = summary + '\\n\\nLineage:\\n' + lineage + '\\n\\nLink: ' + link;
    return {subject: subj, body: body};
  }
  function openMailto(subject, body){
    var href = 'mailto:?subject=' + encodeQS(subject) + '&body=' + encodeQS(body);
    window.location.href = href;
  }
  function injectRowControls(){
    var tbl = document.getElementById('{TABLE_ID}'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb = tbl.tBodies[0];
    for(var i=0;i<tb.rows.length;i++){
      (function(tr){
        var cell = tr.cells[0]; if(!cell) return; if(cell.className.indexOf('{DOM_PREFIX}find-cell')===-1) cell.className += ' {DOM_PREFIX}find-cell';
        var existingFind = cell.querySelector('a.find-btn'); // keep your “Find” link
        var cb = document.createElement('input'); cb.type='checkbox'; cb.className='{DOM_PREFIX}selbox'; cb.title='Select row';
        cb.addEventListener('change', updateSelAll);
        cell.insertBefore(cb, existingFind || cell.firstChild);
        // optional per-row Email button
        var emailBtn = document.createElement('a'); emailBtn.href='#'; emailBtn.className='{DOM_PREFIX}btn {DOM_PREFIX}btn-mini'; emailBtn.textContent='Email'; emailBtn.title='Email this row'; emailBtn.style.marginLeft='6px';
        emailBtn.addEventListener('click', function(ev){ ev.preventDefault(); var payload = buildRowEmail(tr); if(payload){ openMailto(payload.subject, payload.body); } });
        if(existingFind){ existingFind.insertAdjacentElement('afterend', emailBtn); } else { cell.appendChild(emailBtn); }
      })(tb.rows[i]);
    }
  }
  function bindBulkEmail(){
    var btn = document.getElementById('{BTN_EMAIL_SEL_ID}'); if(!btn) return;
    btn.addEventListener('click', function(){
      var tbl = document.getElementById('{TABLE_ID}'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb = tbl.tBodies[0];
      var parts = []; var count = 0;
      for(var i=0;i<tb.rows.length;i++){
        var tr = tb.rows[i]; if(tr.style.display==='none') continue;
        var cb = tr.cells[0] && tr.cells[0].querySelector('.{DOM_PREFIX}selbox');
        if(cb && cb.checked){ var p = buildRowEmail(tr); if(p){ parts.push(p.body); count++; } }
      }
      if(!count){ alert('No rows selected. Tick the checkboxes in the Find column.'); return; }
      var subject = 'ONS Yates Register: ' + count + ' selection' + (count>1?'s':'');
      var body = parts.join('\\n\\n---\\n\\n');
      openMailto(subject, body);
    });
    var clr = document.getElementById('{BTN_CLEAR_SEL_ID}'); if(clr){ clr.addEventListener('click', function(){
      var tbl = document.getElementById('{TABLE_ID}'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb = tbl.tBodies[0];
      for(var i=0;i<tb.rows.length;i++){ var cb = tb.rows[i].cells[0] && tb.rows[i].querySelector('.{DOM_PREFIX}selbox'); if(cb) cb.checked=false; }
      updateSelAll();
    }); }
  }

  // misc UI
  function setupBackToTop(){
    var btt=document.getElementById('{BACK_TO_TOP_ID}'); var cont=document.querySelector('.{DOM_PREFIX}table-scroll');
    function onAny(){ var y=(window.scrollY||window.pageYOffset||0); var cy=cont?cont.scrollTop:0; btt.style.display=(y>200||cy>200)?'block':'none'; }
    window.addEventListener('scroll',onAny,{passive:true}); if(cont) cont.addEventListener('scroll',onAny,{passive:true});
    onAny(); if(btt){ btt.addEventListener('click',function(){ if(cont) cont.scrollTo({top:0,behavior:'smooth'}); window.scrollTo({top:0,behavior:'smooth'}); }); }
  }

  // init
  function init(){
    bindSortButtons(); bindHeaderSort(); bindPartials(); stampLastUpdated(); loadAutoCount();
    setupBackToTop(); bindSearch();
    injectRowControls(); bindSelectAll(); bindBulkEmail(); updateSelAll();
  }
  init();
})();
//]]>
</script>
</body></html>
"""

FULL_HTML = (TEMPLATE_HTML
    .replace("{TABLE_CSS}", TABLE_CSS)
    .replace("{UPDATED_BLOCK}", UPDATED_BLOCK)
    .replace("{DYNAMIC_BLOCK}", DYNAMIC_BLOCK)
    .replace("{HTML_TABLE_SCROLLING}", html_table_scrolling)
    .replace("{COUNT_URL}", JS_COUNT_URL)
    .replace("{DOM_PREFIX}", DOM_ID_PREFIX)
    .replace("{TABLE_ID}", TABLE_ID)
    .replace("{MASTER_CB_ID}", MASTER_CB_ID)
    .replace("{SEARCH_ID}", SEARCH_ID)
    .replace("{BTN_EMAIL_SEL_ID}", BTN_EMAIL_SEL_ID)
    .replace("{BTN_CLEAR_SEL_ID}", BTN_CLEAR_SEL_ID)
    .replace("{BACK_TO_TOP_ID}", BACK_TO_TOP_ID)
)
# ====== CUT STOP [5/6] REFACTOR-Gold 2 — HTML (table + CSS + JS, no uploads) =======================


# ====== CUT START [6/6] REFACTOR-Gold 2 — Save + Upload + Reports + Partials + Printable ===========






import os, re, posixpath
from collections import Counter
import pandas as pd

ENC          = "iso-8859-15"
LOCAL_NAME   = globals().get("LOCAL_NAME",  "ons_yates_dna_register.htm")
REMOTE_NAME  = globals().get("REMOTE_NAME", "ons_yates_dna_register.htm")
HOME_URL     = "https://yates.one-name.net/ons_yates_dna_register.htm"

# Safely embed the JS count URL (escape single quotes)
JS_COUNT_URL = (globals().get("COUNT_PUBLIC_URL","") or "").replace("'", "%27")

# ---------- Bind subject_code_col (the resolver code column, i.e., “Match to”) ----------
# Prefer an existing binding; otherwise fall back to match_to_col (from [3/6]); last resort a literal name.
subject_code_col = globals().get("subject_code_col") or globals().get("match_to_col") or "Match to"
# If the fallback literal isn’t present in df, and match_to_col exists, use that explicitly.
try:
    if 'df' in globals():
        if subject_code_col not in list(df.columns) and 'match_to_col' in globals() and globals()['match_to_col'] in list(df.columns):
            subject_code_col = globals()['match_to_col']
except Exception:
    pass

# 1) Write main page HTML exactly as built upstream
with open(LOCAL_NAME, "w", encoding=ENC, errors="xmlcharrefreplace") as f:
    f.write(FULL_HTML)
print(f"✅ Wrote HTML: {os.path.abspath(LOCAL_NAME)}")

# 2) Upload main page + optional assets
with ftp_connect() as ftps:
    try:
        ftps.mkd("partials")
    except Exception:
        pass
    try:
        ftps.delete(_remote_path(REMOTE_NAME))
    except Exception:
        pass

    ftp_upload_overwrite(ftps, LOCAL_NAME, _remote_path(REMOTE_NAME))

    if os.path.exists(globals().get("MATCH_COUSINS_CSV","the_match_cousins.csv")):
        ftp_upload_overwrite(ftps, globals().get("MATCH_COUSINS_CSV","the_match_cousins.csv"),
                             _remote_path(globals().get("MATCH_COUSINS_CSV","the_match_cousins.csv")))

    if os.path.exists(globals().get("LOCAL_COUNT_FILE","autosomal_count.txt")):
        ftp_upload_overwrite(ftps,
                             globals().get("LOCAL_COUNT_FILE","autosomal_count.txt"),
                             _remote_path(globals().get("REMOTE_COUNT_NAME","autosomal_count.txt")))
        if globals().get("COUNT_PUBLIC_URL"):
            print(f"✅ Published autosomal count: {globals().get('LOCAL_COUNT_FILE')} → {globals().get('COUNT_PUBLIC_URL')}")
    else:
        print(f"⚠️ Count file not found locally: {globals().get('LOCAL_COUNT_FILE','autosomal_count.txt')}")

    try:
        ftps.quit()
    except Exception:
        pass

# 3) Build filtered data for reports
SUPPRESS_PREFIXES = ["y-dna", "dar", "sar"]
def _suppress_by_match_summary(ms: str) -> bool:
    s = (str(ms) or "").strip().lower()
    return bool(s) and any(s.startswith(prefix + " is") for prefix in SUPPRESS_PREFIXES)

display_df_all = display_df.copy()
_suppress_mask = display_df_all["Match Summary"].map(_suppress_by_match_summary)
display_df_filtered = display_df_all[~_suppress_mask].reset_index(drop=True)
filtered_df_for_reports = df.loc[display_df_all.index[~_suppress_mask]].copy()

# ---------- Resolver usage / counts ----------
used_series = filtered_df_for_reports[subject_code_col].astype(str).map(lambda x: str(x).strip().lower())
counts = Counter([c for c in used_series if c and c != "nan"])

rows = []
all_keys = set(globals().get("MATCH_TO_UNMASKED", {}).keys())
for code in sorted(all_keys):
    rows.append((code, globals()["MATCH_TO_UNMASKED"].get(code, ""), counts.get(code, 0)))
for code in sorted(set(counts.keys()) - all_keys):
    rows.append((code, "(unmapped)", counts.get(code, 0)))

usage_df = pd.DataFrame(rows, columns=["Match to (code)", "Unmasked", "Count"]).sort_values(
    ["Match to (code)"], ascending=[True]
)
usage_df_alpha = usage_df.reset_index(drop=True)

RESOLVER_USAGE_CSV = "resolver_usage_report.csv"
usage_df_alpha.to_csv(RESOLVER_USAGE_CSV, index=False, encoding=ENC)
print("✅ Wrote resolver usage CSV:", os.path.abspath(RESOLVER_USAGE_CSV))

# ---------- Lineage Count ----------
def _escape_html(s: str) -> str:
    return (str(s).replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;"))

def _norm_key_component(fullname: str) -> str:
    nm = smart_titlecase(str(fullname)); parts = nm.split()
    if not parts: return ""
    last   = re.sub(r"[^A-Za-z]", "", parts[-1])
    firsts = re.sub(r"[^A-Za-z]", "", "".join(parts[:-1]))[:7]
    return f"{last}{firsts}"

def oldest_pair_key(token_list):
    if not token_list: return ""
    first = token_list[0]
    a, b = derive_common_from_first_token([first])
    if not a and not b:
        parts = re.split(r"\s*&\s*", first)
        a = parts[0] if parts else ""
        b = parts[1] if len(parts) > 1 else ""
    ka = _norm_key_component(a) if a else ""
    kb = _norm_key_component(b) if b else ""
    return f"{ka}&{kb}" if ka or kb else ""

lc_counter = Counter()
for _, row in filtered_df_for_reports.iterrows():
    toks = split_tokens(row[path_col])
    key = oldest_pair_key(toks)
    if key: lc_counter[key] += 1

lc_rows = sorted(lc_counter.items(), key=lambda x: (-x[1], x[0]))
lineage_count_df = pd.DataFrame(lc_rows, columns=["Oldest Distant Ancestor (normalized Last+First7) pair", "Count"])

LINEAGE_COUNT_CSV = "lineage_count_report.csv"
lineage_count_df.to_csv(LINEAGE_COUNT_CSV, index=False, encoding=ENC)
print("✅ Wrote lineage count CSV:", os.path.abspath(LINEAGE_COUNT_CSV))

# Upload report CSVs
try:
    with ftp_connect() as ftps:
        ftp_upload_overwrite(ftps, RESOLVER_USAGE_CSV, _remote_path(RESOLVER_USAGE_CSV))
        ftp_upload_overwrite(ftps, LINEAGE_COUNT_CSV, _remote_path(LINEAGE_COUNT_CSV))
        try: ftps.quit()
        except Exception: pass
    print("✅ Published report CSVs.")
except Exception as e:
    print(f"⚠️ Upload of report CSVs failed: {e}")

# ---------- Helpers for Match Count partial ----------
def _build_person_details_html(code_lc: str) -> str:
    mask = filtered_df_for_reports[subject_code_col].astype(str).str.strip().str.lower() == code_lc
    if not mask.any(): return "<em>No rows for this person.</em>"
    rows_idx = filtered_df_for_reports[mask].index
    mini = display_df.loc[rows_idx, ["Find", "Match Summary", LINEAGE_HEADER_SAFE]].copy()
    return mini.to_html(index=False, escape=False, classes="mini-table", border=0)

# ---------- Match Count partial (tight wrapper + working toolbar) ----------
def _render_match_count_full_html(usage_alpha: pd.DataFrame) -> str:
    try:
        total_persons = int(usage_alpha.shape[0])
        total_matches = int(pd.to_numeric(usage_alpha["Count"], errors="coerce").fillna(0).sum())
    except Exception:
        total_persons, total_matches = usage_alpha.shape[0], 0

    rows_html = []
    for _, r in usage_alpha.iterrows():
        code    = str(r["Match to (code)"])
        code_lc = code.strip().lower()
        unm     = _escape_html(r["Unmasked"])
        cnt     = int(pd.to_numeric(r["Count"], errors="coerce")) if str(r["Count"]).strip() else 0
        rows_html.append(
            "<tr class='top-row' data-code='{code_lc}'>"
            "<td class='sel'><input type='checkbox' class='selbox' data-code='{code_lc}' title='Select' /></td>"
            "<td class='code'>{code}</td>"
            "<td class='unmasked'>{unm}</td>"
            "<td class='count'><a href='#' class='count-toggle' data-code='{code_lc}' aria-expanded='false'>{cnt}</a></td>"
            "</tr>".format(code_lc=_escape_html(code_lc), code=_escape_html(code), unm=unm, cnt=f"{cnt:,}")
        )
        rows_html.append(
            "<tr class='details-row' id='details-{code_lc}' style='display:none;'>"
            "<td colspan='4'><div class='details-outer'>{detail}</div></td>"
            "</tr>".format(code_lc=_escape_html(code_lc), detail=_build_person_details_html(code_lc))
        )

    style = (
        "<style>"
        " body{font-family:Georgia,'Times New Roman',serif;background:#fff;color:#111;margin:0;padding:18px;line-height:1.6;font-size:15px}"
        " .wrap{max-width:900px;margin:0 auto}"
        " h2{font-size:20px;margin:0 0 10px 0;text-align:center;border-bottom:2px solid #5b79b8;padding-bottom:6px}"
        " .updated{font-size:12px;color:#555;text-align:center;margin:2px 0 10px 0}"
        " .stats{text-align:center;color:#444;font-size:13px;margin:6px 0 10px 0}"
        " .toolbar{display:flex;flex-wrap:wrap;gap:8px;align-items:center;justify-content:flex-start;margin:8px 0 10px 0}"
        " .btn{display:inline-block;border:1px solid #5b79b8;background:#5b79b8;color:#fff;padding:6px 12px;border-radius:6px;text-decoration:none;cursor:pointer}"
        " .btn:hover{background:#4668aa}"
        " .master{margin-left:auto;font-size:13px;color:#333}"
        " table#mc-table{border-collapse:collapse;width:100%;table-layout:fixed}"
        " #mc-table col.sel{width:72px} #mc-table col.code{width:26%} #mc-table col.unmasked{width:48%} #mc-table col.count{width:16%}"
        " #mc-table th,#mc-table td{border:1px solid #ccc;padding:8px 10px;vertical-align:top;word-wrap:break-word;overflow-wrap:break-word}"
        " #mc-table th{background:#e9eef9;position:sticky;top:0;z-index:1}"
        " #mc-table td.count{text-align:right}"
        " #mc-table a.count-toggle{display:inline-block;padding:2px 6px;background:#5b79b8;color:#fff;border-radius:4px;border:1px solid #3e5a97;text-decoration:none}"
        " #mc-table a.count-toggle:hover{background:#4668aa}"
        " .details-outer{background:#f9fbff;border:1px solid #d7e1fb;margin:8px 2px;padding:8px;border-radius:6px}"
        " .mini-table{border-collapse:collapse;width:100%;table-layout:fixed}"
        " .mini-table th,.mini-table td{border:1px solid #ddd;padding:6px 8px;vertical-align:top}"
        " .mini-table th{background:#f2f6ff}"
        " .mini-table col:nth-child(1){width:140px} "
        " .mini-table col:nth-child(2){width:" + str(globals().get('COL_A_PX',520)) + "px} "
        " .mini-table col:nth-child(3){width:auto}"
        " @media screen and (max-width:900px){ body{padding:12px;font-size:14px} .toolbar{gap:6px} }"
        "</style>"
    )

    updated = (
        "<div class='updated'>"
        "<a href='" + HOME_URL + "' target='_blank' rel='noopener'>Home</a>"
        " &nbsp;|&nbsp; Last updated: <span id='last-updated'></span>"
        " &nbsp;|&nbsp; Autosomal matches: <span id='auto-count' class='js-count'></span>"
        "</div>"
    )

    toolbar = (
        "<div class='toolbar'>"
        " <button type='button' class='btn' id='btn-display'>Display Checked</button>"
        " <button type='button' class='btn' id='btn-email'>Email Checked</button>"
        " <button type='button' class='btn' id='btn-print'>Print Checked</button>"
        " <label class='master'><input type='checkbox' id='sel-all' /> Select all</label>"
        "</div>"
    )

    html = (
        "<!DOCTYPE html><html lang='en'><head>"
        "<meta charset='iso-8859-15' />"
        "<meta name='viewport' content='width=device-width, initial-scale=1.0' />"
        "<title>Match Count (by resolver code)</title>"
        + style +
        "</head><body>"
        "<div class='wrap'>"
        "<h2>Match Count (by resolver code, A&rightarrow;Z)</h2>"
        + updated +
        "<div class='stats'>Total distinct &ldquo;Match to&rdquo; persons: " + f"{total_persons:,}" +
        " &nbsp;|&nbsp; Total matches counted: " + f"{total_matches:,}" + "</div>"
        + toolbar +
        "<table id='mc-table'>"
        "<colgroup><col class='sel'><col class='code'><col class='unmasked'><col class='count'></colgroup>"
        "<thead><tr><th>Select</th><th>Match to (code)</th><th>Unmasked</th><th>Count</th></tr></thead>"
        "<tbody>" + "".join(rows_html) + "</tbody></table>"
        "</div>"
        "<script>(function(){"
        " document.addEventListener('DOMContentLoaded', function(){"
        "  function z(n){return(n<10?'0':'')+n;}"
        "  function stamp(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date());"
        "   el.textContent=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());}"
        "  function fmt(n){try{var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US');}catch(e){return String(n||'');}}"
        "  function loadCnt(){var el=document.getElementById('auto-count'); if(!el) return; var url='" + JS_COUNT_URL + "';"
        "   try{var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);"
        "    xhr.onreadystatechange=function(){ if(xhr.readyState===4){ if(xhr.status>=200&&xhr.status<300){ var m=(xhr.responseText||'').match(/(\\d+)/); el.textContent = fmt(m?m[1]:'') || '(unavailable)'; } else { el.textContent='(unavailable)'; } } };"
        "    xhr.send(null);}catch(e){ el.textContent='(unavailable)'; }}"
        "  stamp(); loadCnt();"

        "  function toggleDetails(code, wantOpen){"
        "    var row=document.getElementById('details-'+code); if(!row) return;"
        "    row.style.display = wantOpen ? '' : 'none';"
        "    var a=document.querySelector(\"a.count-toggle[data-code='\"+code+\"']\"); if(a){ a.setAttribute('aria-expanded', wantOpen?'true':'false'); }"
        "  }"
        "  function selectedCodes(){"
        "    var out=[]; var cbs=document.querySelectorAll('#mc-table .selbox');"
        "    for(var i=0;i<cbs.length;i++){ if(cbs[i].checked){ var c=cbs[i].getAttribute('data-code'); if(c) out.push(c); }}"
        "    return out;"
        "  }"

        "  var master=document.getElementById('sel-all');"
        "  if(master){ master.addEventListener('change', function(){"
        "    var want=master.checked; var cbs=document.querySelectorAll('#mc-table .selbox');"
        "    for(var i=0;i<cbs.length;i++){ cbs[i].checked = want; }"
        "  }); }"

        "  var btnDisp=document.getElementById('btn-display');"
        "  if(btnDisp){ btnDisp.addEventListener('click', function(){"
        "    var sel=selectedCodes();"
        "    var all=document.querySelectorAll('tr.details-row'); for(var i=0;i<all.length;i++){ all[i].style.display='none'; }"
        "    for(var j=0;j<sel.length;j++){ toggleDetails(sel[j], true); }"
        "    if(sel.length){ try{ document.getElementById('details-'+sel[0]).scrollIntoView({behavior:'smooth',block:'start'});}catch(e){} }"
        "  }); }"

        "  function enc(s){ return encodeURIComponent(String(s||'')); }"
        "  function textOf(el){ return (el && (el.textContent||el.innerText)||'').trim(); }"

        "  var btnEmail=document.getElementById('btn-email');"
        "  if(btnEmail){ btnEmail.addEventListener('click', function(){"
        "    var sel=selectedCodes(); if(!sel.length){ alert('No persons selected.'); return; }"
        "    function collectMatches(code){"
        "      var out=[]; var host=document.getElementById('details-'+code); if(!host) return out;"
        "      var rows=host.querySelectorAll('table.mini-table tbody tr');"
        "      for(var i=0;i<rows.length;i++){"
        "        var tds=rows[i].cells; if(!tds || tds.length<3) continue;"
        "        var a=tds[0].querySelector('a'); var href=a ? a.href : '';"
        "        var summary=textOf(tds[1]);"
        "        out.push({summary:summary, href:href});"
        "      }"
        "      return out;"
        "    }"
        "    var lines=[], total=0;"
        "    for(var k=0;k<sel.length;k++){"
        "      var code=sel[k];"
        "      var tr=document.querySelector(\"tr.top-row[data-code='\"+code+\"']\"); if(!tr) continue;"
        "      var codeTxt=textOf(tr.querySelector('.code'));"
        "      var unmTxt =textOf(tr.querySelector('.unmasked'));"
        "      var cntTxt =textOf(tr.querySelector('.count a'));"
        "      var header = codeTxt + (unmTxt? ' — '+unmTxt : '') + (cntTxt? ' ('+cntTxt+')' : '');"
        "      lines.push('== ' + header + ' ==');"
        "      var matches = collectMatches(code);"
        "      if(!matches.length){"
        "        lines.push('- (no inline rows available)');"
        "      } else {"
        "        total += matches.length;"
        "        for(var m=0;m<matches.length;m++){"
        "          var row = matches[m];"
        "          lines.push('- ' + row.summary);"
        "          if(row.href){ lines.push(row.href); }"
        "        }"
        "      }"
        "      if(k < sel.length-1) lines.push('');"
        "    }"
        "    var subj='ONS Yates: ' + sel.length + ' person(s), ' + total + ' match row(s)';"
        "    var body=lines.join('\\n');"
        "    var href='mailto:?subject='+enc(subj)+'&body='+enc(body);"
        "    window.location.href=href;"
        "  }); }"

        "  var btnPrint=document.getElementById('btn-print');"
        "  if(btnPrint){ btnPrint.addEventListener('click', function(){"
        "    var sel=selectedCodes(); if(!sel.length){ alert('No persons selected.'); return; }"
        "    for(var i=0;i<sel.length;i++){ toggleDetails(sel[i], true); }"
        "    setTimeout(function(){ window.print(); }, 100);"
        "  }); }"

        "  var tbl=document.getElementById('mc-table');"
        "  if(tbl){ tbl.addEventListener('click',function(ev){"
        "    var a=ev.target && ev.target.closest ? ev.target.closest('a.count-toggle') : null;"
        "    if(!a) return; ev.preventDefault();"
        "    var code=a.getAttribute('data-code'); if(!code) return;"
        "    var row=document.getElementById('details-'+code); if(!row) return;"
        "    var open=row.style.display!== 'none';"
        "    row.style.display = open ? 'none' : '';"
        "    a.setAttribute('aria-expanded', open?'false':'true');"
        "    if(!open){ try{ row.scrollIntoView({behavior:'smooth',block:'start'});}catch(e){} }"
        "  },false); }"
        " });"
        "})();</script>"
        "</body></html>"
    )
    return html

match_partial_html = _render_match_count_full_html(usage_df_alpha)

# ---------- Lineage Count partial ----------
lineage_style = (
    "<style>"
    " body{font-family:Georgia,'Times New Roman',serif;background:#fff;color:#111;margin:0;padding:18px;line-height:1.6;font-size:15px}"
    " .wrap{max-width:1100px;margin:0 auto}"
    " h2{font-size:20px;margin:0 0 10px 0;text-align:center;border-bottom:2px solid #5b79b8;padding-bottom:6px}"
    " .updated{font-size:12px;color:#555;text-align:center;margin:2px 0 10px 0}"
    " table.partial-table{border-collapse:collapse;width:100%;table-layout:auto}"
    " table.partial-table th,table.partial-table td{border:1px solid #ccc;padding:8px 10px;vertical-align:top}"
    " table.partial-table th{background:#e9eef9;position:sticky;top:0}"
    " @media screen and (max-width:900px){ body{padding:12px;font-size:14px} }"
    "</style>"
)
lineage_updated = (
    "<div class='updated'>"
    "<a href='" + HOME_URL + "' target='_blank' rel='noopener'>Home</a>"
    " &nbsp;|&nbsp; Last updated: <span id='last-updated'></span>"
    " &nbsp;|&nbsp; Autosomal matches: <span id='auto-count' class='js-count'></span>"
    "</div>"
)
lineage_js = (
    "<script>(function(){"
    " document.addEventListener('DOMContentLoaded', function(){"
    "  function z(n){return(n<10?'0':'')+n;}"
    "  function stamp(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date());"
    "   el.textContent=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());}"
    "  function fmt(n){try{var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US');}catch(e){return String(n||'');}}"
    "  function loadCnt(){var el=document.getElementById('auto-count'); if(!el) return; var url='" + JS_COUNT_URL + "';"
    "   try{var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);"
    "    xhr.onreadystatechange=function(){ if(xhr.readyState===4){ if(xhr.status>=200&&xhr.status<300){ var m=(xhr.responseText||'').match(/(\\d+)/); el.textContent = fmt(m?m[1]:'') || '(unavailable)'; } else { el.textContent='(unavailable)'; } } };"
    "    xhr.send(null);}catch(e){ el.textContent='(unavailable)'; }}"
    "  stamp(); loadCnt();"
    " });"
    "})();</script>"
)

lineage_partial_html = (
    "<!DOCTYPE html><html lang='en'><head>"
    "<meta charset='iso-8859-15' />"
    "<meta name='viewport' content='width=device-width, initial-scale=1.0' />"
    "<title>Lineage Count (by oldest distant ancestor pair)</title>"
    + lineage_style +
    "</head><body>"
    "<div class='wrap'>"
    "<h2>Lineage Count (by oldest distant ancestor pair)</h2>"
    + lineage_updated +
    lineage_count_df.to_html(index=False, escape=False, classes='partial-table', border=0) +
    "</div>"
    + lineage_js +
    "</body></html>"
)

# 4) Write & upload partials
with open("match_count.htm", "w", encoding=ENC, errors="xmlcharrefreplace") as f:
    f.write(match_partial_html)
with open("lineage_count.htm", "w", encoding=ENC, errors="xmlcharrefreplace") as f:
    f.write(lineage_partial_html)
print("✅ Wrote partials locally: match_count.htm, lineage_count.htm")

try:
    with ftp_connect() as ftps:
        try:
            ftps.mkd("partials")
        except Exception:
            pass
        ftp_upload_overwrite(ftps, "match_count.htm",   "partials/match_count.htm")
        ftp_upload_overwrite(ftps, "lineage_count.htm", "partials/lineage_count.htm")
        try: ftps.quit()
        except Exception: pass
    print("✅ Published count partials to /partials/")
except Exception as e:
    print(f"⚠️ Upload of count partials failed: {e}")

# 5) Printable page (adds dynamic header)
PRINT_PARTIAL_NAME  = "cousin_list_print.htm"
PRINT_PARTIAL_LOCAL = PRINT_PARTIAL_NAME
_print_table_html = (display_df_filtered if not display_df_filtered.empty else display_df_all).to_html(
    index=False, escape=False, classes="print-table", border=1
)
PRINT_PARTIAL_HTML = (
    "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\"\n"
    "  \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">\n"
    "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\">\n"
    "<head>\n"
    "<meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />\n"
    "<meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\" />\n"
    "<title>ONS Yates Study &ndash; Cousin List (Printable)</title>\n"
    "<style type=\"text/css\">\n"
    "  body { font-family: Georgia, 'Times New Roman', serif; color:#000; background:#fff; margin:0; padding:16px; line-height:1.45; }\n"
    "  .wrap { max-width: 1100px; margin: 0 auto; }\n"
    "  h1 { font-size: 22px; margin: 0 0 8px 0; text-align:center; }\n"
    "  .updated { font-size:12px; color:#555; text-align:center; margin:2px 0 12px 0; }\n"
    "  .actions { text-align:center; margin: 10px 0 16px 0; }\n"
    "  .btn { display:inline-block; border:1px solid #444; background:#f4f4f4; color:#000; padding:6px 12px; text-decoration:none; }\n"
    "  table.print-table { border-collapse: collapse; width: 100%; table-layout: fixed; }\n"
    "  table.print-table th, table.print-table td { border:  1px solid #222; padding: 6px 8px; vertical-align: top; }\n"
    "  table.print-table th { background: #eee; }\n"
    "  @media print { .actions { display: none; } @page { margin: 0.6in; } }\n"
    "</style>\n"
    "</head>\n"
    "<body>\n"
    "  <div class=\"wrap\">\n"
    "    <h1>ONS Yates Study &ndash; Cousin List (Printable)</h1>\n"
    "    <div class=\"updated\"><a href=\"" + HOME_URL + "\" target=\"_blank\" rel=\"noopener\">Home</a>"
    " &nbsp;|&nbsp; Last updated: <span id=\"last-updated\"></span>"
    " &nbsp;|&nbsp; Autosomal matches: <span id=\"auto-count\" class=\"js-count\"></span></div>\n"
    "    <div class=\"actions\"><a class=\"btn\" href=\"#\" onclick=\"window.print();return false;\">Print this page</a></div>\n"
    "   " + _print_table_html + "\n"
    "    <div class=\"actions\" style=\"margin-top:16px;\"><a class=\"btn\" href=\"#\" onclick=\"window.print();return false;\">Print this page</a></div>\n"
    "  </div>\n"
    "<script>(function(){"
    " document.addEventListener('DOMContentLoaded', function(){"
    "  function z(n){return(n<10?'0':'')+n;}"
    "  function stamp(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date());"
    "   el.textContent=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());}"
    "  function fmt(n){try{var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US');}catch(e){return String(n||'');}}"
    "  function loadCnt(){var el=document.getElementById('auto-count'); if(!el) return; var url='" + JS_COUNT_URL + "';"
    "   try{var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);"
    "    xhr.onreadystatechange=function(){ if(xhr.readyState===4){ if(xhr.status>=200&&xhr.status<300){ var m=(xhr.responseText||'').match(/(\\d+)/); el.textContent = fmt(m?m[1]:'') || '(unavailable)'; } else { el.textContent='(unavailable)'; } } };"
    "    xhr.send(null);}catch(e){ el.textContent='(unavailable)'; }}"
    "  stamp(); loadCnt();"
    " });"
    "})();</script>"
    "</body>\n"
    "</html>\n"
)
with open(PRINT_PARTIAL_LOCAL, "w", encoding=ENC, errors="xmlcharrefreplace") as f:
    f.write(PRINT_PARTIAL_HTML)

try:
    with ftp_connect() as ftps:
        try:
            ftps.mkd("partials")
        except Exception:
            pass
        ftp_upload_overwrite(ftps, PRINT_PARTIAL_LOCAL, "partials/" + PRINT_PARTIAL_NAME)
        try: ftps.quit()
        except Exception: pass
    print(f"✅ Uploaded printable page: /partials/{PRINT_PARTIAL_NAME}")
except Exception as e:
    print(f"⚠️ Printable page upload failed: {e}")

print(f"✅ HTML published at https://yates.one-name.net/{REMOTE_NAME}")
# ====== CUT STOP [6/6] REFACTOR-Gold 2 — Save + Upload + Reports + Partials + Printable ===========



















✅ Loaded CSV — 1556 rows, 9 columns from /content/final_combined_df_with_value_labels.csv
✅ Columns: {'ID (detected)': 'ID#', 'Match to': 'Match to', 'Name': 'Name', 'cM': 'cM', 'Lineage': 'Yates DNA Ancestral Line'}
✅ Wrote local CSV (Column A): /content/the_match_cousins.csv
✅ Wrote HTML: /content/ons_yates_dna_register.htm
✅ Published autosomal count: autosomal_count.txt → 
✅ Wrote resolver usage CSV: /content/resolver_usage_report.csv
✅ Wrote lineage count CSV: /content/lineage_count_report.csv
✅ Published report CSVs.
✅ Wrote partials locally: match_count.htm, lineage_count.htm
✅ Published count partials to /partials/
✅ Uploaded printable page: /partials/cousin_list_print.htm
✅ HTML published at https://yates.one-name.net/ons_yates_dna_register.htm
✅ Loaded CSV — 1556 rows, 9 columns from /content/final_combined_df_with_value_labels.csv
✅ Columns: {'ID': 'id#', 'Match to': 'match to', 'Name': 'name', 'cM': 'cm', 'Lineage': 'yates dna ancestral line', 'SubjectCode': 'id#'}
✅ Wrote 

In [15]:
#EXPERIMENTAL 3
# Gold 3 — create yates_ancestor_register.htm (mobile-friendly, sortable)

# ====== CUT START [A] REFACTOR-Gold 3 — Config + FTP + Constants + Rules =========================
import os
import posixpath
import socket
from ftplib import FTP_TLS

# ---------- Script identity ----------
SCRIPT_NAME = "REFACTOR-Gold 3"
SECTION_NAME = "[A] Config + FTP + Constants + Rules"
DOM_ID_PREFIX = "g3-"  # all HTML ids/classes must begin with "g3-"
NS_PREFIX = "G3_"      # Python-visible constant/function prefix

# ---------- Encoding ----------
ENC = "iso-8859-15"

# ---------- Environment / FTP ----------
FTP_HOST = os.getenv("FTP_HOST", "")
FTP_USER = os.getenv("FTP_USER", "")
FTP_PASS = os.getenv("FTP_PASS", "")
FTP_DIR  = os.getenv("FTP_DIR", "/gengen/")
FTP_TIMEOUT = int(os.getenv("FTP_TIMEOUT", "45"))
FTP_TLS_REQUIRED = True  # enforce FTPS

# ---------- File naming (Gold 3: Lineages) ----------
# Main page (this already exists live per Ron): yates_ancestor_register.htm
LOCAL_NAME  = os.getenv("G3_LOCAL_NAME",  "yates_ancestor_register.htm")
REMOTE_NAME = os.getenv("G3_REMOTE_NAME", "yates_ancestor_register.htm")

# Partials unique to Gold 3 to avoid clashes with Gold 2
PARTIAL_MATCH_COUNT   = os.getenv("G3_PARTIAL_MATCH_COUNT",   "partials/g3_match_count.htm")
PARTIAL_LINEAGE_COUNT = os.getenv("G3_PARTIAL_LINEAGE_COUNT", "partials/g3_lineage_count.htm")
PARTIAL_PRINTABLE     = os.getenv("G3_PARTIAL_PRINTABLE",     "partials/g3_print.htm")

# Optional counters/diagnostics (distinct from Gold 2)
COUNT_PUBLIC_URL = os.getenv("G3_COUNT_PUBLIC_URL", "")
LOCAL_COUNT_FILE = os.getenv("G3_LOCAL_COUNT_FILE", "g3_autosomal_count.txt")
REMOTE_COUNT_NAME = os.getenv("G3_REMOTE_COUNT_NAME", "g3_autosomal_count.txt")

# ---------- Paths ----------
def _remote_path(name: str) -> str:
    """Join FTP_DIR and name with posix semantics."""
    base = FTP_DIR.rstrip("/")
    return f"{base}/{name.lstrip('/')}"

# ---------- FTP helpers (namespaced logic, no side effects on import) ----------
def ftp_connect() -> FTP_TLS:
    """Return a logged-in FTPS handle with passive mode and timeout."""
    if not FTP_HOST or not FTP_USER or not FTP_PASS:
        raise RuntimeError("FTP credentials are not set in environment variables.")
    ftps = FTP_TLS(timeout=FTP_TIMEOUT)
    ftps.connect(host=FTP_HOST, port=21)
    ftps.auth()  # upgrade to TLS
    ftps.prot_p()  # secure data connection
    ftps.login(user=FTP_USER, passwd=FTP_PASS)
    ftps.set_pasv(True)
    # Best-effort hostname note (for diagnostics)
    try:
        ftps.hostname = socket.gethostbyname(FTP_HOST)  # type: ignore[attr-defined]
    except Exception:
        pass
    return ftps

def ftp_upload_overwrite(ftps: FTP_TLS, local_path: str, remote_name: str) -> None:
    """Upload file to FTP_DIR/remote_name, overwriting if exists."""
    remote = _remote_path(remote_name)
    # Ensure remote directory exists (best-effort, ignore failures on nested)
    parts = remote.split("/")
    dir_parts, filename = parts[:-1], parts[-1]
    cwd = "/"
    try:
        ftps.cwd("/")
    except Exception:
        pass
    for d in dir_parts:
        if not d:
            continue
        cwd = posixpath.join(cwd, d)
        try:
            ftps.mkd(cwd)
        except Exception:
            pass
    with open(local_path, "rb") as fh:
        ftps.storbinary(f"STOR {remote}", fh)

# ---------- Guardrails ----------
# This script must not import or reference any G2_* symbols.
# All DOM ids, classes, and JS functions created by this script must use the "g3-" prefix.
# All files written by this script must use the Gold 3 filenames defined above.
# ====== CUT STOP  [A] REFACTOR-Gold 3 — Add 5 returns for spacing ================
#
#
#
#
#
# ====== CUT START [B] REFACTOR-Gold 3 — Helpers / Resolver (root-level /partials support) ========





import os
import io
import csv
import posixpath
from typing import Dict, Tuple

try:
    import pandas as pd
except Exception:
    pd = None

# ---------- Absolute root-level /partials support (G3) ----------
# Primary location: /partials/match_to_unmasked.csv (absolute from server root)
# Fallback location: /gengen/partials/match_to_unmasked.csv (under FTP_DIR)
G3_SERVER_PARTIALS_DIR = os.getenv("G3_SERVER_PARTIALS_DIR", "/partials")
G3_SERVER_MAPPING_BASENAME = os.getenv("G3_SERVER_MAPPING_BASENAME", "match_to_unmasked.csv")
G3_LOCAL_MAPPING_CACHE = os.getenv("G3_LOCAL_MAPPING_CACHE", "g3_match_to_unmasked.csv")

def g3_remote_abs(*parts: str) -> str:
    """Join into an absolute POSIX path that does NOT prepend FTP_DIR."""
    cleaned = [str(p).strip("/") for p in parts if p is not None]
    return "/" + "/".join([p for p in cleaned if p])

def g3_remote_under_ftpdir(name: str) -> str:
    """Fallback to FTP_DIR-prefixed path (e.g., /gengen/partials/...)."""
    base = str(globals().get("FTP_DIR", "/")).rstrip("/")
    return f"{base}/{name.lstrip('/')}"

def g3_ftps_fetch_to_local(ftps, remote_path: str, local_path: str) -> Tuple[bool, str]:
    """Try to RETR a remote file to local_path. Return (ok, error_text)."""
    try:
        # Ensure parent dir exists
        parent = os.path.dirname(local_path)
        if parent and not os.path.isdir(parent):
            os.makedirs(parent, exist_ok=True)
        with open(local_path, "wb") as fh:
            ftps.retrbinary(f"RETR {remote_path}", fh.write)
        return True, ""
    except Exception as e:
        return False, f"{type(e).__name__}: {e}"

def g3_detect_cols(df) -> Tuple[str, str]:
    """Pick code and unmasked columns from a DataFrame by heuristics."""
    cols = [str(c) for c in df.columns]
    code_candidates = ["Match to", "match_to", "match to", "code", "Code"]
    name_candidates = ["Unmasked", "unmasked", "name", "Name", "Unmask", "unmask"]
    code_col = None
    name_col = None
    lowered = {c.lower(): c for c in cols}
    for c in code_candidates:
        key = c.replace(" ", "")
        for col in cols:
            if col == c or col.lower().replace(" ", "") == key:
                code_col = col
                break
        if code_col:
            break
    for c in name_candidates:
        key = c.replace(" ", "")
        for col in cols:
            if col == c or col.lower().replace(" ", "") == key:
                name_col = col
                break
        if name_col:
            break
    if code_col is None:
        code_col = cols[0]
    if name_col is None:
        name_col = cols[1] if len(cols) > 1 else cols[0]
    return code_col, name_col

def g3_read_resolver_csv(local_path: str) -> Dict[str, str]:
    """Load resolver csv to {code: unmasked} with minimal normalization."""
    if pd is not None:
        df = pd.read_csv(local_path, dtype=str, encoding="iso-8859-15", keep_default_na=False)
        code_col, name_col = g3_detect_cols(df)
        mapping = {}
        for _, row in df.iterrows():
            k = str(row.get(code_col, "")).strip()
            v = str(row.get(name_col, "")).strip()
            if k:
                mapping[k] = v
        return mapping
    # Fallback without pandas
    mapping = {}
    with open(local_path, "r", encoding="iso-8859-15", errors="replace", newline="") as fh:
        reader = csv.reader(fh)
        rows = list(reader)
    if not rows:
        return mapping
    header = rows[0]
    if len(header) >= 2 and not header[0].strip().isdigit():
        # Assume first row is header
        data_rows = rows[1:]
        code_idx = 0
        name_idx = 1
    else:
        # No header
        data_rows = rows
        code_idx = 0
        name_idx = 1 if len(rows[0]) > 1 else 0
    for r in data_rows:
        if not r:
            continue
        k = (r[code_idx] if len(r) > code_idx else "").strip()
        v = (r[name_idx] if len(r) > name_idx else "").strip()
        if k:
            mapping[k] = v
    return mapping

def g3_load_resolver_from_server() -> Dict[str, str]:
    """Load resolver CSV from root-level /partials first, then fallback to /gengen/partials.
    Writes a local cache copy and returns the mapping dict.
    """
    if "ftp_connect" not in globals():
        raise RuntimeError("ftp_connect() is not available; ensure Section [A] is loaded first.")
    ftps = ftp_connect()
    try:
        # Primary: absolute root /partials
        primary = g3_remote_abs(G3_SERVER_PARTIALS_DIR, G3_SERVER_MAPPING_BASENAME)  # e.g., /partials/match_to_unmasked.csv
        ok, err = g3_ftps_fetch_to_local(ftps, primary, G3_LOCAL_MAPPING_CACHE)
        if not ok:
            # Fallback: FTP_DIR/partials (e.g., /gengen/partials/...)
            fallback = g3_remote_under_ftpdir(posixpath.join("partials", G3_SERVER_MAPPING_BASENAME))
            ok2, err2 = g3_ftps_fetch_to_local(ftps, fallback, G3_LOCAL_MAPPING_CACHE)
            if not ok2:
                raise RuntimeError(
                    "Resolver not found on server at either location:\n"
                    f"  1) {primary}\n"
                    f"  2) {fallback}\n"
                    "Action: Upload match_to_unmasked.csv to /partials/ (root) or to /gengen/partials/ and re-run.\n"
                    f"Details: primary error: {err}; fallback error: {err2}"
                )
        mapping = g3_read_resolver_csv(G3_LOCAL_MAPPING_CACHE)
        return mapping
    finally:
        try:
            ftps.quit()
        except Exception:
            pass

# Global resolver dict for Gold 3
G3_MATCH_TO_UNMASKED = g3_load_resolver_from_server()

# ====== CUT STOP  [B] REFACTOR-Gold 3 — Add 5 returns for spacing ===============================
#
#
#
#
#
# ====== CUT START [C] REFACTOR-Gold 3 — CSV Load & Column Detection ==============================





import os
import re
from typing import Dict, List, Optional

try:
    import pandas as pd
except Exception:
    pd = None

# ---------- Purpose ----------
# Load the canonical Gold 1 output (CSV) and detect core columns for lineages.
# Exposes:
#   - g3_df_core : the loaded DataFrame (no HTML transforms)
#   - g3_cols    : dict of detected column names (id, name, match_to, subject_code, lineage-ish hints)
#   - G3_COLUMNS : optional preferred display order for [E]/[F] (set only if obvious)

# ---------- Source selection (Gold 1 output) ----------
# You can point at either env var or default filename in current working dir.
G3_SOURCE_CSV = os.getenv("G3_SOURCE_CSV", os.getenv("G1_CSV_PATH", "final_combined_df_with_value_labels.csv"))
ENC = globals().get("ENC", "iso-8859-15")

# ---------- Local helpers (namespaced; independent of Gold 2) ----------
def g3_normalize_colnames(df):
    if df is None or getattr(df, "columns", None) is None:
        return df
    new = df.copy()
    new.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]
    return new

def g3_find_col(df, regex_list: Optional[List[str]] = None, fallback_list: Optional[List[str]] = None) -> Optional[str]:
    if df is None or getattr(df, "columns", None) is None:
        return None
    regex_list = regex_list or []
    fallback_list = fallback_list or []
    cols = list(df.columns)
    # regex
    for pattern in regex_list:
        try:
            rx = re.compile(pattern, flags=re.IGNORECASE)
        except Exception:
            continue
        for c in cols:
            if rx.search(str(c)):
                return c
    # exact fallback
    for name in fallback_list:
        t = str(name).strip().lower()
        for c in cols:
            if str(c).strip().lower() == t:
                return c
    return cols[0] if cols else None

def g3_find_cols(df, patterns_or_names: List[str]) -> List[str]:
    hits = []
    if df is None or getattr(df, "columns", None) is None:
        return hits
    cols = list(df.columns)
    for token in patterns_or_names:
        is_regex = bool(re.search(r"[\\^$.*+?()\\[\\]|]", token))
        if is_regex:
            try:
                rx = re.compile(token, flags=re.IGNORECASE)
            except Exception:
                continue
            for c in cols:
                if rx.search(str(c)):
                    hits.append(c)
        else:
            low = token.strip().lower()
            for c in cols:
                if str(c).strip().lower() == low:
                    hits.append(c)
    # de-dupe
    seen = set()
    uniq = []
    for c in hits:
        if c not in seen:
            uniq.append(c)
            seen.add(c)
    return uniq

# ---------- Load CSV ----------
if pd is None:
    raise RuntimeError("pandas is required in Section [C]; install/enable pandas in Colab.")

if not os.path.isfile(G3_SOURCE_CSV):
    raise RuntimeError(
        "Gold 3 cannot find the Gold 1 CSV.\n"
        f"Looked for: {G3_SOURCE_CSV}\n"
        "Set env G3_SOURCE_CSV or G1_CSV_PATH to the correct file, then re-run."
    )

g3_df_core = pd.read_csv(G3_SOURCE_CSV, dtype=str, encoding=ENC, keep_default_na=False)
g3_df_core = g3_normalize_colnames(g3_df_core)

# ---------- Column detection ----------
g3_cols: Dict[str, Optional[str]] = {}

g3_cols['id'] = g3_find_col(
    g3_df_core,
    [r'^(id#|personid|id)$', r'person\s*id'],
    ["ID#", "PersonID", "personID", "id"]
)

g3_cols['match_to'] = g3_find_col(
    g3_df_core,
    [r'^match\s*to$', r'^matchto$'],
    ["Match to", "Match"]
)

g3_cols['name'] = g3_find_col(
    g3_df_core,
    [r'^(name|full\s*name)$'],
    ["Name"]
)

# subject code / resolver code column (used widely in both tools)
g3_cols['subject_code'] = g3_find_col(
    g3_df_core,
    [r'^(subject\s*code|subjectcode|code)$'],
    ["Subject Code", "subject_code", "Code"]
)

# lineage-ish hints (we won't enforce one here; [D] can decide which to use)
# Try to gather potential columns used for grouping/lineage display.
lineage_candidates = g3_find_cols(
    g3_df_core,
    [
        r'lineage', r'branch', r'child', r'ancestor', r'pair', r'couple',
        "Named Pair", "named pair", "Pair", "pair"
    ]
)
g3_cols['lineage_candidates'] = lineage_candidates

# Optional suggested order for [E], if we detect the common trio
# You can override G3_COLUMNS in later sections if needed.
if all([g3_cols.get('match_to'), g3_cols.get('name')]):
    # We do not set 'Lineage' automatically here; [D] will craft a final column.
    G3_COLUMNS = ["Find", "Match Summary", "Lineage"]

# No disk writes in this section.
# ====== CUT STOP  [C] REFACTOR-Gold 3 — Add 5 returns for spacing ===============================
#
#
#
#
#
# ====== CUT START [D] REFACTOR-Gold 3 — Transform (display_df) ================================





import re
from typing import Dict, List, Optional

try:
    import pandas as pd
except Exception:
    pd = None

# ---------- Assumptions from prior sections ----------
# - [B] may define: G3_MATCH_TO_UNMASKED (resolver dict) — optional
# - [C] defines: g3_df_core (DataFrame), g3_cols (dict with keys: id, name, match_to, subject_code, lineage_candidates)
# - This section produces: g3_display_df (3-column DataFrame: Find | Match Summary | Lineage)
# - No uploads or disk writes here.

ENC = globals().get("ENC", "iso-8859-15")

def g3_html_escape(s: str) -> str:
    import html as _h
    return _h.escape("" if s is None else str(s), quote=True)

def g3_slugify(s: str) -> str:
    s = re.sub(r"[^A-Za-z0-9]+", "-", "" if s is None else str(s))
    s = re.sub(r"-{2,}", "-", s).strip("-")
    return s.lower()

def g3_coalesce(*vals) -> str:
    for v in vals:
        if v is None:
            continue
        s = str(v).strip()
        if s and s.lower() != "nan":
            return s
    return ""

def g3_build_lineage_html(row, lineage_cols: List[str]) -> str:
    """Join available lineage columns into a compact breadcrumb using &rarr;."""
    parts = []
    for c in lineage_cols:
        try:
            val = row.get(c, "")
        except Exception:
            try:
                val = row[c]
            except Exception:
                val = ""
        txt = str(val).strip()
        if not txt or txt.lower() == "nan":
            continue
        parts.append(g3_html_escape(txt))
    if not parts:
        return "<span class=\"g3-lineage\">&hellip;</span>"
    return "<span class=\"g3-lineage\">" + " &rarr; ".join(parts) + "</span>"

def g3_resolve_name(code_value: str, fallback_name: str) -> str:
    """Use G3_MATCH_TO_UNMASKED when present; otherwise fallback to provided name."""
    mapping = globals().get("G3_MATCH_TO_UNMASKED", None)
    if mapping and isinstance(mapping, dict):
        resolved = mapping.get(str(code_value).strip(), "")
        if resolved:
            return resolved
    return fallback_name

def g3_build_display_df(df_core, cols: Dict[str, Optional[str]]) -> "pd.DataFrame":
    if pd is None:
        raise RuntimeError("pandas is required in Section [D]; install/enable pandas in Colab.")
    if df_core is None or cols is None:
        raise RuntimeError("g3_df_core / g3_cols are required to build g3_display_df.")

    id_c   = cols.get("id")
    name_c = cols.get("name")
    code_c = cols.get("match_to")
    subj_c = cols.get("subject_code")
    lineage_cands = cols.get("lineage_candidates") or []

    # Build rows
    out_rows = []
    for _, row in df_core.iterrows():
        id_val   = g3_coalesce(row.get(id_c) if id_c else None)
        name_val = g3_coalesce(row.get(name_c) if name_c else None)
        code_val = g3_coalesce(row.get(code_c) if code_c else None)
        subj_val = g3_coalesce(row.get(subj_c) if subj_c else None)

        # Anchor id: prefer subject_code, then id, else slugified name or code
        anchor_key = g3_coalesce(subj_val, id_val, name_val, code_val)
        anchor = g3_slugify(anchor_key) if anchor_key else "row"

        # Resolved name for match summary
        resolved = g3_resolve_name(code_val, name_val or code_val)

        # --- Find column ---
        # Show display text = resolved name if available, else name/code/id.
        disp_text = g3_coalesce(resolved, name_val, code_val, id_val)
        find_html = f"<a id=\"g3-{anchor}\" name=\"g3-{anchor}\">{g3_html_escape(disp_text)}</a>"

        # --- Match Summary column ---
        # Show resolved name and (code) if distinct; append subject code in small text if present.
        ms_bits = []
        if resolved:
            ms_bits.append(g3_html_escape(resolved))
        if code_val and (resolved.strip().lower() != code_val.strip().lower()):
            ms_bits.append(f"(<span class=\"g3-code\">{g3_html_escape(code_val)}</span>)")
        if subj_val:
            ms_bits.append(f"<span class=\"g3-subject\">&nbsp;&middot;&nbsp;{g3_html_escape(subj_val)}</span>")
        ms_html = " ".join(ms_bits) if ms_bits else g3_html_escape(disp_text)

        # --- Lineage column (HTML) ---
        lineage_html = g3_build_lineage_html(row, lineage_cands)

        out_rows.append({
            "Find": find_html,
            "Match Summary": ms_html,
            "Lineage": lineage_html,
        })

    display_df = pd.DataFrame(out_rows, columns=["Find", "Match Summary", "Lineage"])
    return display_df

# ---------- Public artifact ----------
g3_display_df = None
try:
    if 'g3_df_core' in globals() and 'g3_cols' in globals():
        g3_display_df = g3_build_display_df(g3_df_core, g3_cols)
except Exception as _e_g3:
    # Leave as None; Section [E] or [F] can call g3_build_display_df(...) explicitly.
    g3_display_df = None

# No disk writes here.
# ====== CUT STOP  [D] REFACTOR-Gold 3 — Add 5 returns for spacing =============================
#
#
#
#
#
# ====== CUT START [E] REFACTOR-Gold 3 — HTML (no uploads) =========================





import html as _py_html
import math

# ---------- Assumptions from prior sections ----------
# - A: ftp_connect(), ftp_upload_overwrite(), _remote_path(), LOCAL_NAME/REMOTE_NAME (G3)
# - B/C/D prepare a DataFrame assigned to g3_display_df OR provide a function to retrieve it.
# - The page built here must use only "g3-" DOM ids/classes and "g3_" JS functions.
#
# This section does NOT perform any FTP writes. Saving/uploading is handled in Section [F].

# ---------- Config / safe defaults (do not collide with G2) ----------
ENC = globals().get("ENC", "iso-8859-15")
G3_TITLE = globals().get("G3_TITLE", "Yates Ancestor Register &ndash; Lineages")
TABLE_WIDTH_PX = int(globals().get("G3_TABLE_WIDTH_PX", 1150))
COL_A_PX = int(globals().get("G3_COL_A_PX", 320))   # first column width
COL_B_PX = int(globals().get("G3_COL_B_PX", 300))   # second column width
COL_C_PX = int(globals().get("G3_COL_C_PX", 530))   # third column width

# The main output filename is set in Section [A] as LOCAL_NAME / REMOTE_NAME.
LOCAL_NAME = globals().get("LOCAL_NAME", "yates_ancestor_register.htm")

# Column order expectation: allow prior sections to define, else fall back to df columns.
G3_COLUMNS = globals().get("G3_COLUMNS", None)  # e.g., ["Find", "Match Summary", "Lineage"]

# ---------- Helpers ----------
def g3_html_escape(s: str) -> str:
    return _py_html.escape("" if s is None else str(s), quote=True)

def g3_format_int(n) -> str:
    try:
        i = int(n)
        return f"{i:,}"
    except Exception:
        return g3_html_escape(n)

def g3_build_table_html(df) -> str:
    """Render the main lineage table using only g3-* ids/classes.
    We intentionally do NOT assume column names; we respect G3_COLUMNS if provided.
    """
    if df is None:
        return "<!-- g3: no dataframe available -->"
    cols = list(G3_COLUMNS) if isinstance(G3_COLUMNS, (list, tuple)) and len(G3_COLUMNS) > 0 else list(df.columns)

    # Column widths map (3-column expectation by default).
    col_widths = [COL_A_PX, COL_B_PX, COL_C_PX]
    # Ensure we have a width per visible column (pad with last width if needed).
    if len(cols) > 3:
        col_widths = col_widths + [col_widths[-1]] * (len(cols) - 3)

    # Table header
    ths = []
    for i, c in enumerate(cols):
        w = col_widths[i] if i < len(col_widths) else col_widths[-1]
        ths.append(f'<th style="width:{int(w)}px">{g3_html_escape(c)}</th>')
    thead = "<thead><tr>" + "".join(ths) + "</tr></thead>"

    # Table body
    trs = []
    for idx, row in df.iterrows():
        tds = []
        for i, c in enumerate(cols):
            val = row.get(c, "")
            # If this column presumably contains pre-built HTML from prior section (e.g., lineage cells),
            # we allow minimal pass-through for strings that look like HTML; otherwise escape it.
            if isinstance(val, str) and ("<a " in val or "<span" in val or "<div" in val or "<br" in val):
                cell_html = val
            else:
                cell_html = g3_html_escape(val)
            tds.append(f"<td>{cell_html}</td>")
        trs.append("<tr>" + "".join(tds) + "</tr>")
    tbody = "<tbody>" + "".join(trs) + "</tbody>"

    table = (
        f'<table id="g3-table" class="g3-table" role="table" aria-label="Lineage table" '
        f'style="width:{TABLE_WIDTH_PX}px">'
        f"{thead}{tbody}</table>"
    )
    return table

def g3_build_css() -> str:
    return (
        "<style type=\"text/css\">\n"
        "  html { scroll-behavior: smooth; }\n"
        "  body { font-family: Georgia, \"Times New Roman\", serif; background:#ffffff; color:#222; margin:0; padding:0; line-height:1.5; }\n"
        f"  .g3-wrap {{ max-width:{TABLE_WIDTH_PX}px; margin:0 auto; background:#ffffff; padding:20px; padding-bottom:48px; }}\n"
        "  a { color:#154b8b; text-decoration:none; } a:hover { text-decoration:underline; }\n"
        "  h1 { margin:0 0 6px 0; font-size:26px; line-height:1.2; text-align:center; }\n"
        "  .g3-updated { font-size:12px; color:#555; text-align:center; margin:2px 0 10px 0; }\n"
        "  .g3-menu { display:flex; gap:8px; flex-wrap:wrap; align-items:center; margin:8px 0 12px 0; }\n"
        "  .g3-menu .g3-pill { display:inline-block; border:1px solid #ccc; padding:6px 10px; border-radius:18px; font-size:13px; }\n"
        "  table.g3-table { border-collapse:collapse; width:100%; table-layout:fixed; }\n"
        "  table.g3-table th, table.g3-table td { border:1px solid #ddd; padding:8px; vertical-align:top; }\n"
        "  table.g3-table th { background:#ffffcc; text-align:left; font-weight:bold; position:sticky; top:0; z-index:2; }\n"
        "  .g3-foot { margin-top:28px; font-size:12px; color:#666; text-align:center; }\n"
        "  /* Print tweaks */\n"
        "  @media print {\n"
        "    .g3-menu { display:none !important; }\n"
        "    .g3-wrap { padding:0; }\n"
        "  }\n"
        "</style>\n"
    )

def g3_last_updated_snippet() -> str:
    """XHTML-safe last updated stamp, 24h YYYY-MM-DD HH:MM, using document.lastModified."""
    return (
        "<script type=\"text/javascript\">\n"
        "//<![CDATA[\n"
        "(function(){\n"
        "  var el = document.getElementById('last-updated');\n"
        "  if(!el) return;\n"
        "  var d = new Date(document.lastModified || new Date());\n"
        "  function z(n){ return (n < 10 ? '0' : '') + n; }\n"
        "  var formatted = d.getFullYear() + '-' + z(d.getMonth()+1) + '-' + z(d.getDate()) + ' ' + z(d.getHours()) + ':' + z(d.getMinutes());\n"
        "  el.innerHTML = 'Last updated: ' + formatted;\n"
        "})();\n"
        "//]]>\n"
        "</script>\n"
    )

def g3_build_header() -> str:
    return (\
        "<h1>Yates Ancestor Register &ndash; Lineages</h1>\n"
        "<div id=\"last-updated\" class=\"g3-updated\">Last updated: &hellip;</div>\n"
    )

def g3_build_menu() -> str:
    # Minimal, non-colliding toolbox; expand in later sections if needed.
    return (\
        "<div class=\"g3-menu\" id=\"g3-menu\">\n"
        "  <span class=\"g3-pill\">Lineages</span>\n"
        "  <span class=\"g3-pill\">Sortable</span>\n"
        "</div>\n"
    )

def g3_build_init_js() -> str:
    """Minimal init; placeholder for future search/sort hooks under g3_ namespace only."""
    return (\
        "<script type=\"text/javascript\">\n"
        "//<![CDATA[\n"
        "function g3_init(){\n"
        "  // Placeholder for future g3-specific behaviors (search, sort, etc.).\n"
        "}\n"
        "if (document.readyState === 'loading') { document.addEventListener('DOMContentLoaded', g3_init); } else { g3_init(); }\n"
        "//]]>\n"
        "</script>\n"
    )

def g3_build_full_html(df) -> str:
    css = g3_build_css()
    header = g3_build_header()
    menu = g3_build_menu()
    table_html = g3_build_table_html(df)
    init_js = g3_build_init_js()
    updated_js = g3_last_updated_snippet()
    # XHTML 1.0 Transitional
    doc = (\
        "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\" \n"
        "  \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">\n"
        "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\" xml:lang=\"en\">\n"
        "<head>\n"
        "  <meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />\n"
        f"  <title>{G3_TITLE}</title>\n"
        f"{css}"
        "</head>\n"
        "<body>\n"
        f"  <div class=\"g3-wrap\" id=\"g3-app\">\n"
        f"    {header}\n"
        f"    {menu}\n"
        f"    {table_html}\n"
        "    <div class=\"g3-foot\">&copy; Yates One-Name Study</div>\n"
        "  </div>\n"
        f"{updated_js}"
        f"{init_js}"
        "</body>\n"
        "</html>\n"
    )
    return doc

# ---------- Public artifact ----------
# If prior sections have prepared g3_display_df, build FULL_HTML now; else expose the function.
FULL_HTML = None
try:
    if 'g3_display_df' in globals() and g3_display_df is not None:
        FULL_HTML = g3_build_full_html(g3_display_df)
except Exception as _g3_e:
    # Leave FULL_HTML as None; Section [F] can call g3_build_full_html(...) explicitly.
    FULL_HTML = None

# This section must not perform any disk writes or FTP operations.
# ====== CUT STOP  [E] REFACTOR-Gold 3 — Add 5 returns for spacing ================
#
#
#
#
#
# ====== CUT START [F] REFACTOR-Gold 3 — Save + Upload + Partials + Printable =====================





import os
import io
import posixpath
from typing import Dict, Any

try:
    import pandas as pd  # used for CSV/printing helpers if available
except Exception:
    pd = None  # allow running without pandas in edge cases

# ---------- Assumptions from prior sections ----------
# - [A] defines: ftp_connect(), ftp_upload_overwrite(), _remote_path(),
#                LOCAL_NAME, REMOTE_NAME, PARTIAL_MATCH_COUNT, PARTIAL_LINEAGE_COUNT, PARTIAL_PRINTABLE
# - [E] defines: g3_build_full_html(df), FULL_HTML (optional), ENC, TABLE_WIDTH_PX
# - Prior section(s) produce: g3_display_df (DataFrame) with columns expected by [E]

# ---------- Local config ----------
ENC = globals().get("ENC", "iso-8859-15")

LOCAL_NAME  = globals().get("LOCAL_NAME",  "yates_ancestor_register.htm")
REMOTE_NAME = globals().get("REMOTE_NAME", "yates_ancestor_register.htm")

PARTIAL_MATCH_COUNT   = globals().get("PARTIAL_MATCH_COUNT",   "partials/g3_match_count.htm")
PARTIAL_LINEAGE_COUNT = globals().get("PARTIAL_LINEAGE_COUNT", "partials/g3_lineage_count.htm")
PARTIAL_PRINTABLE     = globals().get("PARTIAL_PRINTABLE",     "partials/g3_print.htm")

# Ensure local partial directories exist when saving to disk
def _ensure_parent_dir(path: str) -> None:
    try:
        parent = os.path.dirname(path)
        if parent and not os.path.isdir(parent):
            os.makedirs(parent, exist_ok=True)
    except Exception:
        pass

# ---------- Builders for partials ----------
def g3_build_match_count_html(df) -> str:
    """Return minimal HTML showing autosomal (row) count with comma formatting."""
    try:
        n = int(len(df)) if df is not None else 0
    except Exception:
        n = 0
    count_str = f"{n:,}"
    return (
        "<!DOCTYPE html>\n"
        "<html><head><meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" /></head>\n"
        "<body>\n"
        f"<div id=\"g3-match-count\" class=\"g3-pill\">Autosomal matches: {count_str}</div>\n"
        "</body></html>\n"
    )

def g3_build_lineage_count_html(df) -> str:
    """Return minimal HTML with a distinct lineage count if a 'Lineage' (or similar) column exists.
    Fallback: unique values of the last column if heuristics fail.
    """
    if df is None:
        uniq = 0
    else:
        cols = list(df.columns)
        target_col = None
        # Heuristics for likely lineage column names
        candidates = ["Lineage", "lineage", "Lineages", "lineages", "Lineage HTML", "lineage HTML"]
        for c in cols:
            if c in candidates or c.lower() in ["lineage", "lineages"]:
                target_col = c
                break
        if target_col is None:
            # fallback to last column
            target_col = cols[-1]
        try:
            uniq = len(set([str(x).strip() for x in df[target_col].tolist()]))
        except Exception:
            uniq = 0
    count_str = f"{uniq:,}"
    return (
        "<!DOCTYPE html>\n"
        "<html><head><meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" /></head>\n"
        "<body>\n"
        f"<div id=\"g3-lineage-count\" class=\"g3-pill\">Lineages: {count_str}</div>\n"
        "</body></html>\n"
    )

def g3_build_printable_html(df) -> str:
    """Simple printable view of the table (no toolbox)."""
    if df is None:
        table_rows = ""
        cols = []
    else:
        cols = list(df.columns)
        # Build rows with HTML-escape except for obvious inline HTML in cells
        def esc(v):
            s = "" if v is None else str(v)
            if ("<a " in s) or ("<span" in s) or ("<div" in s) or ("<br" in s):
                return s
            import html as _h
            return _h.escape(s, quote=True)
        trs = []
        for _, row in df.iterrows():
            tds = "".join([f"<td>{esc(row.get(c, ''))}</td>" for c in cols])
            trs.append(f"<tr>{tds}</tr>")
        table_rows = "\n".join(trs)
    thead = "".join([f"<th>{c}</th>" for c in cols])
    return (\
        "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\" \n"
        "  \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">\n"
        "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\" xml:lang=\"en\">\n"
        "<head>\n"
        "  <meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />\n"
        "  <title>Yates Ancestor Register &ndash; Printable</title>\n"
        "  <style type=\"text/css\">\n"
        "    body { font-family: Georgia, \"Times New Roman\", serif; background:#fff; color:#000; }\n"
        "    table { border-collapse: collapse; width: 100%; }\n"
        "    th, td { border: 1px solid #ccc; padding: 6px; vertical-align: top; }\n"
        "    th { background: #ffffcc; }\n"
        "    @media print { a { color:#000 !important; text-decoration:none; } }\n"
        "  </style>\n"
        "</head>\n"
        "<body>\n"
        "  <h1>Yates Ancestor Register &ndash; Printable</h1>\n"
        f"  <table id=\"g3-print-table\"><thead><tr>{thead}</tr></thead><tbody>\n"
        f"{table_rows}\n"
        "  </tbody></table>\n"
        "</body>\n"
        "</html>\n"
    )

# ---------- Save helpers ----------
def g3_write_text(local_path: str, text: str) -> None:
    _ensure_parent_dir(local_path)
    with open(local_path, "w", encoding=ENC, errors="replace") as fh:
        fh.write(text)

def g3_save_local(df) -> Dict[str, str]:
    """Write main HTML and partials to local disk. Returns dict of paths."""
    outputs = {}
    # Main
    if 'FULL_HTML' in globals() and isinstance(globals()['FULL_HTML'], str) and globals()['FULL_HTML']:
        html = globals()['FULL_HTML']
    else:
        if 'g3_build_full_html' not in globals():
            raise RuntimeError("g3_build_full_html is not available in scope.")
        if 'g3_display_df' not in globals():
            raise RuntimeError("g3_display_df is not available in scope.")
        html = g3_build_full_html(df)
    g3_write_text(LOCAL_NAME, html)
    outputs['main'] = os.path.abspath(LOCAL_NAME)

    # Partials
    g3_write_text(PARTIAL_MATCH_COUNT, g3_build_match_count_html(df))
    outputs['match_count'] = os.path.abspath(PARTIAL_MATCH_COUNT)

    g3_write_text(PARTIAL_LINEAGE_COUNT, g3_build_lineage_count_html(df))
    outputs['lineage_count'] = os.path.abspath(PARTIAL_LINEAGE_COUNT)

    g3_write_text(PARTIAL_PRINTABLE, g3_build_printable_html(df))
    outputs['printable'] = os.path.abspath(PARTIAL_PRINTABLE)

    return outputs

def g3_upload_all(ftps, local_map: Dict[str, str]) -> None:
    """Upload main and partials via already-authenticated FTPS connection."""
    if ftps is None:
        raise RuntimeError("FTPS handle is None.")
    # main
    ftp_upload_overwrite(ftps, local_map['main'], REMOTE_NAME)
    # partials: use same basename on server as local for clarity
    ftp_upload_overwrite(ftps, local_map['match_count'], PARTIAL_MATCH_COUNT)
    ftp_upload_overwrite(ftps, local_map['lineage_count'], PARTIAL_LINEAGE_COUNT)
    ftp_upload_overwrite(ftps, local_map['printable'], PARTIAL_PRINTABLE)

# ---------- Orchestrator ----------
def g3_run_save_and_upload(do_upload: bool = True) -> Dict[str, Any]:
    """Save all artifacts locally, and optionally upload over FTPS.
    Returns a summary dict with local paths and flags.
    """
    if 'g3_display_df' not in globals():
        raise RuntimeError("g3_display_df is not in scope; cannot render HTML/partials.")
    df = globals()['g3_display_df']
    out = {'saved': False, 'uploaded': False, 'paths': {}}

    paths = g3_save_local(df)
    out['paths'] = paths
    out['saved'] = True

    if do_upload:
        ftps = ftp_connect()
        try:
            g3_upload_all(ftps, paths)
            out['uploaded'] = True
        finally:
            try:
                ftps.quit()
            except Exception:
                pass
    return out

# ====== CUT STOP  [F] REFACTOR-Gold 3 — Add 5 returns for spacing =================




In [ ]:
# Gold 3 — create yates_ancestor_register.htm (mobile-friendly, sortable)

# ====== CUT START [A] REFACTOR-Gold 3 — Config + FTP + Constants + Rules =========================
import os, re, json, time, io, posixpath, socket
import pandas as pd
from ftplib import FTP_TLS

# ---------- Ron Rules ----------
# - All edits are full-section replacements using explicit CUT START/STOP lines.
# - Script is split into easily replaceable sections; provide code in python


# ---------- CONFIG ----------
# Column name defaults used across sections
subject_code_col = "Match to"
path_col = "Yates DNA Ancestral Line"
LINEAGE_HEADER = "Lineage (Starting with oldest ancestor, the line is:)"  # used in table headings
ARROW_ENTITY = "&rarr;"
REMOVE_PERIOD_AT_END = True

# Layout
TABLE_WIDTH_PX = 3150
COL_A_PX = 50  # width for "Match Name" column in HTML

# Data / Output
CSV_PATH = "final_combined_df_with_value_labels.csv"     # default input
LOCAL_NAME = "yates_ancestor_register.htm"               # main page filename (local)
REMOTE_NAME = "yates_ancestor_register.htm"              # remote filename (same name)

# Local exports
MATCH_COUSINS_CSV = "the_match_cousins.csv"  # Column A (what’s shown)

# Remote I/O toggles (same creds/dir as HTML upload)
REMOTE_READ = True      # Pull resolver CSV from server before processing
UPLOAD_COLUMN_A = True  # Push the_match_cousins.csv to server after build

# TNG link pieces for matchee hotlink (still used in some partials)
TNG_BASE = "https://yates.one-name.net/tng"
TNG_TREE = "tree1"

# FTP settings (hard timeouts)
FTP_DIR = os.environ.get("FTP_DIR", "").strip()  # e.g., "gengen" or ""
FTP_TIMEOUT = int(os.environ.get("FTP_TIMEOUT", "30"))  # connect + socket timeout
FTP_PASSIVE = True  # passive safest behind NAT

# ---------- Autosomal Count (Colab → Website) ----------
LOCAL_COUNT_FILE = "/content/autosomal_count.txt"  # produced in Colab
REMOTE_COUNT_NAME = "autosomal_count.txt"          # destination on host
COUNT_PUBLIC_URL = f"/{FTP_DIR}/{REMOTE_COUNT_NAME}" if FTP_DIR else f"/{REMOTE_COUNT_NAME}"

# ---------- Resolver (single source: website) ----------
SERVER_PARTIALS_DIR = "partials"
SERVER_MAPPING_BASENAME = "match_to_unmasked.csv"
SERVER_MAPPING_REMOTE = posixpath.join(SERVER_PARTIALS_DIR, SERVER_MAPPING_BASENAME)
SERVER_MAPPING_LOCAL_CACHE = "match_to_unmasked.server.csv"  # temp file we download to

# ---------- FTP helpers (with timeouts & cleanup) ----------
def ftp_connect():
    ftps = FTP_TLS(timeout=FTP_TIMEOUT)
    socket.setdefaulttimeout(FTP_TIMEOUT)
    ftps.connect(os.environ['FTP_HOST'], int(os.environ.get('FTP_PORT', 21)))
    ftps.login(os.environ['FTP_USER'], os.environ['FTP_PASS'])
    try:
        ftps.prot_p()  # secure data channel if supported
    except Exception:
        pass
    try:
        ftps.set_pasv(FTP_PASSIVE)
    except Exception:
        pass
    if FTP_DIR:
        try:
            ftps.cwd(FTP_DIR)
        except Exception:
            parts = [p for p in FTP_DIR.split("/") if p]
            for p in parts:
                try:
                    ftps.mkd(p)
                except Exception:
                    pass
                ftps.cwd(p)
    return ftps

def _remote_path(name: str) -> str:
    return posixpath.join(FTP_DIR, name) if FTP_DIR else name

def ftp_download_if_exists(ftps: FTP_TLS, remote_name: str, local_name: str) -> bool:
    try:
        with open(local_name, "wb") as f:
            ftps.retrbinary(f"RETR " + remote_name, f.write)
        print(f"⬇️ Pulled remote file: {remote_name} → {os.path.abspath(local_name)}")
        return True
    except Exception as e:
        print(f"ℹ️ Remote not found or unreadable: {remote_name} ({e})")
        try:
            if os.path.exists(local_name):
                os.remove(local_name)
        except Exception:
            pass
        return False

def ftp_upload_overwrite(ftps: FTP_TLS, local_path: str, remote_name: str):
    try:
        with open(local_path, "rb") as fh:
            ftps.storbinary(f"STOR {remote_name}", fh)
        print(f"⬆️ Uploaded: {local_path} → {remote_name}")
    except Exception as e:
        raise RuntimeError(f"Upload failed for {local_path} → {remote_name}: {e}")
# ====== CUT STOP [A] REFACTOR-Gold 3 — Config + FTP + Constants + Rules =========================


# ====== CUT START [B] REFACTOR-Gold 3 — Resolver (server-only) + Name Helpers ===================
import os, re, pandas as pd

# ---------- Load resolver from WEBSITE only ----------
def _read_mapping_csv(path: str) -> pd.DataFrame:
    encs = ("iso-8859-15", "utf-8-sig", "utf-8", "cp1252", "latin1")
    last = None
    for enc in encs:
        try:
            df = pd.read_csv(path, encoding=enc, dtype=str, keep_default_na=False)
            break
        except Exception as e:
            last = e
            df = None
    if df is None:
        raise RuntimeError(f"Unable to read mapping CSV {path}: {last}")
    if df.shape[1] < 2:
        raise RuntimeError("Mapping CSV must have at least two columns: code, unmasked")

    df = df.iloc[:, :2].copy()
    df.columns = ["code", "unmasked"]
    df["code"] = df["code"].astype(str).str.strip().str.lower()
    df["unmasked"] = df["unmasked"].astype(str).str.strip()
    df = df[df["code"] != ""].drop_duplicates(subset=["code"], keep="first")
    if df.empty:
        raise RuntimeError("Mapping CSV is empty after normalization.")
    return df

def load_resolver_from_server() -> dict:
    """
    Download /partials/match_to_unmasked.csv → local temp; fail if missing.
    Relies on ftp_connect(), _remote_path(), and SERVER_* constants from [1/6].
    """
    with ftp_connect() as ftps:
        ok = ftp_download_if_exists(
            ftps,
            _remote_path(SERVER_MAPPING_REMOTE),
            SERVER_MAPPING_LOCAL_CACHE
        )
        try:
            ftps.quit()
        except Exception:
            pass

    if not ok:
        raise RuntimeError(
            f"Resolver not found on server: /{_remote_path(SERVER_MAPPING_REMOTE)}. "
            f"Upload {SERVER_MAPPING_BASENAME} to /{SERVER_PARTIALS_DIR}/ and re-run."
        )

    df_map = _read_mapping_csv(SERVER_MAPPING_LOCAL_CACHE)
    print(f"✅ Loaded resolver from server: {len(df_map)} codes")
    return dict(zip(df_map["code"], df_map["unmasked"]))  # single source of truth

# Global resolver dict
MATCH_TO_UNMASKED = load_resolver_from_server()

# ---------- Helpers ----------
def find_col(df, patterns, prefer_exact=None):
    """Find a column by regex patterns, preferring exact names if present."""
    cols = list(df.columns)
    lowmap = {c.lower(): c for c in cols}
    if prefer_exact:
        for name in prefer_exact:
            if name in df.columns:
                return name
            if name and name.lower() in lowmap:
                return lowmap[name.lower()]
    for pat in patterns:
        rx = re.compile(pat, re.I)
        for c in cols:
            if rx.search(c):
                return c
    return None

# Split tokens for lineage path (handles →, &rarr;, arrows, commas, pipes, etc.)
SEP_RE = re.compile(r"\s*(?:→|&rarr;|\u2192|;|>|,|~{2,}|/{2,}|\|{2,})\s*")
def split_tokens(s):
    if pd.isna(s): return []
    if not isinstance(s, str): s = str(s)
    return [p.strip() for p in SEP_RE.split(s) if str(p).strip()]

def _clean_piece(text: str) -> str:
    t = re.sub(r'~+', ' ', str(text))
    t = re.sub(r'\s+', ' ', t)
    return t.strip()

_PARTICLES = {"de","del","della","der","van","von","da","dos","das","di","la","le","du","of"}

def _smart_title(token: str) -> str:
    if not token:
        return token
    token = re.sub(
        r"(^|\b)([a-z])(['’])([a-z])",
        lambda m: m.group(1)+m.group(2).upper()+m.group(3)+m.group(4).upper(),
        token.lower()
    )
    token = "-".join([w.capitalize() for w in token.split("-")])
    token = re.sub(r"\bmc([a-z])",  lambda m: "Mc"+m.group(1).upper(),  token)
    token = re.sub(r"\bmac([a-z])", lambda m: "Mac"+m.group(1).upper(), token)
    return token

def smart_titlecase(name: str) -> str:
    name = _clean_piece(name)
    if not name:
        return name
    if "," in name:
        last, first = [p.strip() for p in name.split(",", 1)]
        pieces = (first + " " + last).split()
    else:
        pieces = name.split()
    out = []
    for i, w in enumerate(pieces):
        out.append(w.lower() if (i>0 and w.lower() in _PARTICLES) else _smart_title(w))
    return " ".join(out)

def surname_given_from_token(token):
    token = token.strip()
    if not token:
        return (token,)
    idx = None
    for i in range(1, len(token)):
        if token[i-1].islower() and token[i].isupper():
            idx = i; break
    if idx is None:
        for i in range(1, len(token)):
            if token[i].isupper():
                idx = i; break
    if idx is None:
        return (token,)
    surname = token[:idx]; given = token[idx:]
    given_spaced = re.sub(r'(?<!^)([A-Z])', r' \1', given)
    return (f"{given_spaced.strip()} {surname.strip()}",)

def normalize_person_name(s: str) -> str:
    if pd.isna(s): return ""
    s = _clean_piece(str(s))
    if "," in s:
        last, first = [p.strip() for p in s.split(",", 1)]
        s = f"{first} {last}"
    if " " not in s and s.isalpha():
        return smart_titlecase(surname_given_from_token(s)[0])
    return smart_titlecase(s)

def truncate_first(name: str, n: int = 4) -> str:
    name = name.strip()
    if not name: return name
    parts = name.split()
    return parts[0][:n] if len(parts) == 1 else f"{parts[0][:n]} {parts[-1]}"

def derive_common_from_first_token(tokens):
    if not tokens:
        return ("", "")
    first = _clean_piece(tokens[0])
    parts = re.split(r"\s*(?:&| and )\s*", first, maxsplit=1, flags=re.I)
    if len(parts) != 2:
        return ("", "")
    def _norm(s):
        return smart_titlecase(s) if " " in s else smart_titlecase(surname_given_from_token(s)[0])
    return (_norm(parts[0]), _norm(parts[1]))

def degree_label_from_generations(g):
    if g <= 1:
        return "parents" if g == 1 else "self"
    if g == 2:
        return "grandparents"
    greats = g - 2
    if greats == 1:
        return "great-grandparents"
    return f"{greats}\u00d7-great-grandparents"

def resolve_match_to(code: str) -> str:
    if not isinstance(code, str): return ""
    return MATCH_TO_UNMASKED.get(str(code).strip().lower(), str(code))

# For converting single-token like RosenbalmJessica → "Jessica Rosenbalm"
_CAMEL_WORDS = re.compile(r"[A-Z][a-z]*|[A-Z]+(?![a-z])|[a-z]+")
def _truncate_alpha(s: str, n: int) -> str:
    return re.sub(r"[^A-Za-z]", "", s)[:n]

def norm_matchee_name(raw: str) -> str:
    raw = str(raw or "").strip()
    if not raw:
        return ""
    if " " in raw or "," in raw:
        nm = smart_titlecase(raw)
        parts = nm.split()
        if len(parts) == 1:
            return nm
        given = parts[0]
        surname = parts[-1]
        return f"{_truncate_alpha(given, 7)} {surname}".strip()
    words = _CAMEL_WORDS.findall(raw)
    while words and len(words[0]) == 1:
        words.pop(0)
    if not words:
        nm = smart_titlecase(surname_given_from_token(raw)[0])
        ps = nm.split()
        if len(ps) == 1:
            return nm
        return f"{_truncate_alpha(ps[0], 7)} {ps[-1]}".strip()
    surname = smart_titlecase(words[0])
    given_candidates = [w for w in words[1:] if w.lower() != surname.lower()]
    if not given_candidates:
        return surname
    given = smart_titlecase(given_candidates[0])
    return f"{_truncate_alpha(given, 7)} {surname}".strip()

def extract_person_id(s: str) -> str:
    m = re.search(r"\bI\d+\b", str(s or ""), re.I)
    return m.group(0).upper() if m else ""
# ====== CUT STOP [B] REFACTOR-Gold 3 — Resolver (server-only) + Name Helpers ===================


# ====== CUT START [C] REFACTOR-Gold 3 — CSV Load + Column Detection + Row Helpers ================
import os, re, pandas as pd, urllib.parse as _u

# Use the path defined in [1/6]; default to the expected file
CSV_IN = os.environ.get("CSV_IN", CSV_PATH if 'CSV_PATH' in globals() else "/content/final_combined_df_with_value_labels.csv")

# ---- Load CSV (robust encodings) ----
_encs = ("utf-8-sig", "utf-8", "cp1252", "iso-8859-15", "latin1")
_last_err = None
df = None
for _e in _encs:
    try:
        df = pd.read_csv(CSV_IN, encoding=_e, dtype=str, keep_default_na=False)
        break
    except Exception as _ex:
        _last_err = _ex
        df = None
if df is None:
    raise RuntimeError(f"Unable to read CSV: {CSV_IN} ({_last_err})")
print(f"✅ Loaded CSV — {len(df)} rows, {len(df.columns)} columns from {os.path.abspath(CSV_IN)}")

# ---- Locate columns (names vary slightly across exports) ----
# relies on find_col() from [2/6]
id_col        = find_col(df, [r'^(id#|personid)$'], ["ID#", "ID", "PersonID", "personID"])
match_to_col  = find_col(df, [r'^match\s*to$'], ["Match to","Match"])
name_col      = find_col(df, [r'^name$'], ["Name"])
cm_col        = find_col(df, [r'^(c\s*:?m|cm)$', r'centi.?morgan'], ["cM","cm"])
path_col      = find_col(df, [r'(yates\s*dna\s*ancestral\s*line|ancestral\s*line|lineage)'],
                         ["Yates DNA Ancestral Line","Ancestral Line","Lineage"])

if not id_col:       raise ValueError("CSV missing an ID#/PersonID column.")
if not match_to_col: raise ValueError("CSV missing 'Match to' column.")
if not name_col:     raise ValueError("CSV missing 'Name' column.")
if not cm_col:       raise ValueError("CSV missing 'cM' column.")
if not path_col:     raise ValueError("CSV missing lineage/path column.")

print("✅ Columns:", {"ID": id_col, "Match to": match_to_col, "Name": name_col, "cM": cm_col, "Lineage": path_col})
# ====== CUT STOP [C] REFACTOR-Gold 3 — CSV Load + Column Detection + Row Helpers ================


# ====== CUT START [D] REFACTOR-Gold 3 — Transform & Column A (build display_df) ==================
import urllib.parse as _u

# ---------- Build the three display columns ----------
match_names = []
lineages    = []
findcol     = []

# Allow absolute link to this page: /{REMOTE_NAME}?q=...
REMOTE_NAME_ABS = "/" + (REMOTE_NAME if 'REMOTE_NAME' in globals() else "yates_ancestor_register.htm")

for _, row in df.iterrows():
    # Subject: resolver code in "Match to" → unmasked subject name (used for Find link)
    subject_raw  = row.get(match_to_col, "")
    subject_name = normalize_person_name(resolve_match_to(subject_raw))

    # Matchee: normalized name ONLY (no sentence)
    pid = extract_person_id(row.get(id_col, ""))
    matchee_name = norm_matchee_name(row.get(name_col, "")) or subject_name

    # Lineage path (tokens oldest→youngest); first token is common couple
    tokens      = split_tokens(row.get(path_col, ""))
    tokens_disp = tokens[:7]  # show first 7 pairs
    if tokens_disp:
        tokens_disp[0] = f"<strong>{tokens_disp[0]}</strong>"
    sep = f" {ARROW_ENTITY} "
    lineage_text = sep.join(tokens_disp) if tokens_disp else ""

    # Per-row quick-open search button (?q=Subject Name)
    q = _u.quote(subject_name)
    quick = (f'<a class="find-btn" href="{REMOTE_NAME_ABS}?q={q}" target="_blank" rel="noopener" '
             f'title="Open a filtered view for {subject_name}">Find</a>')

    match_names.append(matchee_name)
    lineages.append(lineage_text)
    findcol.append(quick)

# Column names
MATCH_NAME_COL = "Match Name"
LINEAGE_HEADER_SAFE = LINEAGE_HEADER if 'LINEAGE_HEADER' in globals() else "Lineage (Starting with oldest ancestor, the line is:)"

# Assemble the display dataframe (GUARDED so “Find” always exists)
df[MATCH_NAME_COL]              = match_names
df[LINEAGE_HEADER_SAFE]         = lineages
df["Find"]                      = findcol  # guarantees presence for [5/6]
display_df = df[["Find", MATCH_NAME_COL, LINEAGE_HEADER_SAFE]]

# Column A CSV for side-uses (now just the match names)
display_df[[MATCH_NAME_COL]].to_csv(MATCH_COUSINS_CSV, index=False, encoding="iso-8859-15")
print("✅ Wrote local CSV (Column A):", os.path.abspath(MATCH_COUSINS_CSV))
# ====== CUT STOP [D] REFACTOR-Gold 3 — Transform & Column A (build display_df) ==================
#
#
#
#
#
# ====== CUT START [E] REFACTOR-Gold 3 — HTML (table + CSS + JS, no uploads) ======================
import html as _html

# Use this as the Home link everywhere
HOME_URL = "https://yates.one-name.net/yates_ancestor_register.htm"

# Make the “Match To Name” column narrower (tweak as needed)
COL_A_PX = 160

# ---------- Build/attach the “Match To Name” column (resolver-based) ----------
# Assumes df, display_df, match_to_col, resolve_match_to(), normalize_person_name() exist from prior sections.
_match_to_names = []
for _, _row in df.iterrows():
    _raw = _row.get(match_to_col, "")
    _nm  = normalize_person_name(resolve_match_to(_raw))
    _match_to_names.append(_nm)

# Important: persist the new column on display_df so later sections (partials/print) can use it.
display_df = display_df.copy()
display_df["Match To Name"] = _match_to_names

# ---------- Build HTML table ----------
LINEAGE_HEADER_SAFE = LINEAGE_HEADER if 'LINEAGE_HEADER' in globals() else "Lineage (Starting with oldest ancestor, the line is:)"
display_for_html = display_df

html_table = display_for_html[["Find", "Match To Name", LINEAGE_HEADER_SAFE]].to_html(
    index=False, escape=False, classes="sortable"
)

# tag table + first row
html_table = html_table.replace(
    '<table border="1" class="dataframe sortable">',
    '<table border="1" class="dataframe sortable" id="refactor-table">', 1
)
html_table = html_table.replace('<tbody>\n<tr>', '<tbody>\n<tr id="first-row">', 1)

# widths via <colgroup>
FIND_PX = 110  # width of Find column (checkbox + Email + Find)
colgroup_html = (
    "<colgroup>\n"
    f"  <col style=\"width:{FIND_PX}px;\" />\n"
    f"  <col style=\"width:{COL_A_PX}px;\" />\n"
    "  <col />\n"
    "</colgroup>\n"
)
html_table = html_table.replace(
    '<table border="1" class="dataframe sortable" id="refactor-table">',
    '<table border="1" class="dataframe sortable" id="refactor-table">\n' + colgroup_html, 1
)

# add Select-all checkbox to header cell “Find”
html_table = html_table.replace(
    "<th>Find</th>",
    "<th>Find&nbsp;<input type=\"checkbox\" id=\"sel-all\" title=\"Select all visible\" /></th>",
    1
)

html_table_scrolling = '<div class="table-scroll">\n' + html_table + '\n</div>'

# ---------- CSS ----------
TABLE_CSS = (
    "<style type=\"text/css\">\n"
    "  html { scroll-behavior: smooth; }\n"
    "  body { font-family: Georgia, 'Times New Roman', serif; background:#ffffff; color:#222; margin:0; padding:0; line-height:1.5; }\n"
    f"  .wrap {{ max-width:{TABLE_WIDTH_PX}px; margin:0 auto; background:#ffffff; padding:20px; padding-bottom:48px; }}\n"
    "  a { color:#154b8b; text-decoration:none; } a:hover { text-decoration:underline; }\n"
    "  h1 { margin:0 0 6px 0; font-size:26px; line-height:1.2; text-align:center; }\n"
    "  .updated { font-size:12px; color:#555; text-align:center; margin:2px 0 10px 0; }\n"
    "  .sortbar { margin:6px 0 10px 0; font-size:13px; background:#ffffff; padding:6px 8px; border-radius:6px;\n"
    "             display:flex; flex-wrap:wrap; gap:5px; align-items:center; border:1px solid #ddd; }\n"
    "  .btn { display:inline-block; border:1px solid #5b79b8; background:#5b79b8; color:#fff; padding:4px 9px;\n"
    "         text-decoration:none; cursor:pointer; border-radius:5px; line-height:1.2; transition:background 0.2s, transform 0.1s; user-select:none; }\n"
    "  .btn:hover { background:#4668aa; transform:translateY(-1px); }\n"
    "  input.btn.search { background:#fff; color:#111; border-color:#bbb; }\n"
    "  .btn-mini { font-size:12px; padding:2px 6px; line-height:1.1; margin-left:6px; }\n"
    "  .find-cell { white-space:nowrap; }\n"
    "  .selbox { margin-right:6px; vertical-align:middle; }\n"
    "  .table-scroll { max-height:70vh; overflow-y:auto; overflow-x:auto; border:1px solid #ddd; }\n"
    f"  table.sortable {{ border-collapse:collapse; width:{TABLE_WIDTH_PX}px; table-layout:fixed; }}\n"
    "  table.sortable th, table.sortable td { border:1px solid #ddd; padding:6px 8px; vertical-align:top; }\n"
    "  table.sortable th { background:#e3eaf8; text-align:left; position:sticky; top:0; z-index:2; box-shadow:0 1px 0 #ccc; }\n"
    "  table.sortable td { word-wrap:break-word; overflow-wrap:break-word; }\n"
    "  #first-row td { border-top:2px solid #999; }\n"
    "  .back-to-top { position:fixed; right:16px; bottom:16px; padding:6px 10px; border:1px solid #3e5a97; background:#5b79b8; color:#fff;\n"
    "                 cursor:pointer; border-radius:6px; font-size:12px; display:none; z-index:9999; }\n"
    "  .back-to-top:hover { background:#4668aa; }\n"
    "  #dynamicContent { margin:10px 0 14px 0; }\n"
    "  @media screen and (max-width: 820px) { .wrap { padding:12px; } h1 { font-size:22px; } }\n"
    "</style>\n"
)

# ---------- Toolbar ----------
DYNAMIC_BLOCK = (
    "<div class=\"sortbar\">\n"
    "  <a class=\"btn\" href=\"https://yates.one-name.net/gengen/dna_cousin_surname_study.htm\" target=\"_blank\">Study Details</a>\n"
    "  <a class=\"btn\" href=\"https://yates.one-name.net/gengen/dna_theory_of_the_case.htm\" target=\"_blank\">Theory in Action</a>\n"
    "  <span class=\"btn\" data-sort-col=\"1\" data-sort-dir=\"asc\">Sort Match &uarr;</span>\n"
    "  <span class=\"btn\" data-sort-col=\"1\" data-sort-dir=\"desc\">Sort Match &darr;</span>\n"
    "  <span class=\"btn\" data-sort-col=\"2\" data-sort-dir=\"asc\">Sort Lineage &uarr;</span>\n"
    "  <span class=\"btn\" data-sort-col=\"2\" data-sort-dir=\"desc\">Sort Lineage &darr;</span>\n"
    "  <a class=\"btn\" href=\"gengen/images/cousin-calculator.jpg\" target=\"_blank\">Cousin Connection</a>\n"
    "  <a class=\"btn\" href=\"gengen/images/Shared_cM_Project_v4.jpg\" target=\"_blank\">Cousin by DNA</a>\n"
    "  <a class=\"btn\" href=\"partials/match_count.htm\" target=\"_blank\" rel=\"noopener\">Match Count</a>\n"
    "  <a class=\"btn\" href=\"partials/lineage_count.htm\" target=\"_blank\" rel=\"noopener\">Lineage Count</a>\n"
    "  <a class=\"btn\" href=\"/partials/cousin_list_print.htm\" target=\"_blank\">Cousin List</a>\n"
    "  <span class=\"btn\" id=\"email-selected\">Email Selected</span>\n"
    "  <span class=\"btn\" id=\"clear-selected\">Clear</span>\n"
    "  <input type=\"text\" id=\"search-box\" class=\"btn search\" size=\"24\" value=\"\" placeholder=\"Search&hellip;\" "
    "         autocomplete=\"off\" autocapitalize=\"off\" spellcheck=\"false\" inputmode=\"search\" enterkeyhint=\"search\" />\n"
    "</div>\n"
    "<div id=\"dynamicContent\"></div>\n"
)

# ---------- Full HTML (string only) ----------
JS_COUNT_URL = COUNT_PUBLIC_URL.replace("'", "%27")

UPDATED_BLOCK = (
    "<div class=\"updated\">"
    f"<a href=\"{HOME_URL}\" target=\"_blank\" rel=\"noopener\">Home</a>"
    " &nbsp;|&nbsp; Last updated: <span id=\"last-updated\"></span>"
    " &nbsp;|&nbsp; Autosomal matches: <span id=\"auto-count\" class=\"js-count\"></span>"
    "</div>"
)

TEMPLATE_HTML = """<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>ONS Yates Study Ancestor Register</title>
{TABLE_CSS}
</head>
<body id="top">
<div class="wrap">
  <h1>ONS Yates Study Ancestor Register</h1>
  {UPDATED_BLOCK}
  {DYNAMIC_BLOCK}
{HTML_TABLE_SCROLLING}
</div>
<button id="back-to-top" class="back-to-top">&#9650; Top</button>
<script type="text/javascript">
//<![CDATA[
(function(){
  function textOf(cell){ return (cell && (cell.textContent || cell.innerText) || '').replace(/\\s+/g,' ').trim().toLowerCase(); }
  function sortTable(tbl, colIndex, dir){
    var tb=tbl && tbl.tBodies ? tbl.tBodies[0] : null; if(!tb) return;
    var rows=[].slice.call(tb.rows||[]);
    var asc=(dir==='asc');
    rows.sort(function(a,b){
      var A=textOf(a.cells[colIndex]), B=textOf(b.cells[colIndex]);
      if(A<B) return asc?-1:1; if(A>B) return asc?1:-1; return 0;
    });
    var frag=document.createDocumentFragment();
    for(var i=0;i<rows.length;i++) frag.appendChild(rows[i]);
    tb.appendChild(frag);
    updateSelAll(); // keep master checkbox in sync after sort
  }
  function bindSortButtons(){
    var tbl=document.getElementById('refactor-table'); var bar=document.querySelector('.sortbar'); if(!(tbl && bar)) return;
    bar.addEventListener('click',function(e){
      var btn=e.target && e.target.closest ? e.target.closest('.btn') : null; if(!btn) return;
      var colAttr=btn.getAttribute('data-sort-col'); if(colAttr==null) return; var col=parseInt(colAttr,10); if(isNaN(col)) return;
      var dir=btn.getAttribute('data-sort-dir')||'asc'; sortTable(tbl,col,dir); e.preventDefault(); return false;
    },false);
  }
  function bindHeaderSort(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tHead && tbl.tHead.rows.length)) return; var ths=tbl.tHead.rows[0].cells;
    for(var i=0;i<ths.length;i++)(function(idx){
      var dir='asc'; ths[idx].addEventListener('click',function(){ dir=(dir==='asc')?'desc':'asc'; sortTable(tbl,idx,dir); },false);
    })(i);
  }

  // partials
  var PARTIAL_BASES=['/partials/','partials/','gengen/partials/','/gengen/partials/'];
  function tryFetchSequential(urls,onOK,onFail){
    if(!urls.length) return onFail('No valid locations'); var url=urls.shift();
    fetch(url,{cache:'no-store'}).then(function(r){ if(!r.ok) throw new Error('HTTP '+r.status); return r.text();})
      .then(onOK).catch(function(){ tryFetchSequential(urls,onOK,onFail);});
  }
  function bindPartials(){
    var bar=document.querySelector('.sortbar'); if(!bar) return;
    bar.addEventListener('click',function(e){
      var btn=e.target && e.target.closest ? e.target.closest('.btn') : null; if(!btn) return;
      var rel=btn.getAttribute('data-load-partial'); if(!rel) return;
      var c=document.getElementById('dynamicContent'); if(!c) return; c.innerHTML='<p><em>Loading latest data&hellip;</em></p>';
      var bust=encodeURIComponent(document.lastModified||(new Date()).toUTCString());
      var bases=PARTIAL_BASES.slice(); var candidates=bases.map(function(b){return b+rel+'?v='+bust;});
      tryFetchSequential(candidates.slice(), function(html){ c.innerHTML=html; }, function(){ c.innerHTML='<p style=\\\"color:#a00;\\\">Could not load content.</p>'; });
    });
  }

  // updated stamp + count
  function stampLastUpdated(){
    var el=document.getElementById('last-updated'); if(!el) return;
    var d=new Date(document.lastModified||new Date());
    function z(n){return(n<10?'0':'')+n;}
    el.innerHTML=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());
  }
  function formatWithCommas(n){
    try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US'); }catch(e){ return String(n||''); }
  }
  function loadAutoCount(){
    var el=document.getElementById('auto-count'); if(!el) return;
    var url='{COUNT_URL}';
    try{
      var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);
      xhr.onreadystatechange=function(){ if(xhr.readyState===4){ if(xhr.status>=200&&xhr.status<300){
        var m=(xhr.responseText||'').match(/(\\d+)/); var num=m?m[1]:'';
        el.textContent = formatWithCommas(num) || '(unavailable)';
      } else { el.textContent='(unavailable)'; } } };
      xhr.send(null);
    }catch(e){ el.textContent='(unavailable)'; }
  }

  // search (filters rows; select-all respects visibility)
  function getParam(name){ var m=location.search.match(new RegExp('[?&]'+name+'=([^&]+)')); return m?decodeURIComponent(m[1].replace(/\\+/g,' ')):''; }
  function bindSearch(){
    var box=document.getElementById('search-box'); var tbl=document.getElementById('refactor-table'); if(!(box && tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb=tbl.tBodies[0];
    function norm(s){ return String(s||'').replace(/\\s+/g,' ').toLowerCase(); }
    function rowText(tr){ var t=''; for(var i=0;i<tr.cells.length;i++){ t+= ' ' + (tr.cells[i].textContent||tr.cells[i].innerText||''); } return norm(t); }
    var cached=[]; (function seed(){ var rows=tb.rows; cached=[]; for(var i=0;i<rows.length;i++){ cached.push({el:rows[i], txt:rowText(rows[i])}); } })();
    function apply(q){
      q=norm(q);
      for(var i=0;i<cached.length;i++){
        var hit = !q || cached[i].txt.indexOf(q)>-1;
        cached[i].el.style.display = hit? '' : 'none';
      }
      updateSelAll();
    }
    var to=null; function onInput(){ if(to) clearTimeout(to); to=setTimeout(function(){ apply(box.value); }, 60); }
    box.addEventListener('input', onInput, false);
    var q0=getParam('q');
    if(q0){ box.value=q0; apply(q0); try{ history.replaceState(null, '', location.pathname); }catch(e){} }
    else { box.value=''; apply(''); setTimeout(function(){ if(!getParam('q')){ box.value=''; apply(''); } }, 0); }
  }

  // --- selection controls ---
  function visibleRowCheckboxes(){
    var tbl=document.getElementById('refactor-table');
    if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return [];
    var tb=tbl.tBodies[0], out=[];
    for(var i=0;i<tb.rows.length;i++){
      var tr=tb.rows[i];
      if(tr.style.display==='none') continue;
      var cb = tr.cells[0] && tr.cells[0].querySelector('.selbox');
      if(cb) out.push(cb);
    }
    return out;
  }
  function updateSelAll(){
    var master=document.getElementById('sel-all'); if(!master) return;
    var cbs=visibleRowCheckboxes();
    if(!cbs.length){ master.indeterminate=false; master.checked=false; return; }
    var checked=cbs.filter(function(cb){ return cb.checked; }).length;
    master.checked = (checked===cbs.length);
    master.indeterminate = (checked>0 && checked<cbs.length);
  }
  function bindSelectAll(){
    var master=document.getElementById('sel-all'); if(!master) return;
    master.addEventListener('change', function(){
      var want=master.checked;
      var cbs=visibleRowCheckboxes();
      for(var i=0;i<cbs.length;i++) cbs[i].checked = want;
      updateSelAll();
    });
  }
  function pageBase(){ try{ return location.origin + location.pathname; }catch(e){ return location.pathname; } }
  function encodeQS(s){ return encodeURIComponent(String(s||'')); }
  function buildRowEmail(tr){
    var cells = tr.cells; if(!cells || cells.length < 3) return null;
    var summary = (cells[1].textContent||'').trim();
    var lineage = (cells[2].textContent||'').trim();
    var subj = 'ONS Yates Register: ' + summary.substring(0, 90);
    var qMatch = (function(){ try{ var em = cells[1].querySelector('strong'); return (em?em.textContent:'').trim(); }catch(e){ return ''; } })();
    var link = pageBase() + (qMatch? ('?q='+encodeQS(qMatch)) : '');
    var body = summary + '\\n\\nLineage:\\n' + lineage + '\\n\\nLink: ' + link;
    return {subject: subj, body: body};
  }
  function openMailto(subject, body){
    var href = 'mailto:?subject=' + encodeQS(subject) + '&body=' + encodeQS(body);
    window.location.href = href;
  }
  function injectRowControls(){
    var tbl = document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb = tbl.tBodies[0];
    for(var i=0;i<tb.rows.length;i++){
      (function(tr){
        var cell = tr.cells[0]; if(!cell) return; if(cell.className.indexOf('find-cell')===-1) cell.className += ' find-cell';
        var existingFind = cell.querySelector('a.find-btn'); // preserve your “Find” link
        var cb = document.createElement('input'); cb.type='checkbox'; cb.className='selbox'; cb.title='Select row';
        cb.addEventListener('change', updateSelAll);
        cell.insertBefore(cb, existingFind || cell.firstChild);
        // optional per-row Email button
        var emailBtn = document.createElement('a'); emailBtn.href='#'; emailBtn.className='btn btn-mini'; emailBtn.textContent='Email'; emailBtn.title='Email this row'; emailBtn.style.marginLeft='6px';
        emailBtn.addEventListener('click', function(ev){ ev.preventDefault(); var payload = buildRowEmail(tr); if(payload){ openMailto(payload.subject, payload.body); } });
        if(existingFind){ existingFind.insertAdjacentElement('afterend', emailBtn); } else { cell.appendChild(emailBtn); }
      })(tb.rows[i]);
    }
  }
  function bindBulkEmail(){
    var btn = document.getElementById('email-selected'); if(!btn) return;
    btn.addEventListener('click', function(){
      var tbl = document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb = tbl.tBodies[0];
      var parts = []; var count = 0;
      for(var i=0;i<tb.rows.length;i++){
        var tr = tb.rows[i]; if(tr.style.display==='none') continue;
        var cb = tr.cells[0] && tr.cells[0].querySelector('.selbox');
        if(cb && cb.checked){ var p = buildRowEmail(tr); if(p){ parts.push(p.body); count++; } }
      }
      if(!count){ alert('No rows selected. Tick the checkboxes in the Find column.'); return; }
      var subject = 'ONS Yates Register: ' + count + ' selection' + (count>1?'s':'');
      var body = parts.join('\\n\\n---\\n\\n');
      openMailto(subject, body);
    });
    var clr = document.getElementById('clear-selected'); if(clr){ clr.addEventListener('click', function(){
      var tbl = document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb = tbl.tBodies[0];
      for(var i=0;i<tb.rows.length;i++){ var cb = tb.rows[i].cells[0] && tb.rows[i].querySelector('.selbox'); if(cb) cb.checked=false; }
      updateSelAll();
    }); }
  }

  // misc UI
  function setupBackToTop(){
    var btt=document.getElementById('back-to-top'); var cont=document.querySelector('.table-scroll');
    function onAny(){ var y=(window.scrollY||window.pageYOffset||0); var cy=cont?cont.scrollTop:0; btt.style.display=(y>200||cy>200)?'block':'none'; }
    window.addEventListener('scroll',onAny,{passive:true}); if(cont) cont.addEventListener('scroll',onAny,{passive:true});
    onAny(); if(btt){ btt.addEventListener('click',function(){ if(cont) cont.scrollTo({top:0,behavior:'smooth'}); window.scrollTo({top:0,behavior:'smooth'}); }); }
  }

  // init
  function init(){
    bindSortButtons(); bindHeaderSort(); bindPartials(); stampLastUpdated(); loadAutoCount();
    setupBackToTop(); bindSearch();
    injectRowControls(); bindSelectAll(); bindBulkEmail(); updateSelAll();
  }
  init();
})();
//]]>
</script>
</body></html>
"""

FULL_HTML = (TEMPLATE_HTML
    .replace("{TABLE_CSS}", TABLE_CSS)
    .replace("{UPDATED_BLOCK}", UPDATED_BLOCK)
    .replace("{DYNAMIC_BLOCK}", DYNAMIC_BLOCK)
    .replace("{HTML_TABLE_SCROLLING}", html_table_scrolling)
    .replace("{COUNT_URL}", JS_COUNT_URL)
)
# ====== CUT STOP [E] REFACTOR-Gold 3 — HTML (table + CSS + JS, no uploads) ======================
#
#
#
#
#
# ====== CUT START [F] REFACTOR-Gold 3 — Save + Upload + Reports + Partials + Printable ===========
import os, re, posixpath
from collections import Counter
import pandas as pd

ENC          = "iso-8859-15"
LOCAL_NAME   = globals().get("LOCAL_NAME",  "yates_ancestor_register.htm")
REMOTE_NAME  = globals().get("REMOTE_NAME", "yates_ancestor_register.htm")
HOME_URL     = "https://yates.one-name.net/yates_ancestor_register.htm"

# Safely embed the JS count URL (escape single quotes)
JS_COUNT_URL = COUNT_PUBLIC_URL.replace("'", "%27")

# 1) Write main page HTML exactly as built upstream
with open(LOCAL_NAME, "w", encoding=ENC, errors="xmlcharrefreplace") as f:
    f.write(FULL_HTML)
print(f"✅ Wrote HTML: {os.path.abspath(LOCAL_NAME)}")

# 2) Upload main page + optional assets
with ftp_connect() as ftps:
    try:
        ftps.mkd("partials")
    except Exception:
        pass
    try:
        ftps.delete(_remote_path(REMOTE_NAME))
    except Exception:
        pass

    ftp_upload_overwrite(ftps, LOCAL_NAME, _remote_path(REMOTE_NAME))

    if os.path.exists(MATCH_COUSINS_CSV):
        ftp_upload_overwrite(ftps, MATCH_COUSINS_CSV, _remote_path(MATCH_COUSINS_CSV))

    if os.path.exists(LOCAL_COUNT_FILE):
        ftp_upload_overwrite(ftps, LOCAL_COUNT_FILE, _remote_path(REMOTE_COUNT_NAME))
        print(f"✅ Published autosomal count: {LOCAL_COUNT_FILE} → {COUNT_PUBLIC_URL}")
    else:
        print(f"⚠️ Count file not found locally: {LOCAL_COUNT_FILE}")

    try:
        ftps.quit()
    except Exception:
        pass

# 3) Build data for reports/partials
display_df_all = display_df.copy()
display_df_filtered = display_df_all.reset_index(drop=True)
filtered_df_for_reports = df.loc[display_df_all.index].copy()

# ---------- Resolver usage report ----------
used_series = filtered_df_for_reports[subject_code_col].astype(str).map(lambda x: str(x).strip().lower())
counts = Counter([c for c in used_series if c and c != "nan"])

rows = []
all_keys = set(globals().get("MATCH_TO_UNMASKED", {}).keys())
for code in sorted(all_keys):
    rows.append((code, globals()["MATCH_TO_UNMASKED"].get(code, ""), counts.get(code, 0)))
for code in sorted(set(counts.keys()) - all_keys):
    rows.append((code, "(unmapped)", counts.get(code, 0)))

usage_df = pd.DataFrame(rows, columns=["Match to (code)", "Unmasked", "Count"]).sort_values(
    ["Match to (code)"], ascending=[True]
)
usage_df_alpha = usage_df.reset_index(drop=True)

RESOLVER_USAGE_CSV = "resolver_usage_report.csv"
usage_df_alpha.to_csv(RESOLVER_USAGE_CSV, index=False, encoding=ENC)
print("✅ Wrote resolver usage CSV:", os.path.abspath(RESOLVER_USAGE_CSV))

# ---------- Lineage Count (by oldest common pair, normalized) ----------
def _escape_html(s: str) -> str:
    return (str(s).replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;"))

def _norm_key_component(fullname: str) -> str:
    nm = smart_titlecase(str(fullname)); parts = nm.split()
    if not parts: return ""
    last   = re.sub(r"[^A-Za-z]", "", parts[-1])
    firsts = re.sub(r"[^A-Za-z]", "", "".join(parts[:-1]))[:7]
    return f"{last}{firsts}"

def oldest_pair_key(token_list):
    if not token_list: return ""
    first = token_list[0]
    a, b = derive_common_from_first_token([first])
    if not a and not b:
        parts = re.split(r"\s*&\s*", first)
        a = parts[0] if parts else ""
        b = parts[1] if len(parts) > 1 else ""
    ka = _norm_key_component(a) if a else ""
    kb = _norm_key_component(b) if b else ""
    return f"{ka}&{kb}" if ka or kb else ""

lc_counter = Counter()
for _, row in filtered_df_for_reports.iterrows():
    toks = split_tokens(row[path_col])
    key = oldest_pair_key(toks)
    if key: lc_counter[key] += 1

lc_rows = sorted(lc_counter.items(), key=lambda x: (-x[1], x[0]))
lineage_count_df = pd.DataFrame(lc_rows, columns=["Oldest Distant Ancestor (normalized Last+First7) pair", "Count"])

LINEAGE_COUNT_CSV = "lineage_count_report.csv"
lineage_count_df.to_csv(LINEAGE_COUNT_CSV, index=False, encoding=ENC)
print("✅ Wrote lineage count CSV:", os.path.abspath(LINEAGE_COUNT_CSV))

# Upload report CSVs
try:
    with ftp_connect() as ftps:
        ftp_upload_overwrite(ftps, RESOLVER_USAGE_CSV, _remote_path(RESOLVER_USAGE_CSV))
        ftp_upload_overwrite(ftps, LINEAGE_COUNT_CSV, _remote_path(LINEAGE_COUNT_CSV))
        try: ftps.quit()
        except Exception: pass
    print("✅ Published report CSVs.")
except Exception as e:
    print(f"⚠️ Upload of report CSVs failed: {e}")

# ---------- Helpers for Match Count partial ----------
def _build_person_details_html(code_lc: str) -> str:
    mask = filtered_df_for_reports[subject_code_col].astype(str).str.strip().str.lower() == code_lc
    if not mask.any(): return "<em>No rows for this person.</em>"
    rows_idx = filtered_df_for_reports[mask].index
    mini = display_df.loc[rows_idx, ["Find", "Match To Name", LINEAGE_HEADER_SAFE]].copy()
    return mini.to_html(index=False, escape=False, classes="mini-table", border=0)

# ---------- Match Count partial ----------
def _render_match_count_full_html(usage_alpha: pd.DataFrame) -> str:
    try:
        total_persons = int(usage_alpha.shape[0])
        total_matches = int(pd.to_numeric(usage_alpha["Count"], errors="coerce").fillna(0).sum())
    except Exception:
        total_persons, total_matches = usage_alpha.shape[0], 0

    rows_html = []
    for _, r in usage_alpha.iterrows():
        code    = str(r["Match to (code)"])
        code_lc = code.strip().lower()
        unm     = _escape_html(r["Unmasked"])
        cnt     = int(pd.to_numeric(r["Count"], errors="coerce")) if str(r["Count"]).strip() else 0
        rows_html.append(
            "<tr class='top-row' data-code='{code_lc}'>"
            "<td class='sel'><input type='checkbox' class='selbox' data-code='{code_lc}' title='Select' /></td>"
            "<td class='code'>{code}</td>"
            "<td class='unmasked'>{unm}</td>"
            "<td class='count'><a href='#' class='count-toggle' data-code='{code_lc}' aria-expanded='false'>{cnt}</a></td>"
            "</tr>".format(code_lc=_escape_html(code_lc), code=_escape_html(code), unm=unm, cnt=f"{cnt:,}")
        )
        rows_html.append(
            "<tr class='details-row' id='details-{code_lc}' style='display:none;'>"
            "<td colspan='4'><div class='details-outer'>{detail}</div></td>"
            "</tr>".format(code_lc=_escape_html(code_lc), detail=_build_person_details_html(code_lc))
        )

    style = (
        "<style>"
        " body{font-family:Georgia,'Times New Roman',serif;background:#fff;color:#111;margin:0;padding:18px;line-height:1.6;font-size:15px}"
        " .wrap{max-width:900px;margin:0 auto}"
        " h2{font-size:20px;margin:0 0 10px 0;text-align:center;border-bottom:2px solid #5b79b8;padding-bottom:6px}"
        " .updated{font-size:12px;color:#555;text-align:center;margin:2px 0 10px 0}"
        " .stats{text-align:center;color:#444;font-size:13px;margin:6px 0 10px 0}"
        " .toolbar{display:flex;flex-wrap:wrap;gap:8px;align-items:center;justify-content:flex-start;margin:8px 0 10px 0}"
        " .btn{display:inline-block;border:1px solid #5b79b8;background:#5b79b8;color:#fff;padding:6px 12px;border-radius:6px;text-decoration:none;cursor:pointer}"
        " .btn:hover{background:#4668aa}"
        " .master{margin-left:auto;font-size:13px;color:#333}"
        " table#mc-table{border-collapse:collapse;width:100%;table-layout:fixed}"
        " #mc-table col.sel{width:72px} #mc-table col.code{width:26%} #mc-table col.unmasked{width:48%} #mc-table col.count{width:16%}"
        " #mc-table th,#mc-table td{border:1px solid #ccc;padding:8px 10px;vertical-align:top;word-wrap:break-word;overflow-wrap:break-word}"
        " #mc-table th{background:#e9eef9;position:sticky;top:0;z-index:1}"
        " #mc-table td.count{text-align:right}"
        " #mc-table a.count-toggle{display:inline-block;padding:2px 6px;background:#5b79b8;color:#fff;border-radius:4px;border:1px solid #3e5a97;text-decoration:none}"
        " #mc-table a.count-toggle:hover{background:#4668aa}"
        " .details-outer{background:#f9fbff;border:1px solid #d7e1fb;margin:8px 2px;padding:8px;border-radius:6px}"
        " .mini-table{border-collapse:collapse;width:100%;table-layout:fixed}"
        " .mini-table th,.mini-table td{border:1px solid #ddd;padding:6px 8px;vertical-align:top}"
        " .mini-table th{background:#f2f6ff}"
        " .mini-table col:nth-child(1){width:140px} "
        f" .mini-table col:nth-child(2){{width:{COL_A_PX}px}} "
        " .mini-table col:nth-child(3){width:auto}"
        " @media screen and (max-width:900px){ body{padding:12px;font-size:14px} .toolbar{gap:6px} }"
        "</style>"
    )

    updated = (
        "<div class='updated'>"
        "<a href='" + HOME_URL + "' target='_blank' rel='noopener'>Home</a>"
        " &nbsp;|&nbsp; Last updated: <span id='last-updated'></span>"
        " &nbsp;|&nbsp; Autosomal matches: <span id='auto-count' class='js-count'></span>"
        "</div>"
    )

    toolbar = (
        "<div class='toolbar'>"
        " <button type='button' class='btn' id='btn-display'>Display Checked</button>"
        " <button type='button' class='btn' id='btn-email'>Email Checked</button>"
        " <button type='button' class='btn' id='btn-print'>Print Checked</button>"
        " <label class='master'><input type='checkbox' id='sel-all' /> Select all</label>"
        "</div>"
    )

    html = (
        "<!DOCTYPE html><html lang='en'><head>"
        "<meta charset='iso-8859-15' />"
        "<meta name='viewport' content='width=device-width, initial-scale=1.0' />"
        "<title>Match Count (by resolver code)</title>"
        + style +
        "</head><body>"
        "<div class='wrap'>"
        "<h2>Match Count (by resolver code, A&rightarrow;Z)</h2>"
        + updated +
        "<div class='stats'>Total distinct &ldquo;Match to&rdquo; persons: " + f"{total_persons:,}" +
        " &nbsp;|&nbsp; Total matches counted: " + f"{total_matches:,}" + "</div>"
        + toolbar +
        "<table id='mc-table'>"
        "<colgroup><col class='sel'><col class='code'><col class='unmasked'><col class='count'></colgroup>"
        "<thead><tr><th>Select</th><th>Match to (code)</th><th>Unmasked</th><th>Count</th></tr></thead>"
        "<tbody>" + "".join(rows_html) + "</tbody></table>"
        "</div>"
        "<script>(function(){"
        " document.addEventListener('DOMContentLoaded', function(){"
        "  function z(n){return(n<10?'0':'')+n;}"
        "  function stamp(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date());"
        "   el.textContent=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());}"
        "  function fmt(n){try{var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US');}catch(e){return String(n||'');}}"
        "  function loadCnt(){var el=document.getElementById('auto-count'); if(!el) return; var url='" + JS_COUNT_URL + "';"
        "   try{var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);"
        "    xhr.onreadystatechange=function(){ if(xhr.readyState===4){ if(xhr.status>=200&&xhr.status<300){ var m=(xhr.responseText||'').match(/(\\d+)/); el.textContent = fmt(m?m[1]:'') || '(unavailable)'; } else { el.textContent='(unavailable)'; } } };"
        "    xhr.send(null);}catch(e){ el.textContent='(unavailable)'; }}"
        "  stamp(); loadCnt();"

        "  function toggleDetails(code, wantOpen){"
        "    var row=document.getElementById('details-'+code); if(!row) return;"
        "    row.style.display = wantOpen ? '' : 'none';"
        "    var a=document.querySelector(\"a.count-toggle[data-code='\"+code+\"']\"); if(a){ a.setAttribute('aria-expanded', wantOpen?'true':'false'); }"
        "  }"
        "  function selectedCodes(){"
        "    var out=[]; var cbs=document.querySelectorAll('#mc-table .selbox');"
        "    for(var i=0;i<cbs.length;i++){ if(cbs[i].checked){ var c=cbs[i].getAttribute('data-code'); if(c) out.push(c); }}"
        "    return out;"
        "  }"

        "  var master=document.getElementById('sel-all');"
        "  if(master){ master.addEventListener('change', function(){"
        "    var want=master.checked; var cbs=document.querySelectorAll('#mc-table .selbox');"
        "    for(var i=0;i<cbs.length;i++){ cbs[i].checked = want; }"
        "  }); }"

        "  var btnDisp=document.getElementById('btn-display');"
        "  if(btnDisp){ btnDisp.addEventListener('click', function(){"
        "    var sel=selectedCodes();"
        "    var all=document.querySelectorAll('tr.details-row'); for(var i=0;i<all.length;i++){ all[i].style.display='none'; }"
        "    for(var j=0;j<sel.length;j++){ toggleDetails(sel[j], true); }"
        "    if(sel.length){ try{ document.getElementById('details-'+sel[0]).scrollIntoView({behavior:'smooth',block:'start'});}catch(e){} }"
        "  }); }"

        "  function enc(s){ return encodeURIComponent(String(s||'')); }"
        "  function textOf(el){ return (el && (el.textContent||el.innerText)||'').trim(); }"

        "  var btnEmail=document.getElementById('btn-email');"
        "  if(btnEmail){ btnEmail.addEventListener('click', function(){"
        "    var sel=selectedCodes(); if(!sel.length){ alert('No persons selected.'); return; }"
        "    function collectMatches(code){"
        "      var out=[]; var host=document.getElementById('details-'+code); if(!host) return out;"
        "      var rows=host.querySelectorAll('table.mini-table tbody tr');"
        "      for(var i=0;i<rows.length;i++){"
        "        var tds=rows[i].cells; if(!tds || tds.length<3) continue;"
        "        var a=tds[0].querySelector('a'); var href=a ? a.href : '';"
        "        var summary=textOf(tds[1]);"
        "        out.push({summary:summary, href:href});"
        "      }"
        "      return out;"
        "    }"
        "    var lines=[], total=0;"
        "    for(var k=0;k<sel.length;k++){"
        "      var code=sel[k];"
        "      var tr=document.querySelector(\"tr.top-row[data-code='\"+code+\"']\"); if(!tr) continue;"
        "      var codeTxt=textOf(tr.querySelector('.code'));"
        "      var unmTxt =textOf(tr.querySelector('.unmasked'));"
        "      var cntTxt =textOf(tr.querySelector('.count a'));"
        "      var header = codeTxt + (unmTxt? ' — '+unmTxt : '') + (cntTxt? ' ('+cntTxt+')' : '');"
        "      lines.push('== ' + header + ' ==');"
        "      var matches = collectMatches(code);"
        "      if(!matches.length){"
        "        lines.push('- (no inline rows available)');"
        "      } else {"
        "        total += matches.length;"
        "        for(var m=0;m<matches.length;m++){"
        "          var row = matches[m];"
        "          lines.push('- ' + row.summary);"
        "          if(row.href){ lines.push(row.href); }"
        "        }"
        "      }"
        "      if(k < sel.length-1) lines.push('');"
        "    }"
        "    var subj='ONS Yates: ' + sel.length + ' person(s), ' + total + ' match row(s)';"
        "    var body=lines.join('\\n');"
        "    var href='mailto:?subject='+enc(subj)+'&body='+enc(body);"
        "    window.location.href=href;"
        "  }); }"

        "  var btnPrint=document.getElementById('btn-print');"
        "  if(btnPrint){ btnPrint.addEventListener('click', function(){"
        "    var sel=selectedCodes(); if(!sel.length){ alert('No persons selected.'); return; }"
        "    for(var i=0;i<sel.length;i++){ toggleDetails(sel[i], true); }"
        "    setTimeout(function(){ window.print(); }, 100);"
        "  }); }"

        "  var tbl=document.getElementById('mc-table');"
        "  if(tbl){ tbl.addEventListener('click',function(ev){"
        "    var a=ev.target && ev.target.closest ? ev.target.closest('a.count-toggle') : null;"
        "    if(!a) return; ev.preventDefault();"
        "    var code=a.getAttribute('data-code'); if(!code) return;"
        "    var row=document.getElementById('details-'+code); if(!row) return;"
        "    var open=row.style.display!== 'none';"
        "    row.style.display = open ? 'none' : '';"
        "    a.setAttribute('aria-expanded', open?'false':'true');"
        "    if(!open){ try{ row.scrollIntoView({behavior:'smooth',block:'start'});}catch(e){} }"
        "  },false); }"
        " });"
        "})();</script>"
        "</body></html>"
    )
    return html

match_partial_html = _render_match_count_full_html(usage_df_alpha)

# ---------- Lineage Count partial ----------
lineage_style = (
    "<style>"
    " body{font-family:Georgia,'Times New Roman',serif;background:#fff;color:#111;margin:0;padding:18px;line-height:1.6;font-size:15px}"
    " .wrap{max-width:1100px;margin:0 auto}"
    " h2{font-size:20px;margin:0 0 10px 0;text-align:center;border-bottom:2px solid #5b79b8;padding-bottom:6px}"
    " .updated{font-size:12px;color:#555;text-align:center;margin:2px 0 10px 0}"
    " table.partial-table{border-collapse:collapse;width:100%;table-layout:auto}"
    " table.partial-table th,table.partial-table td{border:1px solid #ccc;padding:8px 10px;vertical-align:top}"
    " table.partial-table th{background:#e9eef9;position:sticky;top:0}"
    " @media screen and (max-width:900px){ body{padding:12px;font-size:14px} }"
    "</style>"
)
lineage_updated = (
    "<div class='updated'>"
    "<a href='" + HOME_URL + "' target='_blank' rel='noopener'>Home</a>"
    " &nbsp;|&nbsp; Last updated: <span id='last-updated'></span>"
    " &nbsp;|&nbsp; Autosomal matches: <span id='auto-count' class='js-count'></span>"
    "</div>"
)
lineage_js = (
    "<script>(function(){"
    " document.addEventListener('DOMContentLoaded', function(){"
    "  function z(n){return(n<10?'0':'')+n;}"
    "  function stamp(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date());"
    "   el.textContent=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());}"
    "  function fmt(n){try{var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US');}catch(e){return String(n||'');}}"
    "  function loadCnt(){var el=document.getElementById('auto-count'); if(!el) return; var url='" + JS_COUNT_URL + "';"
    "   try{var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);"
    "    xhr.onreadystatechange=function(){ if(xhr.readyState===4){ if(xhr.status>=200&&xhr.status<300){ var m=(xhr.responseText||'').match(/(\\d+)/); el.textContent = fmt(m?m[1]:'') || '(unavailable)'; } else { el.textContent='(unavailable)'; } } };"
    "    xhr.send(null);}catch(e){ el.textContent='(unavailable)'; }}"
    "  stamp(); loadCnt();"
    " });"
    "})();</script>"
)

lineage_partial_html = (
    "<!DOCTYPE html><html lang='en'><head>"
    "<meta charset='iso-8859-15' />"
    "<meta name='viewport' content='width=device-width, initial-scale=1.0' />"
    "<title>Lineage Count (by oldest distant ancestor pair)</title>"
    + lineage_style +
    "</head><body>"
    "<div class='wrap'>"
    "<h2>Lineage Count (by oldest distant ancestor pair)</h2>"
    + lineage_updated +
    lineage_count_df.to_html(index=False, escape=False, classes='partial-table', border=0) +
    "</div>"
    + lineage_js +
    "</body></html>"
)

# 4) Write & upload partials
with open("match_count.htm", "w", encoding=ENC, errors="xmlcharrefreplace") as f:
    f.write(match_partial_html)
with open("lineage_count.htm", "w", encoding=ENC, errors="xmlcharrefreplace") as f:
    f.write(lineage_partial_html)
print("✅ Wrote partials locally: match_count.htm, lineage_count.htm")

try:
    with ftp_connect() as ftps:
        try: ftps.mkd("partials")
        except Exception: pass
        ftp_upload_overwrite(ftps, "match_count.htm",   "partials/match_count.htm")
        ftp_upload_overwrite(ftps, "lineage_count.htm", "partials/lineage_count.htm")
        try: ftps.quit()
        except Exception: pass
    print("✅ Published count partials to /partials/")
except Exception as e:
    print(f"⚠️ Upload of count partials failed: {e}")

# 5) Printable page (adds dynamic header)
PRINT_PARTIAL_NAME  = "cousin_list_print.htm"
PRINT_PARTIAL_LOCAL = PRINT_PARTIAL_NAME
_print_table_html_df = (display_df_filtered if not display_df_filtered.empty else display_df_all).copy()
_print_table_html = _print_table_html_df[["Find", "Match To Name", LINEAGE_HEADER_SAFE]].to_html(
    index=False, escape=False, classes="print-table", border=1
)
PRINT_PARTIAL_HTML = (
    "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\"\n"
    "  \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">\n"
    "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\">\n"
    "<head>\n"
    "<meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />\n"
    "<meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\" />\n"
    "<title>ONS Yates Study &ndash; Cousin List (Printable)</title>\n"
    "<style type=\"text/css\">\n"
    "  body { font-family: Georgia, 'Times New Roman', serif; color:#000; background:#fff; margin:0; padding:16px; line-height:1.45; }\n"
    "  .wrap { max-width: 1100px; margin: 0 auto; }\n"
    "  h1 { font-size: 22px; margin: 0 0 8px 0; text-align:center; }\n"
    "  .updated { font-size:12px; color:#555; text-align:center; margin:2px 0 12px 0; }\n"
    "  .actions { text-align:center; margin: 10px 0 16px 0; }\n"
    "  .btn { display:inline-block; border:1px solid #444; background:#f4f4f4; color:#000; padding:6px 12px; text-decoration:none; }\n"
    "  table.print-table { border-collapse: collapse; width: 100%; table-layout: fixed; }\n"
    "  table.print-table th, table.print-table td { border:  1px solid #222; padding: 6px 8px; vertical-align: top; }\n"
    "  table.print-table th { background: #eee; }\n"
    "  @media print { .actions { display: none; } @page { margin: 0.6in; } }\n"
    "</style>\n"
    "</head>\n"
    "<body>\n"
    "  <div class=\"wrap\">\n"
    "    <h1>ONS Yates Study &ndash; Cousin List (Printable)</h1>\n"
    "    <div class=\"updated\"><a href=\"" + HOME_URL + "\" target=\"_blank\" rel=\"noopener\">Home</a>"
    " &nbsp;|&nbsp; Last updated: <span id=\"last-updated\"></span>"
    " &nbsp;|&nbsp; Autosomal matches: <span id=\"auto-count\" class=\"js-count\"></span></div>\n"
    "    <div class=\"actions\"><a class=\"btn\" href=\"#\" onclick=\"window.print();return false;\">Print this page</a></div>\n"
    "   " + _print_table_html + "\n"
    "    <div class=\"actions\" style=\"margin-top:16px;\"><a class=\"btn\" href=\"#\" onclick=\"window.print();return false;\">Print this page</a></div>\n"
    "  </div>\n"
    "<script>(function(){"
    " document.addEventListener('DOMContentLoaded', function(){"
    "  function z(n){return(n<10?'0':'')+n;}"
    "  function stamp(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date());"
    "   el.textContent=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());}"
    "  function fmt(n){try{var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US');}catch(e){return String(n||'');}}"
    "  function loadCnt(){var el=document.getElementById('auto-count'); if(!el) return; var url='" + JS_COUNT_URL + "';"
    "   try{var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);"
    "    xhr.onreadystatechange=function(){ if(xhr.readyState===4){ if(xhr.status>=200&&xhr.status<300){ var m=(xhr.responseText||'').match(/(\\d+)/); el.textContent = fmt(m?m[1]:'') || '(unavailable)'; } else { el.textContent='(unavailable)'; } } };"
    "    xhr.send(null);}catch(e){ el.textContent='(unavailable)'; }}"
    "  stamp(); loadCnt();"
    " });"
    "})();</script>"
    "</body>\n"
    "</html>\n"
)
with open(PRINT_PARTIAL_LOCAL, "w", encoding=ENC, errors="xmlcharrefreplace") as f:
    f.write(PRINT_PARTIAL_HTML)

try:
    with ftp_connect() as ftps:
        try: ftps.mkd("partials")
        except Exception: pass
        ftp_upload_overwrite(ftps, PRINT_PARTIAL_LOCAL, "partials/" + PRINT_PARTIAL_NAME)
        try: ftps.quit()
        except Exception: pass
    print(f"✅ Uploaded printable page: /partials/{PRINT_PARTIAL_NAME}")
except Exception as e:
    print(f"⚠️ Printable page upload failed: {e}")

print(f"✅ HTML published at https://yates.one-name.net/{REMOTE_NAME}")
# ====== CUT STOP [F] REFACTOR-Gold 3 — Save + Upload + Reports + Partials + Printable ===========












⬇️ Pulled remote file: partials/match_to_unmasked.csv → /content/match_to_unmasked.server.csv
✅ Loaded resolver from server: 79 codes
✅ Loaded CSV — 7 rows, 9 columns from /content/final_combined_df_with_value_labels.csv
✅ Columns: {'ID': 'ID#', 'Match to': 'Match to', 'Name': 'Name', 'cM': 'cM', 'Lineage': 'Yates DNA Ancestral Line'}
✅ Wrote local CSV (Column A): /content/the_match_cousins.csv
✅ Wrote HTML: /content/yates_ancestor_register.htm
⬆️ Uploaded: yates_ancestor_register.htm → yates_ancestor_register.htm
⬆️ Uploaded: the_match_cousins.csv → the_match_cousins.csv
⬆️ Uploaded: /content/autosomal_count.txt → autosomal_count.txt
✅ Published autosomal count: /content/autosomal_count.txt → /autosomal_count.txt
✅ Wrote resolver usage CSV: /content/resolver_usage_report.csv
✅ Wrote lineage count CSV: /content/lineage_count_report.csv
⬆️ Uploaded: resolver_usage_report.csv → resolver_usage_report.csv
⬆️ Uploaded: lineage_count_report.csv → lineage_count_report.csv
✅ Published report C

In [ ]:
## Gold #4 for Y-DNA Grid with Auto-Adjusting Column Widths

import os
import pandas as pd
from datetime import datetime
from zoneinfo import ZoneInfo
from ftplib import FTP_TLS

# ── PATHS ─────────────────────────────────────────────────────────────────
combo_csv  = "/content/y_dna_user_detail_combo.csv"
output_csv = "/content/y_dna_grid.csv"
output_htm = "/content/y_dna_grid.htm"

# ── 1) Load vertical data ─────────────────────────────────────────────────
df = pd.read_csv(combo_csv)

# Rename “Date” → “Era”
if "Date" in df.columns:
    df.rename(columns={"Date": "Era"}, inplace=True)

# ── 2) Insert Action *after* Era ──────────────────────────────────────────
# Era is at index 1, so Action goes at index 2
df.insert(2, "Action", ["→"] * len(df))

# ── 3) Save vertical CSV ─────────────────────────────────────────────────
df.to_csv(output_csv, index=False)
print(f"✅ Saved vertical grid CSV to {output_csv}")

# ── 4) Build HTML ─────────────────────────────────────────────────────────
now = datetime.now(ZoneInfo("America/New_York"))
ts  = now.strftime("%-m/%-d/%y, %-I:%M %p EDT")
cols = df.columns.tolist()

html = f"""<!DOCTYPE html>
<html>
<head><meta charset="UTF-8"><title>Yates Y-DNA Grid</title>
<style>
body {{
  background: #faf9d3;
  font-family: Arial, sans-serif;
  font-size: 14px;
  margin: 0;
  padding: 0;
}}
.container {{
  padding: 10px;
}}
.table-container {{
  overflow-x: auto;
  max-height: 80vh;
}}
table {{
  border: 2px solid #333;
  border-collapse: collapse;
  margin: 0 auto;
}}
table.mainsection {{
  /* allows CSS targeting of blank under “Year” */
}}
thead {{
  display: table-header-group;
}}
thead th {{
  position: sticky;
  top: 0;
  background: #333;
  color: #fff;
  padding: 6px;
  border: 1px solid #999;
  z-index: 3;
}}
a {{
  color: #fff;
  text-decoration: underline;
}}
.era {{
  background: #666;
  color: #eee;
  padding: 6px;
  border: 1px solid #999;
  font-size: 0.9em;
}}
.action {{
  background: #fff;
  padding: 6px;
  border: 1px solid #999;
  text-align: center;
}}
td {{
  padding: 6px;
  border: 1px solid #999;
  text-align: center;
}}
th:nth-child(n+4),
td:nth-child(n+4) {{
  border: 1px solid #333;
}}
.match {{
  background: #fff;
}}
.blank {{
  background: #ccc;
  color: #ccc;
}}
/* make the blank under the “Year” header match the era-cell background */
table.mainsection td.blank:nth-child(2) {{
  background-color: #fdfcd0;
}}
</style>
</head>
<body>
  <div class="container">
    <h1 style="text-align:center">Yates Y-DNA Grid</h1>
    <p style="text-align:center;font-size:0.9em">Updated: {ts}</p>
    <p style="text-align:center;margin-bottom:12px">
      <a href="https://yates.one-name.net/gengen/dna_cousin_surname_study.htm">
        Return to DNA Cousin Surname Study
      </a>
    </p>
    <div class="table-container">
      <table class="mainsection">
        <thead>
          <tr>"""

# Header row
for i, c in enumerate(cols):
    if i == 0:
        html += "<th>SNP</th>"
    elif i == 1:
        html += "<th>Year</th>"
    elif i == 2:
        html += "<th>Action</th>"
    else:
        pid = c.split("-")[0].upper()
        html += (
          '<th>'
          f'<a href="https://yates.one-name.net/tng/verticalchart.php?'
          f'personID={pid}&tree=tree1&parentset=0&display=vertical&generations=15">{c}</a>'
          '</th>'
        )

html += """
          </tr>
        </thead>
        <tbody>"""

# Data rows
for _, row in df.iterrows():
    html += "<tr>"
    for i, c in enumerate(cols):
        v = row[c]
        if i == 0:
            html += f"<td>{v}</td>"
        elif i == 1:
            html += '<td class="blank">–</td>' if pd.isna(v) or not str(v).strip() else f'<td class="era">{v}</td>'
        elif i == 2:
            html += '<td class="blank">–</td>' if pd.isna(v) or not str(v).strip() else f'<td class="action">{v}</td>'
        else:
            html += '<td class="blank">–</td>' if pd.isna(v) or not str(v).strip() else f'<td class="match">{v}</td>'
    html += "</tr>"

html += """
        </tbody>
      </table>
    </div>
  </div>
</body>
</html>"""

with open(output_htm, "w", encoding="utf-8") as f:
    f.write(html)
print(f"✅ Saved vertical XHTML to {output_htm}")

# ── 5) FTP upload ────────────────────────────────────────────────────────
ftp = FTP_TLS()
ftp.connect(os.environ["FTP_HOST"], int(os.environ["FTP_PORT"]))
ftp.login(os.environ["FTP_USER"], os.environ["FTP_PASS"])
ftp.prot_p()
for path in (output_csv, output_htm):
    fn = os.path.basename(path)
    try:
        ftp.delete(fn)
    except:
        pass
    with open(path, "rb") as fp:
        ftp.storbinary(f"STOR {fn}", fp)
ftp.quit()
print("✅ Uploaded CSV & HTML to server")



FileNotFoundError: [Errno 2] No such file or directory: '/content/y_dna_user_detail_combo.csv'

In [ ]:
## EXP

import os
import pandas as pd
from datetime import datetime
from zoneinfo import ZoneInfo
from ftplib import FTP_TLS

# ── CONFIG ───────────────────────────────────────────────────────────────
info_csv   = "/content/haplogroup_info.csv"
user_csv   = "/content/y_dna_user_detail.csv"
output_csv = "/content/y_dna_grid.csv"
output_htm = "/content/y_dna_grid.htm"

# ── 1) Load & prepare haplogroup info ───────────────────────────────────
df_info = pd.read_csv(info_csv)
if "Date" in df_info.columns:
    df_info.rename(columns={"Date": "Era"}, inplace=True)
df_info = df_info.loc[df_info["Haplogroup"].drop_duplicates().index]
hap_order = df_info["Haplogroup"].tolist()
era_map   = dict(zip(df_info["Haplogroup"], df_info.get("Era", [""] * len(df_info))))

# ── 2) Load user detail table ───────────────────────────────────────────
df_users = pd.read_csv(user_csv)
if "User_ID" not in df_users.columns:
    df_users.rename(columns={df_users.columns[0]: "User_ID"}, inplace=True)
user_chains = [
    [str(v) for v in row.drop(labels=["User_ID"]).tolist() if pd.notna(v) and str(v).strip()]
    for _, row in df_users.iterrows()
]

# ── 3) Insert new SNPs after parent ──────────────────────────────────────
for chain in user_chains:
    prev = None
    for h in chain:
        if prev and h not in hap_order:
            idx = hap_order.index(prev)
            hap_order.insert(idx + 1, h)
        prev = h
# Build final eras list
eras = [era_map.get(h, "") for h in hap_order]

# ── 4) Build horizontal grid DataFrame ───────────────────────────────────
for h in hap_order:
    if h not in df_users.columns:
        df_users[h] = ""
df_grid_h = df_users[["User_ID"] + hap_order]

# ── 5) Transform to vertical layout ─────────────────────────────────────
df_vert = df_grid_h.set_index("User_ID").T
# Insert Era as first column
df_vert.insert(0, 'Era', eras)
df_vert.index.name = 'SNP'
df_grid = df_vert.reset_index()

# ── 6) Save vertical CSV ─────────────────────────────────────────────────
df_grid.to_csv(output_csv, index=False)
print(f"✅ Vertical grid CSV saved to {output_csv}")

# ── 7) Generate XHTML (vertical) ────────────────────────────────────────
now = datetime.now(ZoneInfo("America/New_York"))
ts  = now.strftime("%-m/%-d/%y, %-I:%M %p EDT")

template = '''<!DOCTYPE html>
<html><head><meta charset="UTF-8"><title>Yates Y-DNA Grid</title>
<style>
  body { background:#faf9d3; font-family:Arial,Helvetica,sans-serif; font-size:14px; }
  table { width:100%; border:1px solid #333; border-collapse:collapse; table-layout:auto; }
  th { background:#333; color:#fff; padding:6px; border:1px solid #999; }
  .era { background:#666; color:#eee; padding:6px; border:1px solid #999; font-size:0.9em; }
  td { padding:6px; border:1px solid #999; text-align:center; white-space:nowrap; }
  .match { background:#fff; }
  .blank { background:#ccc; color:#ccc; }
</style>
</head><body>
  <h1 style="text-align:center;">Yates Y-DNA Grid</h1>
  <table>
'''  # end template

# Build header row
cols = df_grid.columns.tolist()
header_html = '<tr><th>SNP</th><th>Era</th>' + ''.join(f'<th>{u}</th>' for u in cols[2:]) + '</tr>'

# Build data rows
rows_html = []
for _, row in df_grid.iterrows():
    cells = []
    for u in cols[2:]:
        v = row[u]
        if pd.isna(v) or not str(v).strip():
            cells.append('<td class="blank">–</td>')
        else:
            cells.append(f'<td class="match">{v}</td>')
    rows_html.append(f'<tr><td>{row["SNP"]}</td><td class="era">{row["Era"]}</td>' + ''.join(cells) + '</tr>')

# Combine and save HTML
html = template + header_html + '\n' + '\n'.join(rows_html) + f'''
  </table>
  <p style="text-align:right;font-size:0.9em;">Updated: {ts}</p>
</body>
</html>'''
with open(output_htm, 'w', encoding='utf-8') as f:
    f.write(html)
print(f"✅ Vertical XHTML Grid saved to {output_htm}")

# ── 8) FTP Upload ───────────────────────────────────────────────────────
ftp = FTP_TLS()
ftp.connect(os.environ['FTP_HOST'], int(os.environ.get('FTP_PORT',21)))
ftp.login(os.environ['FTP_USER'], os.environ['FTP_PASS'])
ftp.prot_p()
for path in [output_csv, output_htm]:
    name = os.path.basename(path)
    try: ftp.delete(name)
    except: pass
    with open(path,'rb') as fp:
        ftp.storbinary(f"STOR {name}", fp)
ftp.quit()
print("✅ Uploaded to server.")



✅ Vertical grid CSV saved to /content/y_dna_grid.csv
✅ Vertical XHTML Grid saved to /content/y_dna_grid.htm
✅ Uploaded to server.


In [ ]:
# Y-DNA cell 1

# === Cell 1: New user settings ===
USER_ID       = 'I56217'  # the new column header
PATH_STRING   = (      # the SNP chain for this user
    "R-M207 > R-M173 > R-M343 > R-M269 > R-FT266064 > R-FT266579 > R-FTF17042"
)
INSERT_MISSING = True       # if True, adds any SNPs from PATH_STRING that aren't yet rows
MASTER_CSV     = '/content/y_dna_user_detail_combo.csv'
UPDATED_CSV    = '/content/y_dna_user_detail_combo_updated.csv'


In [ ]:
# Cell 2: Load → Append User → Save

import pandas as pd

# 1) Load the existing master CSV
df = pd.read_csv(MASTER_CSV)

# 2) Normalize the first column name to 'SNP' for easy matching
first_col = df.columns[0]
if first_col != 'SNP':
    df.rename(columns={first_col: 'SNP'}, inplace=True)

# 3) Parse the new user's SNP chain
chain = PATH_STRING.split('>')

# 4) Optionally insert any SNPs not yet present (appends at bottom)
if INSERT_MISSING:
    missing = [s for s in chain if s not in df['SNP'].values]
    if missing:
        df = pd.concat([df, pd.DataFrame([{'SNP': s} for s in missing])],
                       ignore_index=True)

# 5) Create the new user column in the next free position
df[USER_ID] = ''

# 6) Populate: copy the SNP value into that column where it matches the chain
df.loc[df['SNP'].isin(chain), USER_ID] = df['SNP']

# 7) Save the updated CSV back to /content
df.to_csv(UPDATED_CSV, index=False)
print(f"✅ Updated CSV saved to {UPDATED_CSV}")


✅ Updated CSV saved to /content/y_dna_user_detail_combo_updated.csv
